In [115]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import math
import matplotlib.pyplot as plt
import warnings
import scorecardpy as sc
warnings.filterwarnings("ignore")
%matplotlib inline

In [116]:
train = pd.read_csv('../../../train.csv')
test = pd.read_csv('../../../test.csv')
submit = pd.read_csv('../../../submit.csv')
train.shape, test.shape, submit.shape

((40000, 21), (15000, 20), (15000, 2))

In [117]:
df = pd.concat([train, test], axis = 0).reset_index(drop = True)
#ZHIWU=1数据无意义，剔除该类数据
df = df[df['HYZK'] ==90 ]
df = df[(df['ZHIYE']==90) | (df['ZHIYE']==13) | (df['ZHIYE']==24) | (df['ZHIYE']==17) ]
df = df[(df['ZHICHEN']==999 )|( df['ZHICHEN']==8)  ]
df = df[df['XUELI']==99 ]
df = df[df['DWJJLX']!=390 ]
del df['ZHIWU']
del df['XUELI']
del df['HYZK']
del df['DWYJCE']

In [118]:
#对数值类的进行转化：
df['GRYJCE'] = df['GRYJCE'] - 237
df['GRJCJS'] = df['GRJCJS'] -237 
df['GRZHYE'] = df['GRZHYE'] - 237
df['GRZHSNJZYE'] = df ['GRZHSNJZYE'] -237
df['GRZHDNGJYE'] = df ['GRZHDNGJYE'] -237
df['DKFFE'] = df['DKFFE'] - 237
df['DKYE'] = df['DKYE']-237

# 离群值处理  -- 有用
df['GRZHYE'] = df['GRZHYE'].apply(lambda x: 200000 if x>200000 else x)
df['GRZHSNJZYE'] = df['GRZHSNJZYE'].apply(lambda x: 200000 if x>200000 else x)
df['GRZHDNGJYE'] = df['GRZHDNGJYE'].apply(lambda x: 30000 if x>30000 else -30000 if x<-30000 else x)

In [119]:
cate_cols = [ 'DWJJLX', 'DWSSHY', 'GRZHZT','XINGBIE']
cate_cols0 = [ 'DWJJLX', 'DWSSHY']

In [120]:
num_cols = ['GRJCJS', 'GRZHYE', 'GRZHSNJZYE', 'GRZHDNGJYE', 'GRYJCE','DKFFE', 'DKYE', 'DKLL']

特征工程

In [121]:
df['missing_rate'] = (df.shape[1] - df.count(axis = 1)) / df.shape[1]


#鬼知道，可能反应收入？
df['YEARPURINCM'] = df['GRZHYE']-df['GRZHSNJZYE']-df['GRZHDNGJYE']
#应还贷款月份，剩余需还贷款月份,已还贷款月份
df['HDZYF'] = df['DKFFE']*(df['DKLL']/100+1)/(df['GRYJCE']*2)
df['HDSYYF'] = df['DKYE']*(df['DKLL']/100+1)/(df['GRYJCE']*2)
df['HDYF'] = (df['DKFFE']-df['DKYE'])/(df['GRYJCE']*2)

#应还总额
df['HDZYFZE'] = df['DKFFE']*(df['DKLL']/100+1)
#待还总额
df['HDSYYFZE'] = df['DKYE']*(df['DKLL']/100+1)

#特征工程衍生的代码
df ['USAGE_RATE']=(df['DKYE']/df['DKFFE'])
df ['USAGE_AMOUNT'] = (df['DKFFE']-df['DKYE'])*(1+df['DKLL']/100)
df ['GJJRATE'] = (df['GRYJCE']/df['GRJCJS']+0.005)//0.01*0.01


# 每月提取额与缴存额的比例
df ['GED_GRY'] = df['GRZHDNGJYE']/df['GRYJCE']
# 提取还是存储？
df ['WITHSIGN'] =  df ['GED_GRY'].apply(lambda x: 0 if x%2 ==0 else 1 ) 
df ['WITHSIGN2'] =  df ['GED_GRY'].apply(lambda x: 1 if x%2 ==0 and x<16 and x>0 else -1 ) 
# 平均每月提取额
df ['WITHDRAW'] =  (df ['GRYJCE']*4*2 - df['GRZHDNGJYE'])*df['WITHSIGN']
df ['WITHSIGN'] = df['WITHSIGN']* (df ['WITHDRAW'].apply(lambda x: 1 if x>0 else 0 if x==0 else -1))
df ['WITHDRAW_SUBMIT'] = df ['WITHDRAW']/df['GRYJCE']

df ['GJSIGN']   =  df ['GRZHDNGJYE'].apply(lambda x: 1 if x>0 else 0 if x==0 else -1  )
df['age'] = ((1609430399 - df['CSNY']) / (365 * 24 * 3600)).astype(int)
df ['DKXE'] = df['GRJCJS']*0.45*(70-df['age'])*12

# 上年结转与余额的关系？
# 上年结转+公积金缴纳+(个人补缴-提取)=当年余额
# 个人净提取额
df ['WITHDRAW2'] =  df['GRZHSNJZYE'] + df ['GRYJCE']*4*2 - df['GRZHYE']

df['USAGEOVER'] = df['DKYE'] == 0 
df['NOUSAGE'] = df['DKYE'] == df['DKFFE']

df ['TOP_USER'] = df['DKFFE'].apply(lambda x: x if (x%50000==0 or x==125000) else 0)
df ['TOP_USER1'] = df['DKFFE'].apply(lambda x: 1 if (x%50000==0 or x==125000) else x)


df.loc[(df['GRZHZT']!=1)&(df['GRZHZT']!=6),'GRZHZT']=-1

df.loc[(df['DWJJLX'].isin(['310','330','390','210','230','171','172','173','174','175','142','143','149','140'])),'DWJJLX']=-1
df.loc[(df['DWJJLX'].isin(['100','179','900'])),'DWJJLX']=-2
df.loc[(df['DWJJLX'].isin(['120','160'])),'DWJJLX']=-3
df.loc[(df['DWJJLX'].isin(['200','150','141'])),'DWJJLX']=-4
df.loc[(df['DWJJLX'].isin(['159','300','130'])),'DWJJLX']=-5

gen_feats = [ 'YEARPURINCM','HDZYF',
            'HDSYYF','HDYF','HDZYFZE','HDSYYFZE','USAGE_RATE','GJJRATE',
             'TOP_USER','TOP_USER1','WITHDRAW','USAGE_AMOUNT','WITHDRAW_SUBMIT','DKXE'
            ]
 
cate_cols += ['USAGEOVER','NOUSAGE','TOP_USER','WITHSIGN','GJJRATE','GJSIGN','WITHSIGN2','ZHIYE','ZHICHEN'] #'DKZS','YEZS','ZHZS','GJZS','SNZS','YJZS',

In [122]:
df['DWJJLX'].value_counts()

-4      20973
 110    14524
-2       7617
-3       4427
-5       2529
 170     2520
 190     1769
-1        621
Name: DWJJLX, dtype: int64

In [123]:
import time
def get_age(df,col = 'age'):
    df[col+"_genFeat1"]=(df['age'] > 18).astype(int)
    df[col+"_genFeat2"]=(df['age'] > 25).astype(int)
    df[col+"_genFeat3"]=(df['age'] > 30).astype(int)
    df[col+"_genFeat4"]=(df['age'] > 35).astype(int)
    df[col+"_genFeat5"]=(df['age'] > 40).astype(int)
    df[col+"_genFeat6"]=(df['age'] > 45).astype(int)
    return df, [col + f'_genFeat{i}' for i in range(1, 7)]

# BORN MONTH
tNow = time.mktime(time.strptime( '2021-01-12 00:00:00','%Y-%m-%d %H:%M:%S'))
df['bornmonth'] = df['CSNY'].apply(lambda x: time.localtime(x)[1])

df, genFeats1 = get_age(df, col = 'age')


In [124]:
def get_daikuanYE(df,col):
    df[col + '_genFeat1'] = (df[col] > 100000).astype(int)
    df[col + '_genFeat2'] = (df[col] > 120000).astype(int)
    df[col + '_genFeat3'] = (df[col] > 140000).astype(int)
    df[col + '_genFeat4'] = (df[col] > 180000).astype(int)
    df[col + '_genFeat5'] = (df[col] > 220000).astype(int)
    df[col + '_genFeat6'] = (df[col] > 260000).astype(int)
    df[col + '_genFeat7'] = (df[col] > 300000).astype(int)
    return df, [col + f'_genFeat{i}' for i in range(1, 8)]

df, genFeats2 = get_daikuanYE(df, col = 'DKYE')
df, genFeats3 = get_daikuanYE(df, col = 'DKFFE')
#genFeats2 = []
#genFeats3 = []

In [125]:
from tqdm import tqdm

In [126]:
drop_feature = ['--QIANGXIANGGUANXISHU_QUEREN','GRZHZT_DKYE_genFeat6_mean', 'numsOf_YEARPURINCM_USAGE_RATE_diff', 'WITHSIGN_GRZHSNJZYE_std', 'GJJRATE_GRZHYE_mean', 'WITHSIGN_GJJRATE_mean', 'GJSIGN_HDSYYF_max', 'numsOf_GRZHSNJZYE_USAGE_RATE_diff', 'TOP_USER_TOP_USER1_max', 'numsOf_GRZHSNJZYE_DKYE_div', 'TOP_USER_DKFFE_genFeat2_sum', 'GJJRATE_DKFFE_genFeat2_sum', 'WITHSIGN_GRYJCE_mean', 'HDYF_DKYE_mean', 'GRZHZT_TOP_USER_std', 'GRZHZT_WITHDRAW_SUBMIT_max', 'NOUSAGE_TOP_USER1_max', 'TOP_USER_HDYF_min', 'GRZHZT_TOP_USER1_std', 'DWJJLX_DKYE_genFeat2_mean', 'numsOf_YEARPURINCM_HDZYF_add', 'TOP_USER1_GJJRATE_sum', 'NOUSAGE_GRZHSNJZYE_mean', 'USAGE_AMOUNT_DKFFE_mean', 'GJJRATE_DKYE_genFeat3_mean', 'HDSYYF_DKFFE_max', 'DKLL_HDZYF_sum', 'NOUSAGE_GRZHYE_std', 'WITHSIGN_GRZHDNGJYE_std', 'GRZHZT_HDSYYF_sum', 'USAGEOVER_YEARPURINCM_max', 'GJJRATE_DKFFE_mean', 'GRZHZT_DKYE_genFeat3_mean', 'WITHSIGN_DKLL_std', 'TOP_USER_DKYE_genFeat5_mean', 'WITHDRAW_SUBMIT_DKFFE_sum', 'NOUSAGE_GRZHSNJZYE_std', 'GRZHDNGJYE', 'numsOf_GRZHDNGJYE_HDZYF_add', 'GRZHSNJZYE_TOP_USER_sum', 'TOP_USER_DKYE_genFeat7_mean', 'numsOf_GRZHYE_DKLL_diff', 'WITHSIGN_GRJCJS_min', 'numsOf_GRZHSNJZYE_HDYF_diff', 'USAGEOVER_YEARPURINCM_min', 'TOP_USER_DKFFE_genFeat4_sum', 'GRZHZT_DKLL_sum', 'USAGEOVER_GRZHSNJZYE_mean', 'YEARPURINCM_GRYJCE_mean', 'numsOf_DKLL_HDYF_diff', 'GRZHZT_GRZHYE_sum', 'GJJRATE_HDZYFZE_sum', 'USAGEOVER_GRJCJS_std', 'GJJRATE_HDSYYFZE_min', 'USAGEOVER_HDSYYFZE_min', 'numsOf_DKYE_USAGE_RATE_add', 'DWSSHY_DKYE_genFeat4_sum', 'WITHSIGN2_DKYE_genFeat5_sum', 'USAGEOVER_DKYE_genFeat7_sum', 'GJJRATE_DKFFE_genFeat3_sum', 'numsOf_GRYJCE_WITHDRAW_SUBMIT_multi', 'numsOf_HDYF_GJJRATE_add', 'WITHDRAW_SUBMIT_GRZHYE_sum', 'YEARPURINCM_DKFFE_sum', 'numsOf_HDSYYF_TOP_USER_add', 'GJJRATE_GRZHYE_sum', 'WITHSIGN_DKFFE_std', 'WITHSIGN_USAGE_AMOUNT_sum', 'NOUSAGE_DKLL_max', 'GRZHZT_HDSYYF_max', 'numsOf_GRZHSNJZYE_HDYF_add', 'numsOf_GJJRATE_TOP_USER1_add', 'numsOf_HDYF_USAGE_AMOUNT_add', 'HDYF_DKFFE_sum', 'GJSIGN_WITHDRAW_std', 'DKLL_USAGE_RATE_sum', 'TOP_USER_TOP_USER1_std', 'USAGE_AMOUNT_DKFFE_max', 'USAGE_RATE', 'numsOf_GRJCJS_DKYE_add', 'GRZHZT_GRZHDNGJYE_std', 'HDSYYF_USAGE_RATE_min', 'TOP_USER1_TOP_USER_mean', 'numsOf_DKLL_HDSYYFZE_diff', 'NOUSAGE_DKYE_genFeat3_sum', 'NOUSAGE_GRYJCE_std', 'WITHSIGN_DKYE_genFeat6_sum', 'DWSSHY_DKFFE_genFeat5_mean', 'TOP_USER_DKFFE_genFeat5_mean', 'WITHSIGN2_TOP_USER1_max', 'TOP_USER_HDZYFZE_min', 'HDYF_HDZYFZE_sum', 'numsOf_HDSYYF_HDSYYFZE_add', 'GRZHZT_0', 'GJJRATE_GRJCJS_sum', 'TOP_USER_DKYE_sum', 'HDYF_GRZHDNGJYE_sum', 'numsOf_HDZYFZE_HDSYYFZE_diff', 'WITHSIGN2_GRJCJS_max', 'HDSYYF_DKLL_sum', 'NOUSAGE_GRJCJS_std', 'WITHSIGN_HDZYFZE_mean', 'YEARPURINCM_GJJRATE_mean', 'TOP_USER1_GRZHYE_sum', 'USAGEOVER_DKFFE_genFeat1_mean', 'USAGEOVER_WITHDRAW_SUBMIT_mean', 'USAGE_RATE_DKFFE_mean', 'NOUSAGE_HDYF_mean', 'DWSSHY_HDYF_min', 'WITHSIGN2_GRZHSNJZYE_mean', 'USAGEOVER_DKLL_sum', 'DKLL_WITHDRAW_sum', 'numsOf_GRZHYE_USAGE_RATE_diff', 'GJJRATE_DKFFE_genFeat3_mean', 'numsOf_HDZYF_TOP_USER_diff', 'WITHSIGN_DKYE_mean', 'TOP_USER1_DKFFE_max', 'numsOf_DKYE_USAGE_RATE_diff', 'USAGEOVER_GJJRATE_std', 'numsOf_DKYE_HDSYYF_add', 'GRZHSNJZYE_DKYE_mean', 'USAGEOVER_GRZHSNJZYE_min', 'WITHSIGN2_HDSYYFZE_max', 'WITHSIGN_DKLL_mean', 'DKLL', 'HDSYYFZE_DKFFE_mean', 'numsOf_HDSYYF_TOP_USER_diff', 'GRZHZT_DKYE_min', 'GRZHZT_GRZHSNJZYE_std', 'WITHSIGN_DKFFE_genFeat6_mean', 'NOUSAGE_DKYE_max', 'GJSIGN_GRYJCE_max', 'NOUSAGE_DKYE_genFeat3_mean', 'WITHSIGN2_DKFFE_std', 'USAGEOVER_HDZYF_min', 'USAGEOVER_TOP_USER_sum', 'NOUSAGE_0', 'WITHSIGN2_GJJRATE_sum', 'WITHSIGN_DKFFE_genFeat2_sum', 'NOUSAGE_WITHDRAW_SUBMIT_mean', 'WITHSIGN_DKYE_genFeat4_mean', 'WITHSIGN_USAGE_RATE_std', 'DKFFE_USAGE_RATE_sum', 'GJSIGN_DKYE_std', 'USAGEOVER_TOP_USER1_max', 'WITHSIGN2_DKFFE_min', 'USAGE_AMOUNT_DKYE_mean', 'HDYF_USAGE_RATE_sum', 'GJSIGN_HDZYFZE_sum', 'WITHDRAW_SUBMIT_HDZYF_max', 'USAGEOVER_HDSYYFZE_mean', 'WITHSIGN2_DKYE_max', 'numsOf_HDSYYF_GJJRATE_diff', 'GJSIGN_DKFFE_min', 'numsOf_DKYE_TOP_USER1_div', 'USAGEOVER_DKYE_std', 'NOUSAGE_DKFFE_mean', 'GJJRATE_HDZYFZE_max', 'numsOf_GRYJCE_DKYE_div', 'numsOf_TOP_USER_TOP_USER1_multi', 'HDSYYFZE_DKYE_min', 'TOP_USER1_TOP_USER_min', 'USAGEOVER_DKFFE_genFeat4_sum', 'GJJRATE_in_GRZHZT_prop', 'numsOf_GRZHYE_YEARPURINCM_diff', 'TOP_USER_TOP_USER_sum', 'WITHSIGN2_DKYE_genFeat4_sum', 'USAGE_AMOUNT', 'WITHSIGN2_HDYF_mean', 'WITHSIGN_TOP_USER1_sum', 'numsOf_GRYJCE_DKYE_add', 'YEARPURINCM_DKFFE_mean', 'GRZHZT_HDZYF_min', 'GRZHZT_YEARPURINCM_sum', 'NOUSAGE_DKYE_genFeat1_sum', 'WITHSIGN_HDZYF_std', 'numsOf_HDZYF_HDYF_diff', 'USAGEOVER_DKFFE_genFeat3_mean', 'WITHSIGN_GRZHYE_std', 'WITHSIGN2_HDSYYFZE_mean', 'numsOf_HDZYF_HDZYFZE_diff', 'GRZHZT_TOP_USER_sum', 'numsOf_GRZHSNJZYE_HDSYYF_diff', 'GRZHZT_DKFFE_genFeat7_sum', 'numsOf_HDSYYFZE_WITHDRAW_SUBMIT_add', 'numsOf_GRYJCE_USAGE_RATE_add', 'NOUSAGE_YEARPURINCM_mean', 'numsOf_DKFFE_GJJRATE_diff', 'DKYE_HDSYYFZE_mean', 'GJJRATE', 'HDZYFZE_TOP_USER1_max', 'GRZHZT_HDSYYF_min', 'numsOf_GRJCJS_USAGE_RATE_add', 'GJSIGN_GRYJCE_std', 'NOUSAGE_USAGE_RATE_max', 'WITHDRAW_SUBMIT_DKFFE_min', 'numsOf_DKLL_HDZYFZE_add', 'numsOf_YEARPURINCM_GJJRATE_diff', 'NOUSAGE_DKFFE_genFeat1_mean', 'numsOf_HDYF_USAGE_RATE_diff', 'NOUSAGE_GRZHYE_sum', 'HDZYFZE_USAGE_RATE_sum', 'HDZYFZE_DKYE_min', 'NOUSAGE_HDYF_sum', 'NOUSAGE_USAGE_AMOUNT_std', 'GRZHSNJZYE_HDSYYFZE_sum', 'USAGEOVER_DKYE_genFeat1_sum', 'USAGEOVER_1', 'USAGEOVER_USAGE_AMOUNT_sum', 'WITHSIGN_DKFFE_genFeat3_sum', 'numsOf_DKFFE_DKYE_diff', 'USAGEOVER_HDSYYF_mean', 'DKFFE_HDZYF_sum', 'numsOf_GJJRATE_TOP_USER1_diff', 'TOP_USER_TOP_USER_min', 'WITHSIGN2_DKYE_genFeat3_sum', 'USAGE_AMOUNT_DKFFE_min', 'GJSIGN_GRZHYE_std', 'NOUSAGE_DKYE_genFeat2_sum', 'numsOf_GRZHSNJZYE_TOP_USER_add', 'GJSIGN_GRYJCE_min', 'numsOf_HDZYF_HDSYYFZE_diff', 'TOP_USER_DKFFE_genFeat5_sum', 'numsOf_DKFFE_DKLL_multi', 'USAGEOVER_HDSYYFZE_std', 'numsOf_GRJCJS_DKLL_diff', 'USAGEOVER_HDZYF_max', 'TOP_USER1_GRZHSNJZYE_sum', 'NOUSAGE_HDZYFZE_max', 'numsOf_DKLL_HDSYYFZE_add', 'WITHSIGN2_YEARPURINCM_std', 'WITHSIGN_HDZYF_sum', 'WITHSIGN2_USAGE_AMOUNT_mean', 'WITHSIGN2_GRZHYE_std', 'GJSIGN_GRZHYE_sum', 'NOUSAGE_TOP_USER_sum', 'GRZHZT_HDZYF_max', 'TOP_USER1_HDSYYFZE_sum', 'GRZHZT_HDYF_max', 'HDSYYF', 'WITHSIGN_DKFFE_genFeat4_mean', 'WITHSIGN_TOP_USER1_mean', 'HDSYYF_USAGE_AMOUNT_sum', 'WITHSIGN_TOP_USER_sum', 'USAGEOVER_DKYE_genFeat4_mean', 'TOP_USER_DKFFE_genFeat4_mean', 'WITHSIGN2_DKYE_sum', 'YEARPURINCM_GRZHSNJZYE_max', 'GJSIGN_GRJCJS_max', 'GJJRATE_DKYE_genFeat7_mean', 'GRZHYE_HDZYFZE_sum', 'TOP_USER_DKYE_genFeat4_sum', 'DWSSHY_DKYE_genFeat2_mean', 'NOUSAGE_GRZHDNGJYE_std', 'WITHSIGN_HDYF_std', 'NOUSAGE_DKYE_genFeat1_mean', 'USAGEOVER_DKFFE_genFeat2_sum', 'GRZHZT_DKFFE_genFeat3_mean', 'GRZHYE_TOP_USER_sum', 'GJSIGN_HDZYF_max', 'NOUSAGE_DKFFE_genFeat4_sum', 'WITHSIGN_DKYE_genFeat6_mean', 'NOUSAGE_DKYE_genFeat7_mean', 'WITHSIGN2_GRJCJS_std', 'numsOf_HDSYYFZE_GJJRATE_add', 'GRZHZT_DKFFE_genFeat4_sum', 'DWSSHY_DKFFE_genFeat1_sum', 'numsOf_GRZHDNGJYE_GJJRATE_diff', 'TOP_USER1', 'HDYF_DKLL_sum', 'GJJRATE_DKFFE_genFeat4_sum', 'WITHSIGN2_DKYE_genFeat6_mean', 'WITHSIGN2_GRZHSNJZYE_sum', 'WITHSIGN2_HDSYYF_std', 'WITHSIGN2_TOP_USER1_sum', 'HDSYYFZE_DKFFE_min', 'NOUSAGE_DKFFE_genFeat3_sum', 'GJJRATE_DKYE_genFeat1_sum', 'DKYE_genFeat5', 'GJSIGN_HDSYYF_mean', 'GRZHZT_HDYF_sum', 'WITHDRAW_SUBMIT_WITHDRAW_min', 'numsOf_DKYE_HDZYFZE_add', 'WITHSIGN2_DKLL_max', 'HDYF_GJJRATE_sum', 'DWJJLX_HDZYFZE_sum', 'GJSIGN_DKYE_genFeat3_sum', 'GRZHZT_WITHDRAW_sum', 'numsOf_DKFFE_TOP_USER1_div', 'GRZHZT_DKFFE_genFeat1_sum', 'USAGEOVER_HDZYF_sum', 'DKFFE_TOP_USER_max', 'USAGEOVER_GRZHDNGJYE_mean', 'numsOf_DKYE_HDSYYFZE_add', 'TOP_USER_DKYE_mean', 'GRZHZT_GRYJCE_std', 'GJJRATE_WITHDRAW_sum', 'WITHDRAW_SUBMIT_HDZYFZE_sum', 'USAGEOVER_DKFFE_genFeat6_sum', 'DKFFE_HDSYYFZE_min', 'USAGEOVER_DKYE_genFeat2_sum', 'numsOf_DKFFE_HDZYFZE_add', 'WITHSIGN2_USAGE_AMOUNT_std', 'numsOf_DKFFE_DKYE_div', 'HDSYYFZE_USAGE_RATE_max', 'WITHSIGN2_WITHDRAW_std', 'DWSSHY_DKYE_genFeat5_sum', 'USAGEOVER_GJJRATE_sum', 'DKYE_genFeat2', 'WITHSIGN2_WITHDRAW_mean', 'DWJJLX_DKYE_genFeat4_sum', 'DKYE_genFeat4', 'TOP_USER1_DKYE_mean', 'WITHSIGN_DKFFE_genFeat7_mean', 'GRZHZT_in_ZHICHEN_prop', 'USAGEOVER_GRZHDNGJYE_sum', 'numsOf_HDZYF_USAGE_RATE_add', 'GJJRATE_DKYE_genFeat6_mean', 'GJSIGN_TOP_USER_std', 'WITHSIGN2_WITHDRAW_SUBMIT_std', 'WITHSIGN_DKYE_genFeat2_sum', 'numsOf_GRJCJS_TOP_USER_add', 'DWJJLX_DKYE_genFeat6_mean', 'NOUSAGE_USAGE_AMOUNT_min', 'USAGEOVER_HDYF_min', 'numsOf_HDZYF_HDSYYFZE_multi', 'DKLL_DKFFE_min', 'GRZHZT_WITHDRAW_max', 'GJSIGN_DKFFE_genFeat1_sum', 'GRZHZT_WITHDRAW_min', 'GJJRATE_DKYE_genFeat7_sum', 'DWSSHY_DKYE_sum', 'DWJJLX_DKYE_sum', 'GRZHZT_GRZHDNGJYE_mean', 'DKLL_GRZHSNJZYE_sum', 'WITHSIGN2_GRZHDNGJYE_std', 'GJJRATE_DKLL_max', 'USAGEOVER_HDSYYFZE_max', 'GJSIGN_DKYE_genFeat6_mean', 'DKFFE_DKYE_min', 'WITHSIGN2_HDYF_sum', 'GJSIGN_HDZYFZE_std', 'USAGEOVER_HDZYFZE_std', 'GJJRATE_DKYE_genFeat4_mean', 'GRZHZT_DKFFE_genFeat1_mean', 'numsOf_DKFFE_HDZYFZE_diff', 'NOUSAGE_HDYF_std', 'NOUSAGE_GJJRATE_std', 'NOUSAGE_DKFFE_genFeat5_sum', 'WITHSIGN2_USAGE_RATE_mean', 'WITHSIGN2_DKFFE_genFeat5_sum', 'GJSIGN_USAGE_AMOUNT_mean', 'numsOf_GRYJCE_HDZYFZE_diff', 'GJJRATE_DKYE_max', 'WITHSIGN2_DKFFE_genFeat6_sum', 'WITHSIGN_DKFFE_genFeat1_sum', 'WITHSIGN2_HDZYFZE_std', 'DWSSHY_DKFFE_genFeat2_sum', 'TOP_USER_DKYE_genFeat4_mean', 'USAGEOVER_GRZHYE_std', 'GJSIGN_count', 'GJSIGN_DKYE_genFeat2_mean', 'DWSSHY_DKFFE_genFeat6_sum', 'DKLL_TOP_USER_sum', 'numsOf_DKLL_TOP_USER1_diff', 'WITHSIGN_DKFFE_genFeat2_mean', 'GJSIGN_USAGE_RATE_mean', 'numsOf_GJJRATE_USAGE_AMOUNT_add', 'USAGEOVER_DKFFE_genFeat5_sum', 'GJJRATE_DKYE_genFeat2_sum', 'GRZHSNJZYE_DKLL_sum', 'YEARPURINCM_GRZHYE_max', 'numsOf_HDZYFZE_TOP_USER1_diff', 'DWJJLX_7', 'HDZYFZE_DKFFE_max', 'GRZHZT_DKYE_genFeat4_sum', 'TOP_USER1_GRYJCE_sum', 'NOUSAGE_HDYF_min', 'USAGEOVER_DKFFE_genFeat7_mean', 'WITHSIGN_1', 'GRZHYE_HDSYYFZE_sum', 'WITHSIGN2_YEARPURINCM_max', 'numsOf_DKYE_HDZYF_diff', 'DKYE_DKFFE_mean', 'DKFFE_TOP_USER_min', 'WITHSIGN2_GRZHDNGJYE_sum', 'GJSIGN_WITHDRAW_SUBMIT_mean', 'GJSIGN_DKYE_genFeat4_mean', 'DKLL_GRZHYE_min', 'WITHSIGN_WITHDRAW_SUBMIT_sum', 'DKLL_HDSYYF_sum', 'DWSSHY_20', 'numsOf_DKFFE_TOP_USER1_diff', 'DKFFE_genFeat1', 'DKLL_GRYJCE_sum', 'GJSIGN_WITHDRAW_mean', 'NOUSAGE_HDSYYFZE_max', 'numsOf_HDZYF_USAGE_AMOUNT_diff', 'GRZHYE_DKFFE_max', 'WITHDRAW_DKFFE_max', 'NOUSAGE_DKFFE_max', 'USAGEOVER_HDZYFZE_mean', 'USAGEOVER_GRZHSNJZYE_std', 'GJSIGN_DKFFE_genFeat7_mean', 'USAGEOVER_DKFFE_std', 'TOP_USER_DKYE_genFeat1_sum', 'numsOf_HDSYYF_USAGE_AMOUNT_diff', 'GRZHZT_GRZHDNGJYE_sum', 'numsOf_DKFFE_TOP_USER_add', 'WITHSIGN_DKYE_genFeat1_mean', 'numsOf_GRYJCE_HDSYYFZE_diff', 'GJSIGN_DKLL_std', 'TOP_USER_DKFFE_genFeat3_mean', 'WITHSIGN2_TOP_USER1_std', 'USAGEOVER_DKYE_genFeat1_mean', 'HDYF_GRJCJS_sum', 'USAGEOVER_TOP_USER1_mean', 'GJJRATE_HDSYYF_min', 'USAGEOVER_DKYE_max', 'GJSIGN_DKFFE_genFeat7_sum', 'GJJRATE_DKFFE_genFeat1_mean', 'USAGEOVER_0', 'WITHSIGN_HDYF_mean', 'numsOf_HDYF_HDSYYFZE_diff', 'GRZHYE_GJJRATE_sum', 'TOP_USER1_TOP_USER_sum', 'GJJRATE_8', 'NOUSAGE_DKYE_genFeat4_sum', 'numsOf_HDSYYF_GJJRATE_add', 'DWSSHY_DKYE_genFeat3_sum', 'GJSIGN_HDZYF_std', 'GJJRATE_DKFFE_genFeat2_mean', 'GJSIGN_HDYF_sum', 'numsOf_HDZYF_WITHDRAW_SUBMIT_diff', 'GJSIGN_DKFFE_genFeat2_sum', 'WITHSIGN2_HDZYF_min', 'GJJRATE_DKYE_mean', 'numsOf_DKYE_WITHDRAW_diff', 'NOUSAGE_GRYJCE_sum', 'GJSIGN_DKFFE_genFeat4_sum', 'numsOf_HDSYYFZE_WITHDRAW_div', 'numsOf_DKFFE_HDSYYFZE_add', 'DWSSHY_USAGE_RATE_max', 'USAGEOVER_GRZHYE_mean', 'WITHSIGN2_HDZYFZE_mean', 'GRZHZT_GRZHSNJZYE_max', 'HDSYYF_TOP_USER1_sum', 'numsOf_YEARPURINCM_HDZYF_diff', 'WITHSIGN2_GRJCJS_sum', 'NOUSAGE_WITHDRAW_SUBMIT_min', 'numsOf_GRZHSNJZYE_USAGE_RATE_add', 'numsOf_HDZYFZE_TOP_USER_diff', 'WITHSIGN_GRJCJS_mean', 'GJJRATE_DKLL_sum', 'TOP_USER1_DKYE_sum', 'NOUSAGE_DKYE_genFeat5_mean', 'numsOf_GRYJCE_GJJRATE_add', 'USAGEOVER_GRZHDNGJYE_max', 'GJSIGN_DKLL_max', 'YEARPURINCM_GRZHSNJZYE_mean', 'DKFFE_DKLL_min', 'GRZHSNJZYE_DKFFE_sum', 'HDZYFZE_DKYE_sum', 'numsOf_DKYE_HDZYF_add', 'HDSYYF_HDYF_sum', 'HDYF_TOP_USER1_sum', 'USAGEOVER_WITHDRAW_std', 'WITHSIGN2_GRJCJS_min', 'WITHSIGN2_DKFFE_genFeat2_sum', 'WITHSIGN2_HDZYF_sum', 'DKLL_HDZYFZE_sum', 'GJJRATE_count', 'GJSIGN_GRZHDNGJYE_mean', 'WITHSIGN_DKFFE_genFeat6_sum', 'numsOf_YEARPURINCM_WITHDRAW_SUBMIT_add', 'WITHSIGN2_DKYE_mean', 'numsOf_GRZHYE_USAGE_RATE_add', 'GJJRATE_DKYE_genFeat5_sum', 'HDZYF_DKLL_min', 'GRZHSNJZYE_DKFFE_min', 'numsOf_DKYE_HDZYFZE_multi', 'HDZYFZE_DKLL_sum', 'GRZHZT_GRZHSNJZYE_min', 'numsOf_DKFFE_HDSYYFZE_div', 'GJJRATE_DKFFE_max', 'TOP_USER_TOP_USER_mean', 'HDSYYF_USAGE_RATE_mean', 'numsOf_USAGE_RATE_WITHDRAW_add', 'DKFFE_TOP_USER1_min', 'numsOf_HDSYYF_HDSYYFZE_diff', 'NOUSAGE_USAGE_RATE_std', 'numsOf_GRYJCE_USAGE_AMOUNT_diff', 'numsOf_DKYE_USAGE_AMOUNT_multi', 'HDSYYF_USAGE_RATE_max', 'USAGEOVER', 'DWSSHY_DKFFE_genFeat7_sum', 'GJJRATE_USAGE_RATE_mean', 'GRZHSNJZYE_DKFFE_max', 'GRYJCE', 'GJSIGN_0', 'DWJJLX_HDZYF_sum', 'USAGEOVER_DKYE_genFeat6_sum', 'numsOf_GRJCJS_DKLL_add', 'numsOf_YEARPURINCM_USAGE_RATE_add', 'WITHSIGN2_WITHDRAW_min', 'USAGEOVER_DKYE_genFeat2_mean', 'YEARPURINCM_DKYE_max', 'NOUSAGE_HDSYYF_max', 'NOUSAGE_TOP_USER1_mean', 'NOUSAGE_DKYE_genFeat6_sum', 'USAGEOVER_WITHDRAW_max', 'WITHSIGN_WITHDRAW_sum', 'TOP_USER_GRJCJS_sum', 'GJSIGN_HDSYYF_sum', 'TOP_USER1_DKFFE_min', 'DWJJLX_DKFFE_genFeat5_sum', 'GRZHZT_DKYE_genFeat1_sum', 'numsOf_HDZYF_TOP_USER_add', 'WITHSIGN_DKYE_genFeat3_sum', 'GJSIGN_DKLL_sum', 'USAGEOVER_count', 'GJSIGN_DKYE_genFeat5_mean', 'NOUSAGE_YEARPURINCM_max', 'GJSIGN_GRZHDNGJYE_min', 'WITHDRAW_SUBMIT_GRZHDNGJYE_sum', 'DWJJLX_DKFFE_genFeat4_mean', 'HDZYFZE_DKLL_min', 'GRZHZT_DKFFE_min', 'numsOf_HDSYYF_USAGE_RATE_add', 'GRZHZT_HDSYYFZE_mean', 'WITHSIGN2_USAGE_AMOUNT_sum', 'DKLL_DKFFE_sum', 'NOUSAGE_HDZYF_min', 'USAGEOVER_GRYJCE_sum', 'WITHSIGN2_TOP_USER_std', 'TOP_USER_DKFFE_genFeat3_sum', 'HDYF_DKYE_max', 'TOP_USER_DKFFE_min', 'NOUSAGE_DKFFE_std', 'NOUSAGE_WITHDRAW_std', 'DKFFE_genFeat3', 'numsOf_HDZYFZE_GJJRATE_add', 'numsOf_DKLL_TOP_USER1_add', 'numsOf_HDZYFZE_USAGE_RATE_diff', 'USAGEOVER_HDYF_std', 'DWJJLX_GRZHSNJZYE_sum', 'numsOf_DKYE_USAGE_AMOUNT_div', 'NOUSAGE_USAGE_RATE_sum', 'numsOf_GRJCJS_HDSYYFZE_add', 'numsOf_GRZHDNGJYE_HDSYYFZE_diff', 'NOUSAGE_DKLL_min', 'YEARPURINCM_DKYE_mean', 'TOP_USER1_DKFFE_mean', 'numsOf_HDZYF_USAGE_RATE_multi', 'USAGEOVER_DKLL_max', 'GRZHZT_GRYJCE_mean', 'GJSIGN_DKFFE_genFeat2_mean', 'GRZHZT_GRYJCE_max', 'numsOf_DKLL_HDSYYF_add', 'DWJJLX_DKFFE_genFeat4_sum', 'WITHDRAW_SUBMIT_DKFFE_max', 'NOUSAGE_YEARPURINCM_sum', 'WITHSIGN2_GRYJCE_sum', 'NOUSAGE_WITHDRAW_SUBMIT_sum', 'numsOf_DKYE_GJJRATE_add', 'TOP_USER1_WITHDRAW_sum', 'USAGEOVER_in_NOUSAGE_prop', 'GJJRATE_DKFFE_genFeat7_mean', 'numsOf_HDSYYFZE_USAGE_RATE_diff', 'WITHSIGN2_DKYE_genFeat1_mean', 'WITHSIGN_DKYE_genFeat3_mean', 'WITHSIGN2_DKYE_genFeat7_sum', 'numsOf_DKLL_HDZYF_diff', 'numsOf_GRZHSNJZYE_GJJRATE_add', 'DWJJLX_DKFFE_genFeat6_sum', 'GRZHZT_DKYE_std', 'WITHSIGN2_HDSYYFZE_sum', 'WITHSIGN2_DKFFE_genFeat4_sum', 'GJSIGN_USAGE_AMOUNT_max', 'WITHSIGN2_GRZHYE_sum', 'WITHSIGN_HDZYFZE_sum', 'TOP_USER_DKYE_max', 'DKLL_TOP_USER1_sum', 'WITHSIGN2_DKLL_sum', 'GJSIGN_HDYF_min', 'GRZHZT_HDZYF_std', 'WITHDRAW_HDSYYFZE_sum', 'numsOf_DKFFE_HDSYYF_add', 'GRZHZT_YEARPURINCM_max', 'DKFFE_GJJRATE_min', 'GRZHZT_GRJCJS_mean', 'numsOf_GRYJCE_DKFFE_div', 'USAGEOVER_DKFFE_genFeat5_mean', 'DWJJLX_DKYE_genFeat1_sum', 'YEARPURINCM_GRZHYE_mean', 'DWSSHY_GRZHYE_min', 'GRZHZT_DKLL_max', 'GRZHZT_HDSYYFZE_sum', 'numsOf_GRZHYE_HDSYYF_diff', 'WITHSIGN_USAGE_RATE_sum', 'NOUSAGE_WITHDRAW_SUBMIT_max', 'numsOf_GRZHDNGJYE_HDZYF_diff', 'USAGEOVER_GRJCJS_sum', 'numsOf_DKLL_HDSYYF_diff', 'numsOf_GRYJCE_DKYE_diff', 'numsOf_HDZYF_USAGE_AMOUNT_add', 'numsOf_DKFFE_DKLL_add', 'GJSIGN_HDYF_mean', 'WITHSIGN2_YEARPURINCM_sum', 'numsOf_YEARPURINCM_HDYF_add', 'GJSIGN_DKYE_genFeat1_sum', 'GJSIGN_HDZYF_mean', 'numsOf_HDZYF_WITHDRAW_multi', 'WITHSIGN_DKYE_max', 'TOP_USER1_USAGE_RATE_sum', 'GJSIGN_GRZHDNGJYE_std', 'GJJRATE_GRZHSNJZYE_std', 'GJSIGN_DKYE_genFeat2_sum', 'numsOf_HDZYF_USAGE_AMOUNT_multi', 'DKFFE_genFeat6', 'DWSSHY_DKFFE_genFeat7_mean', 'numsOf_DKYE_DKLL_diff', 'WITHSIGN2_DKFFE_genFeat7_sum', 'WITHSIGN2_GJJRATE_std', 'numsOf_DKLL_WITHDRAW_diff', 'numsOf_DKYE_USAGE_AMOUNT_add', 'NOUSAGE_TOP_USER1_sum', 'USAGEOVER_TOP_USER_mean', 'HDSYYF_TOP_USER_sum', 'numsOf_GRZHYE_GRZHDNGJYE_diff', 'numsOf_GRZHDNGJYE_TOP_USER_diff', 'USAGEOVER_DKFFE_mean', 'DWJJLX_DKFFE_genFeat1_mean', 'WITHSIGN2', 'WITHSIGN2_GRYJCE_max', 'TOP_USER_DKFFE_genFeat6_mean', 'numsOf_HDYF_USAGE_RATE_add', 'numsOf_DKFFE_TOP_USER_div', 'numsOf_GRZHSNJZYE_TOP_USER_diff', 'GJSIGN_DKYE_genFeat6_sum', 'DWJJLX_DKFFE_genFeat3_sum', 'GRZHYE', 'HDYF_YEARPURINCM_sum', 'USAGEOVER_DKFFE_genFeat3_sum', 'numsOf_YEARPURINCM_HDSYYF_add', 'numsOf_DKLL_HDYF_div', 'GRZHZT_YEARPURINCM_std', 'NOUSAGE_HDZYFZE_min', 'NOUSAGE_HDSYYFZE_mean', 'numsOf_DKFFE_WITHDRAW_add', 'GRJCJS', 'numsOf_TOP_USER_WITHDRAW_diff', 'HDZYFZE_DKYE_max', 'USAGEOVER_USAGE_AMOUNT_min', 'USAGEOVER_WITHDRAW_SUBMIT_min', 'WITHSIGN_WITHDRAW_mean', 'numsOf_YEARPURINCM_TOP_USER_add', 'GJSIGN_HDZYFZE_min', 'NOUSAGE_HDSYYF_min', 'WITHSIGN2_DKFFE_mean', 'HDSYYF_DKFFE_sum', 'NOUSAGE_DKYE_sum', 'GJJRATE_GRYJCE_sum', 'GJSIGN_YEARPURINCM_mean', 'DWSSHY_DKYE_genFeat1_sum', 'GRZHZT_WITHDRAW_SUBMIT_mean', 'numsOf_GRZHYE_WITHDRAW_SUBMIT_add', 'numsOf_DKFFE_USAGE_RATE_add', 'GJJRATE_DKFFE_min', 'NOUSAGE_HDZYF_std', 'WITHSIGN_HDZYFZE_std', 'DWJJLX_DKYE_max', 'HDYF_DKFFE_max', 'WITHSIGN2_DKLL_mean', 'USAGEOVER_GRZHDNGJYE_std', 'numsOf_DKLL_USAGE_AMOUNT_add', 'WITHSIGN2_DKFFE_genFeat3_sum', 'WITHSIGN_HDSYYF_sum', 'HDZYFZE_DKLL_max', 'WITHDRAW_SUBMIT_GJJRATE_sum', 'numsOf_GRZHDNGJYE_HDYF_diff', 'DWSSHY_DKYE_genFeat7_sum', 'numsOf_DKLL_HDZYFZE_diff', 'GRZHZT_DKFFE_genFeat6_sum', 'DKFFE_HDZYFZE_min', 'USAGEOVER_DKFFE_sum', 'DWJJLX_DKFFE_genFeat2_mean', 'GRZHZT_GRZHDNGJYE_min', 'numsOf_DKYE_TOP_USER_diff', 'GRZHZT_DKFFE_std', 'numsOf_HDSYYFZE_USAGE_RATE_add', 'DWJJLX_DKFFE_std', 'GRZHSNJZYE_HDZYFZE_sum', 'GJSIGN_WITHDRAW_sum', 'numsOf_DKFFE_TOP_USER_diff', 'USAGEOVER_GRJCJS_min', 'DKFFE_genFeat4', 'DWJJLX_DKFFE_genFeat6_mean', 'WITHSIGN2_DKLL_std', 'USAGEOVER_DKYE_genFeat5_mean', 'numsOf_GRZHSNJZYE_WITHDRAW_SUBMIT_add', 'NOUSAGE_GRYJCE_mean', 'GJJRATE_DKFFE_genFeat5_mean', 'USAGEOVER_DKFFE_genFeat6_mean', 'NOUSAGE_WITHDRAW_SUBMIT_std', 'WITHSIGN_2', 'GRZHZT_GJJRATE_sum', 'TOP_USER_DKYE_genFeat1_mean', 'GJJRATE_DKFFE_genFeat1_sum', 'DKLL_DKYE_std', 'WITHSIGN_GRJCJS_sum', 'DWJJLX_DKFFE_genFeat3_mean', 'numsOf_DKYE_WITHDRAW_SUBMIT_diff', 'DKYE_genFeat1', 'NOUSAGE_HDZYF_mean', 'GJSIGN_WITHDRAW_max', 'GRZHZT_DKYE_genFeat2_sum', 'DKLL_DKFFE_max', 'numsOf_DKLL_USAGE_AMOUNT_diff', 'numsOf_GRJCJS_DKFFE_add', 'HDYF', 'TOP_USER_GRZHDNGJYE_min', 'DKYE_genFeat3', 'HDSYYF_DKLL_max', 'DWJJLX_GRZHYE_sum', 'WITHSIGN2_GRZHDNGJYE_min', 'HDYF_DKFFE_mean', 'GJSIGN_HDSYYF_std', 'numsOf_GRJCJS_GRYJCE_add', 'WITHSIGN2_GRYJCE_std', 'USAGEOVER_DKYE_genFeat7_mean', 'USAGEOVER_USAGE_RATE_sum', 'GRZHZT_TOP_USER1_sum', 'numsOf_HDZYFZE_TOP_USER_div', 'DWJJLX_DKYE_genFeat4_mean', 'WITHSIGN_YEARPURINCM_mean', 'DKYE_HDSYYFZE_min', 'numsOf_DKYE_GJJRATE_diff', 'WITHSIGN_DKYE_genFeat4_sum', 'WITHSIGN2_DKFFE_genFeat1_mean', 'WITHSIGN2_DKFFE_genFeat4_mean', 'TOP_USER_DKFFE_mean', 'numsOf_GRZHYE_HDZYF_diff', 'USAGEOVER_GJJRATE_mean', 'USAGEOVER_HDZYF_mean', 'TOP_USER_DKFFE_genFeat1_mean', 'numsOf_DKFFE_GJJRATE_add', 'WITHSIGN2_HDYF_std', 'NOUSAGE_DKFFE_genFeat3_mean', 'numsOf_HDSYYFZE_TOP_USER_add', 'GRZHZT_DKYE_genFeat3_sum', 'TOP_USER_DKYE_genFeat2_sum', 'DKLL_HDSYYFZE_sum', 'DWJJLX_DKFFE_sum', 'numsOf_TOP_USER_TOP_USER1_add', 'NOUSAGE_DKFFE_genFeat5_mean', 'DWJJLX_HDZYF_std', 'numsOf_DKFFE_HDSYYFZE_multi', 'GJSIGN_GRZHYE_max', 'numsOf_HDYF_HDSYYFZE_add', 'DKFFE_DKYE_mean', 'WITHSIGN2_WITHDRAW_SUBMIT_mean', 'WITHSIGN_DKFFE_genFeat1_mean', 'TOP_USER_DKFFE_std', 'HDSYYF_DKFFE_mean', 'NOUSAGE_USAGE_AMOUNT_sum', 'WITHSIGN2_USAGE_RATE_std', 'numsOf_GRYJCE_HDZYF_multi', 'GRZHZT_DKFFE_genFeat2_mean', 'GJSIGN_DKFFE_genFeat5_mean', 'HDZYFZE_TOP_USER_mean', 'DWSSHY_DKFFE_genFeat3_sum', 'numsOf_GRZHYE_HDYF_add', 'NOUSAGE_DKYE_genFeat6_mean', 'WITHSIGN_DKYE_genFeat5_mean', 'DKYE_genFeat6', 'GRZHZT_DKFFE_mean', 'WITHSIGN_DKFFE_sum', 'GJSIGN_DKFFE_genFeat1_mean', 'WITHSIGN2_DKYE_genFeat2_sum', 'GRZHZT_HDZYFZE_std', 'USAGEOVER_DKYE_mean', 'DKLL_GRZHYE_sum', 'USAGEOVER_WITHDRAW_sum', 'WITHSIGN_WITHDRAW_max', 'HDYF_TOP_USER_sum', 'GJSIGN_TOP_USER1_max', 'USAGEOVER_USAGE_RATE_min', 'DWSSHY_DKYE_genFeat2_sum', 'numsOf_DKLL_HDSYYF_div', 'numsOf_HDZYFZE_WITHDRAW_SUBMIT_add', 'numsOf_GRJCJS_TOP_USER_diff', 'DWJJLX_DKLL_max', 'GRZHZT_GRJCJS_std', 'GJSIGN_DKFFE_genFeat6_sum', 'GJSIGN_DKYE_genFeat4_sum', 'TOP_USER_0', 'WITHSIGN2_DKYE_genFeat4_mean', 'HDSYYF_DKFFE_min', 'NOUSAGE_DKYE_min', 'GRZHZT_GRJCJS_sum', 'TOP_USER', 'WITHSIGN_GJJRATE_sum', 'HDYF_USAGE_RATE_mean', 'USAGEOVER_HDSYYF_sum', 'USAGEOVER_TOP_USER1_std', 'USAGEOVER_DKYE_sum', 'NOUSAGE_GRJCJS_sum', 'WITHSIGN_DKYE_genFeat7_sum', 'WITHSIGN2_DKFFE_genFeat2_mean', 'NOUSAGE_in_DWSSHY_prop', 'GJJRATE_DKYE_genFeat1_mean', 'NOUSAGE', 'USAGEOVER_YEARPURINCM_mean', 'WITHSIGN2_GRZHDNGJYE_mean', 'numsOf_USAGE_RATE_USAGE_AMOUNT_diff', 'numsOf_DKYE_WITHDRAW_SUBMIT_add', 'GRZHZT_DKYE_genFeat5_sum', 'USAGEOVER_HDSYYF_max', 'HDSYYF_DKLL_min', 'GJSIGN_1', 'YEARPURINCM_DKFFE_max', 'USAGEOVER_USAGE_AMOUNT_max', 'GJJRATE_DKYE_sum', 'NOUSAGE_TOP_USER_std', 'NOUSAGE_USAGE_AMOUNT_max', 'HDZYF', 'HDYF_DKYE_sum', 'DKLL_HDYF_sum', 'numsOf_DKFFE_USAGE_AMOUNT_div', 'WITHDRAW_SUBMIT_TOP_USER_sum', 'GRZHZT_DKFFE_genFeat3_sum', 'numsOf_DKFFE_TOP_USER_multi', 'numsOf_YEARPURINCM_HDSYYF_diff', 'WITHSIGN2_DKYE_std', 'DWSSHY_DKFFE_sum', 'USAGEOVER_GRYJCE_mean', 'WITHSIGN_HDZYF_mean', 'TOP_USER_DKFFE_genFeat7_sum', 'GJSIGN_DKFFE_sum', 'GRZHZT_DKYE_genFeat1_mean', 'NOUSAGE_DKYE_std', 'NOUSAGE_DKLL_mean', 'DWJJLX_USAGE_RATE_sum', 'DKYE_DKFFE_min', 'DWJJLX_DKYE_genFeat5_sum', 'HDSYYF_USAGE_AMOUNT_mean', 'numsOf_DKLL_WITHDRAW_add', 'USAGEOVER_HDYF_sum', 'GRZHZT_DKFFE_genFeat2_sum', 'GJJRATE_DKFFE_genFeat5_sum', 'numsOf_GRZHYE_DKLL_add', 'DKLL_DKFFE_mean', 'WITHSIGN_DKYE_genFeat2_mean', 'HDZYF_DKLL_max', 'GRZHZT_DKYE_max', 'GJJRATE_DKYE_min', 'WITHSIGN_GRZHDNGJYE_min', 'GJJRATE_USAGE_RATE_min', 'numsOf_DKYE_HDSYYF_diff', 'GJJRATE_GJJRATE_mean', 'TOP_USER_DKYE_genFeat6_sum', 'DWJJLX_DKYE_genFeat5_mean', 'DKLL_DKYE_sum', 'numsOf_DKFFE_USAGE_AMOUNT_diff', 'HDZYFZE_TOP_USER1_mean', 'WITHSIGN_DKFFE_genFeat5_sum', 'USAGEOVER_USAGE_RATE_mean', 'DKFFE_TOP_USER1_mean', 'DWJJLX_DKFFE_genFeat1_sum', 'DKFFE_DKYE_max', 'WITHSIGN_DKFFE_genFeat4_sum', 'numsOf_GRZHSNJZYE_DKLL_add', 'TOP_USER1_USAGE_AMOUNT_sum', 'HDSYYFZE_GRZHSNJZYE_sum', 'TOP_USER_DKYE_genFeat3_mean', 'YEARPURINCM_DKYE_sum', 'numsOf_DKFFE_DKLL_diff', 'GJSIGN_GRJCJS_std', 'WITHSIGN2_DKYE_genFeat6_sum', 'GJSIGN_HDYF_std', 'WITHSIGN2_GRZHYE_mean', 'numsOf_DKLL_YEARPURINCM_diff', 'GJSIGN_GJJRATE_std', 'DWJJLX_GRYJCE_max', 'numsOf_DKFFE_TOP_USER1_add', 'numsOf_GRZHDNGJYE_HDYF_add', 'USAGEOVER_GRYJCE_std', 'GJJRATE_USAGE_RATE_sum', 'GJSIGN_DKYE_max', 'TOP_USER_DKYE_genFeat6_mean', 'WITHSIGN_DKYE_sum', 'HDSYYFZE_DKFFE_max', 'GRZHZT_USAGE_AMOUNT_sum', 'GJSIGN_HDZYF_sum', 'USAGEOVER_DKFFE_min', 'GRZHSNJZYE', 'WITHSIGN2_WITHDRAW_SUBMIT_sum', 'GRZHZT_TOP_USER1_mean', 'GRZHZT_DKYE_genFeat5_mean', 'GRZHZT_DKYE_sum', 'GJJRATE_DKYE_genFeat4_sum', 'WITHSIGN_DKFFE_mean', 'WITHSIGN_GRYJCE_std', 'GJSIGN_2', 'GRZHZT_GRZHDNGJYE_max', 'numsOf_HDSYYF_USAGE_RATE_diff', 'GJJRATE_DKYE_genFeat6_sum', 'numsOf_WITHDRAW_WITHDRAW_SUBMIT_add', 'TOP_USER_DKFFE_max', 'DWJJLX_DKLL_min', 'GJSIGN_DKYE_sum', 'NOUSAGE_GRJCJS_mean', 'WITHSIGN_DKYE_genFeat7_mean', 'WITHDRAW_SUBMIT_DKFFE_mean', 'DKFFE_GRJCJS_sum', 'numsOf_YEARPURINCM_HDYF_diff', 'GJSIGN_USAGE_RATE_std', 'DKLL_GJJRATE_sum', 'USAGEOVER_HDSYYFZE_sum', 'numsOf_HDZYFZE_TOP_USER_add', 'numsOf_HDZYF_USAGE_RATE_diff', 'GRZHZT_in_NOUSAGE_prop', 'numsOf_DKYE_HDYF_diff', 'GRZHZT_GRZHYE_std', 'WITHSIGN2_HDZYF_max', 'numsOf_TOP_USER_USAGE_AMOUNT_add', 'numsOf_DKYE_HDYF_add', 'DKYE_DKFFE_max', 'USAGEOVER_DKLL_mean', 'WITHSIGN2_DKYE_genFeat2_mean', 'numsOf_HDZYF_HDSYYF_diff', 'NOUSAGE_in_GJJRATE_prop', 'WITHSIGN2_1', 'GJSIGN_DKYE_genFeat7_mean', 'numsOf_DKYE_USAGE_RATE_div', 'numsOf_DKLL_YEARPURINCM_add', 'TOP_USER1_HDZYF_sum', 'HDSYYF_DKYE_mean', 'WITHDRAW', 'USAGEOVER_DKYE_genFeat6_mean', 'GJSIGN_WITHDRAW_min', 'USAGEOVER_GJJRATE_max', 'NOUSAGE_in_GRZHZT_prop', 'HDSYYFZE_DKYE_mean', 'WITHSIGN2_GRYJCE_min', 'WITHSIGN_HDYF_sum', 'numsOf_HDZYFZE_USAGE_AMOUNT_div', 'GJSIGN_WITHDRAW_SUBMIT_sum', 'numsOf_GRYJCE_DKFFE_add', 'NOUSAGE_DKFFE_sum', 'NOUSAGE_WITHDRAW_max', 'HDYF_DKYE_min', 'NOUSAGE_HDZYF_max', 'TOP_USER1_HDZYF_max', 'TOP_USER_7', 'GJJRATE_HDZYF_mean', 'NOUSAGE_DKYE_genFeat5_sum', 'WITHSIGN_DKYE_genFeat5_sum', 'TOP_USER1_DKLL_sum', 'NOUSAGE_GRYJCE_min', 'numsOf_DKFFE_WITHDRAW_SUBMIT_add', 'DWSSHY_DKFFE_genFeat4_mean', 'NOUSAGE_YEARPURINCM_min', 'USAGEOVER_DKYE_min', 'DWJJLX_in_GRZHZT_prop', 'USAGEOVER_TOP_USER_std', 'HDSYYF_DKYE_min', 'numsOf_HDZYFZE_USAGE_RATE_multi', 'numsOf_YEARPURINCM_GJJRATE_add', 'HDSYYF_USAGE_AMOUNT_max', 'GRZHZT_GJJRATE_mean', 'WITHSIGN_GRYJCE_sum', 'WITHSIGN2_TOP_USER1_mean', 'HDSYYF_DKLL_mean', 'GJSIGN_DKFFE_genFeat3_sum', 'numsOf_GRYJCE_USAGE_RATE_diff', 'WITHSIGN2_HDZYFZE_sum', 'GRZHDNGJYE_DKLL_sum', 'HDSYYF_GJJRATE_sum', 'DKFFE_HDZYFZE_sum', 'USAGEOVER_YEARPURINCM_std', 'NOUSAGE_DKFFE_genFeat6_mean', 'TOP_USER_USAGE_RATE_max', 'GJSIGN_USAGE_AMOUNT_std', 'TOP_USER1_GRJCJS_sum', 'TOP_USER1_HDSYYF_sum', 'numsOf_DKYE_TOP_USER_add', 'GJJRATE_HDZYF_std', 'numsOf_GRZHDNGJYE_WITHDRAW_SUBMIT_add', 'YEARPURINCM_DKFFE_min', 'numsOf_GRZHSNJZYE_DKLL_diff', 'numsOf_TOP_USER_USAGE_AMOUNT_diff', 'DKFFE_HDSYYF_sum', 'GRZHZT_DKFFE_genFeat4_mean', 'TOP_USER_TOP_USER1_sum', 'NOUSAGE_USAGE_RATE_min', 'USAGEOVER_DKFFE_genFeat7_sum', 'USAGEOVER_DKYE_genFeat3_mean', 'GRZHZT_HDZYFZE_min', 'DKYE', 'HDZYFZE_DKFFE_min', 'DKFFE', 'GRZHZT_YEARPURINCM_mean', 'WITHDRAW_SUBMIT_DKLL_sum', 'GRZHSNJZYE_GJJRATE_sum', 'DKFFE_genFeat7', 'numsOf_GRZHDNGJYE_DKLL_add', 'DKFFE_genFeat2', 'NOUSAGE_count', 'DKFFE_DKLL_sum', 'DKLL_HDZYF_max', 'numsOf_DKFFE_USAGE_RATE_multi', 'HDYF_GRZHYE_sum', 'numsOf_GRYJCE_HDSYYF_multi', 'WITHDRAW_DKFFE_min', 'NOUSAGE_DKFFE_genFeat7_mean', 'TOP_USER1_TOP_USER_max', 'WITHSIGN2_HDSYYF_max', 'WITHSIGN2_YEARPURINCM_min', 'HDSYYFZE_USAGE_RATE_mean', 'DKLL_USAGE_AMOUNT_sum', 'DKYE_HDZYFZE_min', 'DKFFE_HDZYFZE_max', 'GRZHZT_DKFFE_genFeat7_mean', 'GJJRATE_DKYE_genFeat3_sum', 'DWSSHY_DKFFE_genFeat4_sum', 'DWSSHY_DKYE_mean', 'WITHDRAW_DKLL_sum', 'GRZHYE_DKLL_sum', 'HDSYYFZE_DKYE_max', 'WITHSIGN2_USAGE_RATE_sum', 'TOP_USER_DKFFE_genFeat6_sum', 'GJSIGN_USAGE_RATE_sum', 'numsOf_DKFFE_WITHDRAW_SUBMIT_diff', 'numsOf_HDZYF_HDSYYFZE_add', 'HDSYYF_DKYE_max', 'WITHSIGN_GRZHYE_mean', 'DWSSHY_HDZYFZE_max', 'GRZHZT_GRYJCE_min', 'WITHDRAW_SUBMIT', 'WITHSIGN_YEARPURINCM_sum', 'GJJRATE_DKYE_genFeat2_mean', 'GJSIGN_DKYE_genFeat1_mean', 'HDSYYFZE_GRJCJS_sum', 'NOUSAGE_GRZHYE_max', 'TOP_USER_GRZHSNJZYE_min', 'DKFFE_TOP_USER_mean', 'GRZHZT_1', 'GJSIGN_DKFFE_genFeat5_sum', 'numsOf_DKLL_HDZYF_add', 'GRZHZT_HDZYFZE_mean', 'numsOf_DKYE_WITHDRAW_add', 'NOUSAGE_DKYE_genFeat2_mean', 'NOUSAGE_HDZYFZE_mean', 'WITHSIGN2_GRZHDNGJYE_max', 'TOP_USER_TOP_USER1_mean', 'GJSIGN_HDSYYFZE_mean', 'USAGEOVER_HDYF_mean', 'DWSSHY_DKFFE_genFeat5_sum', 'numsOf_DKFFE_HDZYF_diff', 'numsOf_DKYE_HDZYF_multi', 'WITHSIGN2_HDSYYF_sum', 'WITHSIGN_GRJCJS_std', 'WITHSIGN_0', 'HDSYYFZE', 'DKFFE_HDZYFZE_mean', 'NOUSAGE_HDSYYF_mean', 'numsOf_USAGE_AMOUNT_WITHDRAW_SUBMIT_add', 'WITHSIGN2_TOP_USER_mean', 'DKYE_genFeat7', 'WITHSIGN_GRZHSNJZYE_sum', 'YEARPURINCM_GRZHYE_min', 'WITHSIGN2_DKYE_genFeat3_mean', 'numsOf_DKYE_DKLL_add', 'GJSIGN_GJJRATE_max', 'HDSYYF_HDZYF_min', 'numsOf_GRJCJS_WITHDRAW_SUBMIT_add', 'NOUSAGE_DKFFE_genFeat4_mean', 'numsOf_GRZHDNGJYE_HDZYFZE_add', 'NOUSAGE_GJJRATE_sum', 'numsOf_DKYE_HDSYYFZE_diff', 'NOUSAGE_GRZHDNGJYE_mean', 'YEARPURINCM_GRZHSNJZYE_min', 'USAGEOVER_WITHDRAW_SUBMIT_max', 'GRZHZT_HDSYYFZE_std', 'USAGEOVER_GRZHSNJZYE_sum', 'TOP_USER_DKYE_genFeat7_sum', 'numsOf_GRZHYE_GJJRATE_add', 'HDSYYF_DKYE_sum', 'TOP_USER_HDZYFZE_std', 'numsOf_GRZHYE_TOP_USER_add', 'NOUSAGE_DKFFE_genFeat7_sum', 'GJSIGN_DKYE_genFeat5_sum', 'numsOf_HDYF_USAGE_AMOUNT_diff', 'numsOf_GRZHDNGJYE_HDSYYFZE_add', 'DKFFE_GRZHSNJZYE_sum', 'DWSSHY_DKFFE_genFeat6_mean', 'WITHSIGN2_WITHDRAW_max', 'WITHSIGN2_YEARPURINCM_mean', 'WITHSIGN2_HDYF_max', 'HDYF_HDSYYFZE_sum', 'NOUSAGE_USAGE_AMOUNT_mean', 'WITHSIGN_DKFFE_min', 'WITHSIGN_DKYE_std', 'DWJJLX_DKFFE_genFeat7_sum', 'GJSIGN_DKFFE_mean', 'GRZHZT_DKFFE_genFeat6_mean', 'TOP_USER_3', 'DWSSHY_DKFFE_genFeat2_mean', 'GJJRATE_DKFFE_std', 'WITHSIGN2_HDZYF_std', 'TOP_USER_DKLL_max', 'DKFFE_TOP_USER1_max', 'DKFFE_USAGE_RATE_min', 'HDZYFZE_TOP_USER_max', 'WITHSIGN2_DKYE_genFeat1_sum', 'USAGEOVER_DKLL_std', 'GRZHZT_GJJRATE_max', 'NOUSAGE_DKFFE_genFeat2_mean', 'GJJRATE_DKFFE_genFeat6_mean', 'GJJRATE_GRZHYE_std', 'numsOf_GRYJCE_TOP_USER_add', 'NOUSAGE_HDSYYFZE_sum', 'WITHSIGN_WITHDRAW_min', 'GJJRATE_DKFFE_genFeat7_sum', 'numsOf_HDZYF_HDZYFZE_add', 'WITHSIGN_YEARPURINCM_max', 'NOUSAGE_GRZHDNGJYE_sum', 'WITHSIGN_HDSYYF_std', 'NOUSAGE_HDSYYF_sum', 'TOP_USER_DKYE_std', 'DWJJLX_DKFFE_min', 'WITHSIGN2_TOP_USER_sum', 'NOUSAGE_WITHDRAW_min', 'numsOf_DKFFE_TOP_USER1_multi', 'GJSIGN_DKFFE_genFeat6_mean', 'HDZYFZE', 'numsOf_DKFFE_HDZYF_add', 'WITHSIGN_DKFFE_genFeat3_mean', 'NOUSAGE_DKYE_genFeat4_mean', 'NOUSAGE_WITHDRAW_sum', 'numsOf_GJJRATE_WITHDRAW_add', 'numsOf_GRJCJS_GJJRATE_diff', 'numsOf_GRZHYE_TOP_USER_diff', 'numsOf_GRZHDNGJYE_HDZYFZE_diff', 'GRZHZT_GRZHYE_mean', 'YEARPURINCM_GRJCJS_mean', 'WITHSIGN2_HDZYF_mean', 'WITHDRAW_GJJRATE_sum', 'GJSIGN_DKLL_mean', 'numsOf_USAGE_RATE_USAGE_AMOUNT_add', 'GRZHZT_USAGE_RATE_sum', 'WITHDRAW_SUBMIT_WITHDRAW_mean', 'DKYE_HDSYYFZE_max', 'USAGEOVER_HDZYFZE_sum', 'NOUSAGE_DKLL_std', 'DKFFE_DKYE_sum', 'YEARPURINCM', 'numsOf_GRZHSNJZYE_GJJRATE_diff', 'GRZHZT_HDSYYFZE_min', 'GRZHZT_in_USAGEOVER_prop', 'NOUSAGE_DKFFE_genFeat6_sum', 'NOUSAGE_GRZHYE_min', 'TOP_USER_5', 'GRZHZT_DKYE_genFeat7_mean', 'numsOf_YEARPURINCM_TOP_USER_diff', 'GJSIGN_GRZHDNGJYE_sum', 'GJSIGN_DKYE_genFeat3_mean', 'GRZHZT_DKLL_min', 'GRZHZT_USAGE_RATE_min', 'USAGEOVER_DKYE_genFeat3_sum', 'DKYE_HDZYFZE_mean', 'DKLL_DKYE_min', 'numsOf_GRYJCE_HDSYYFZE_add', 'HDZYF_DKLL_mean', 'DWJJLX_DKYE_genFeat3_sum', 'GRZHYE_DKFFE_mean', 'NOUSAGE_HDZYFZE_std', 'GJJRATE_HDSYYFZE_sum', 'NOUSAGE_YEARPURINCM_std', 'GRZHZT_DKYE_mean', 'numsOf_HDSYYF_WITHDRAW_multi', 'NOUSAGE_DKFFE_min', 'GRZHZT_TOP_USER1_max', 'WITHSIGN2_WITHDRAW_SUBMIT_max', 'NOUSAGE_GJJRATE_mean', 'HDSYYFZE_DKFFE_sum', 'TOP_USER1_HDZYFZE_sum', 'NOUSAGE_HDZYF_sum', 'USAGEOVER_TOP_USER1_sum', 'GJSIGN_GRZHSNJZYE_max', 'GJSIGN', 'WITHSIGN2_HDSYYF_mean', 'DWJJLX_DKYE_genFeat3_mean', 'HDZYFZE_DKFFE_mean', 'GJSIGN_DKFFE_genFeat3_mean', 'numsOf_GRJCJS_DKYE_diff', 'DWJJLX_HDZYF_max', 'numsOf_HDSYYF_USAGE_AMOUNT_add', 'WITHSIGN2_count', 'TOP_USER_DKYE_genFeat2_mean', 'GRZHZT_GRZHYE_max', 'GJSIGN_USAGE_RATE_max', 'GJSIGN_TOP_USER_mean', 'WITHDRAW_TOP_USER_sum', 'USAGEOVER_DKFFE_genFeat2_mean', 'GJJRATE_HDSYYF_mean', 'HDSYYF_HDZYFZE_sum', 'USAGE_RATE_USAGE_AMOUNT_sum', 'GRZHZT_DKYE_genFeat7_sum', 'USAGEOVER_USAGE_AMOUNT_std', 'numsOf_GRJCJS_HDSYYFZE_diff', 'USAGEOVER_DKFFE_genFeat4_mean', 'GRZHZT_GRZHSNJZYE_sum', 'numsOf_GRJCJS_GJJRATE_add', 'HDSYYF_GRZHDNGJYE_sum', 'numsOf_HDZYFZE_GJJRATE_diff', 'DWJJLX_DKYE_genFeat7_sum', 'numsOf_GRYJCE_YEARPURINCM_diff', 'numsOf_GRZHDNGJYE_TOP_USER_add', 'GRZHSNJZYE_DKFFE_mean', 'USAGEOVER_USAGE_RATE_max', 'GJSIGN_DKYE_genFeat7_sum', 'NOUSAGE_1', 'WITHDRAW_SUBMIT_TOP_USER1_sum', 'DWSSHY_DKFFE_max', 'USAGEOVER_DKFFE_genFeat1_sum', 'WITHSIGN2_HDSYYFZE_std', 'DWJJLX_HDZYF_mean', 'WITHSIGN2_DKYE_genFeat5_mean', 'GRZHZT_HDYF_mean', 'NOUSAGE_GRJCJS_max', 'WITHSIGN_GRZHDNGJYE_max', 'USAGEOVER_HDSYYF_std', 'TOP_USER_DKFFE_genFeat7_mean', 'numsOf_GRZHYE_GJJRATE_diff', 'WITHSIGN2_0', 'GJSIGN_HDZYF_min', 'GJJRATE_DKYE_genFeat5_mean', 'TOP_USER_DKFFE_sum', 'WITHSIGN2_WITHDRAW_sum', 'GRZHZT_HDZYF_sum', 'NOUSAGE_DKLL_sum', 'GJJRATE_DKFFE_genFeat6_sum', 'DKLL_HDSYYF_min', 'numsOf_GJJRATE_USAGE_AMOUNT_diff', 'GRZHZT_GRYJCE_sum', 'numsOf_DKFFE_GJJRATE_div', 'NOUSAGE_GRJCJS_min', 'GJSIGN_TOP_USER1_std', 'numsOf_GRZHDNGJYE_USAGE_RATE_add', 'WITHSIGN2_DKFFE_genFeat7_mean', 'NOUSAGE_GRZHSNJZYE_max', 'GRZHZT_DKLL_mean', 'GRZHZT_DKYE_genFeat6_sum', 'GRZHZT_DKFFE_sum', 'DKLL_DKYE_max', 'GRJCJS_DKYE_sum', 'GJSIGN_GRZHYE_mean', 'GRJCJS_DKFFE_sum', 'USAGEOVER_GRYJCE_min', 'numsOf_TOP_USER_WITHDRAW_add', 'GJJRATE_DKYE_std', 'USAGEOVER_USAGE_AMOUNT_mean', 'TOP_USER1_HDYF_sum', 'GJSIGN_GRJCJS_sum', 'numsOf_GRZHYE_GRYJCE_diff', 'GRZHZT_DKFFE_genFeat5_mean', 'HDZYFZE_TOP_USER1_min', 'HDYF_HDSYYF_sum', 'NOUSAGE_HDYF_max', 'numsOf_GRYJCE_GJJRATE_diff', 'DWJJLX_DKYE_genFeat1_mean', 'USAGEOVER_GRJCJS_max', 'numsOf_GRYJCE_DKLL_diff', 'numsOf_GRYJCE_HDYF_multi', 'DKFFE_genFeat5', 'numsOf_HDZYFZE_USAGE_AMOUNT_diff', 'TOP_USER_DKFFE_genFeat2_mean', 'numsOf_GRJCJS_HDZYFZE_diff', 'WITHSIGN_GRZHDNGJYE_sum', 'GRZHZT_2', 'USAGEOVER_YEARPURINCM_sum', 'NOUSAGE_GRZHYE_mean', 'numsOf_HDZYF_WITHDRAW_SUBMIT_add', 'DWJJLX_0', 'NOUSAGE_DKFFE_genFeat2_sum', 'WITHSIGN2_DKYE_genFeat7_mean', 'GJJRATE_TOP_USER1_sum', 'numsOf_HDSYYFZE_GJJRATE_diff', 'TOP_USER1_DKYE_max', 'GJSIGN_GJJRATE_sum', 'DKLL_DKYE_mean', 'GJJRATE_DKLL_min', 'numsOf_HDZYFZE_USAGE_RATE_add', 'numsOf_GRYJCE_USAGE_AMOUNT_add', 'GRZHZT_GRZHYE_min', 'USAGEOVER_HDSYYF_min', 'USAGEOVER_GRJCJS_mean', 'numsOf_GRZHDNGJYE_DKLL_diff', 'numsOf_GRZHDNGJYE_USAGE_RATE_diff', 'numsOf_HDZYF_GJJRATE_diff', 'WITHDRAW_HDZYFZE_sum', 'NOUSAGE_HDSYYFZE_min', 'HDYF_DKFFE_min', 'TOP_USER_count', 'TOP_USER_TOP_USER_max', 'GJSIGN_GRJCJS_min', 'WITHSIGN2_HDZYFZE_min', 'numsOf_GRZHDNGJYE_GJJRATE_add', 'GJSIGN_GRZHSNJZYE_sum', 'NOUSAGE_TOP_USER1_std', 'NOUSAGE_TOP_USER_mean', 'numsOf_HDSYYF_USAGE_AMOUNT_multi', 'GRZHZT_GRZHSNJZYE_mean', 'WITHSIGN_GRZHDNGJYE_mean', 'WITHSIGN_DKFFE_genFeat7_sum', 'TOP_USER_DKFFE_genFeat1_sum', 'DWJJLX_DKFFE_genFeat2_sum', 'GJSIGN_GRZHDNGJYE_max', 'HDZYFZE_GRJCJS_sum', 'GRZHZT_DKFFE_genFeat5_sum', 'numsOf_GRJCJS_HDZYFZE_add', 'TOP_USER_DKYE_genFeat5_sum', 'YEARPURINCM_DKYE_min', 'WITHSIGN_DKFFE_genFeat5_mean', 'USAGEOVER_HDZYFZE_min', 'GJSIGN_DKYE_mean', 'NOUSAGE_DKYE_mean', 'NOUSAGE_DKYE_genFeat7_sum', 'WITHSIGN_count', 'NOUSAGE_HDSYYF_std', 'GRZHZT_WITHDRAW_SUBMIT_sum', 'TOP_USER_TOP_USER1_min', 'numsOf_GRYJCE_TOP_USER_diff', 'WITHDRAW_SUBMIT_HDSYYFZE_sum', 'USAGEOVER_WITHDRAW_min', 'numsOf_GRYJCE_DKLL_add', 'GRZHZT_HDSYYF_std', 'numsOf_HDSYYFZE_TOP_USER_diff', 'numsOf_GRZHYE_HDYF_diff', 'USAGEOVER_USAGE_RATE_std', 'numsOf_GJJRATE_WITHDRAW_diff', 'NOUSAGE_HDZYFZE_sum', 'USAGEOVER_HDZYF_std', 'WITHSIGN_HDSYYFZE_sum', 'TOP_USER_DKYE_genFeat3_sum', 'WITHSIGN2_GRYJCE_mean', 'GRZHZT_in_WITHSIGN2_prop', 'WITHSIGN2_DKFFE_genFeat1_sum', 'WITHSIGN2_DKFFE_genFeat5_mean', 'numsOf_HDZYF_GJJRATE_add', 'NOUSAGE_USAGE_RATE_mean', 'GRZHZT_DKYE_genFeat4_mean', 'WITHSIGN2_DKFFE_genFeat6_mean', 'USAGEOVER_WITHDRAW_mean', 'NOUSAGE_GRZHSNJZYE_sum', 'DWJJLX_DKYE_genFeat6_sum', 'DWSSHY_DKYE_genFeat6_sum', 'GJSIGN_GRYJCE_sum', 'WITHSIGN2_DKFFE_sum', 'TOP_USER1_DKYE_min', 'USAGEOVER_GRZHYE_sum', 'GJSIGN_DKFFE_std', 'USAGEOVER_DKYE_genFeat5_sum', 'GJSIGN_HDSYYFZE_std', 'GJSIGN_DKFFE_genFeat4_mean', 'numsOf_GRJCJS_USAGE_RATE_diff', 'DKLL_GRJCJS_sum', 'WITHSIGN2_DKFFE_genFeat3_mean', 'numsOf_GRYJCE_DKFFE_diff', 'WITHSIGN_HDSYYFZE_std', 'GRZHDNGJYE_DKFFE_max', 'NOUSAGE_WITHDRAW_mean', 'USAGEOVER_HDYF_max', 'numsOf_DKYE_WITHDRAW_SUBMIT_div', 'NOUSAGE_DKFFE_genFeat1_sum', 'GJJRATE_GRZHSNJZYE_sum', 'WITHSIGN2_GJJRATE_mean', 'GJJRATE_GJJRATE_max', 'GRZHZT_HDZYFZE_sum', 'USAGEOVER_WITHDRAW_SUBMIT_std', 'DWJJLX_DKYE_genFeat2_sum', 'TOP_USER1_DKFFE_sum', 'USAGEOVER_WITHDRAW_SUBMIT_sum', 'WITHSIGN_DKYE_genFeat1_sum', 'GJJRATE_DKFFE_sum', 'numsOf_DKLL_HDYF_add', 'NOUSAGE_HDSYYFZE_std', 'GRZHZT_DKYE_genFeat2_mean', 'WITHSIGN_DKLL_sum', 'numsOf_GRYJCE_YEARPURINCM_add', 'WITHSIGN2_GRJCJS_mean', 'WITHSIGN_GRYJCE_min', 'USAGE_RATE_DKFFE_max', 'numsOf_HDZYF_HDYF_div', 'GJJRATE_DKFFE_genFeat4_mean', 'numsOf_DKYE_DKLL_multi', 'WITHSIGN_GRZHYE_sum', 'HDZYFZE_TOP_USER_min', 'USAGEOVER_DKYE_genFeat4_sum', 'NOUSAGE_in_USAGEOVER_prop', 
                'WITHSIGN2_GRZHSNJZYE_std']+['--QIANGXIANGGUANXISHU_QUEREN',
                                             'DKYE_genFeat2_DKFFE_sum', 'DKFFE_genFeat3_DKYE_max', 'DKYE_genFeat6_WITHDRAW_mean', 'DKFFE_genFeat1_TOP_USER1_sum', 'DKYE_genFeat6_HDZYF_std', 'DKYE_genFeat4_TOP_USER_sum', 'DKFFE_genFeat2_DKXE_mean', 'DKYE_genFeat6_GRYJCE_max', 'DKYE_genFeat3_DKFFE_mean', 'DKFFE_genFeat7_GRZHYE_sum', 'DKFFE_genFeat3_YEARPURINCM_min', 'DKYE_genFeat7_GRZHDNGJYE_std', 'DKFFE_genFeat6_TOP_USER_std', 'DKFFE_genFeat3_TOP_USER_mean', 'DKFFE_genFeat7_DKXE_sum', 'ZHICHEN_in_WITHSIGN_prop', 'DKYE_genFeat5_DKLL_min', 'USAGEOVER_in_ZHIYE_prop', 'XINGBIE_GRJCJS_std',
                'ZHICHEN_USAGE_RATE_sum', 'XINGBIE_DKLL_max', 'DKFFE_genFeat4_DKXE_std', 'XINGBIE_GRYJCE_sum', 'DKYE_genFeat2_DKYE_min', 'DKYE_genFeat6_HDSYYF_mean', 'XINGBIE_WITHDRAW_SUBMIT_sum', 'ZHICHEN_DKFFE_sum', 'DKFFE_genFeat2_TOP_USER1_std', 'XINGBIE_GRZHYE_sum', 'DKYE_genFeat4_GRJCJS_sum', 'DKYE_genFeat5_WITHDRAW_max', 'DKYE_genFeat4_HDZYF_mean', 'DKYE_genFeat4_WITHDRAW_SUBMIT_std', 'DKFFE_genFeat2_GJJRATE_std', 'DKYE_genFeat1_USAGE_RATE_min', 'DKYE_genFeat5_HDSYYF_max', 'DKYE_genFeat5_HDSYYF_min', 'DKFFE_genFeat6_DKFFE_std', 'DKFFE_genFeat7_HDZYFZE_sum', 'DKYE_genFeat1_HDZYF_min', 'DKYE_genFeat2_TOP_USER_mean', 'DKYE_genFeat1_HDYF_std', 'GRZHZT_in_DWSSHY_prop', 'DKYE_genFeat3_HDSYYF_std', 'DKYE_genFeat2_DKYE_std', 'DKYE_genFeat6_USAGE_AMOUNT_std', 'ZHIYE_in_TOP_USER_prop', 'DKFFE_genFeat5_TOP_USER1_std', 'DKYE_genFeat6_WITHDRAW_max', 'DKYE_genFeat6_HDYF_mean', 'DKYE_genFeat7_GRJCJS_sum', 'DKFFE_genFeat1_GRJCJS_max', 'XINGBIE_USAGE_RATE_sum', 'DKFFE_genFeat6_DKYE_mean', 'DKYE_genFeat6_HDSYYF_sum', 'DKFFE_genFeat2_GRZHDNGJYE_max', 'DKFFE_genFeat6_WITHDRAW_SUBMIT_std', 'DKYE_genFeat5_TOP_USER_std', 'DKFFE_genFeat5_HDYF_mean', 'DKFFE_genFeat3_TOP_USER1_std', 'XINGBIE_DKFFE_genFeat3_mean', 'DKYE_genFeat2_HDZYF_std', 'DKYE_genFeat3_HDSYYF_min', 'DKYE_genFeat5_DKLL_sum', 'DKFFE_genFeat5_GRJCJS_mean', 'DKYE_genFeat7_GJJRATE_sum', 'DKFFE_genFeat6_TOP_USER1_mean', 'DKYE_genFeat1_GRYJCE_min', 'DKYE_genFeat1_YEARPURINCM_min', 'ZHIYE_GJJRATE_sum', 'DKYE_genFeat7_HDSYYFZE_max', 'DKFFE_genFeat2_GRZHSNJZYE_std', 'XINGBIE_DKFFE_genFeat4_sum', 'DKYE_genFeat7_USAGE_RATE_std', 'ZHIYE_DKFFE_sum', 'DKYE_genFeat2_HDYF_mean', 'DKYE_genFeat7_TOP_USER1_sum', 'DKFFE_genFeat7_HDZYF_max', 'DKYE_genFeat5_GJJRATE_sum', 'DKYE_genFeat6_HDSYYF_min', 'ZHICHEN_DKYE_genFeat2_sum', 'DKYE_genFeat7_HDYF_min', 'DKFFE_genFeat2_USAGE_RATE_std', 'XINGBIE_WITHDRAW_max', 'DKFFE_genFeat6_GRZHYE_sum', 'DKFFE_genFeat1_HDZYFZE_std', 'DKFFE_genFeat1_GRYJCE_min', 'DKYE_genFeat6_GRJCJS_sum', 'DKYE_genFeat7_DKYE_mean', 'DKYE_genFeat3_HDZYFZE_sum', 'DKYE_genFeat7_HDSYYF_min', 'DKFFE_genFeat2_GRJCJS_max', 'ZHICHEN_DKYE_mean', 'DKYE_genFeat4_HDSYYFZE_max', 'DKYE_genFeat6_HDSYYFZE_max', 'DKFFE_genFeat2_GRJCJS_std', 'DKYE_genFeat3_GRJCJS_max', 'DKFFE_genFeat1_GJJRATE_std', 'DKYE_genFeat6_GRYJCE_mean', 'DKYE_genFeat6_HDZYF_max', 'DKFFE_genFeat6_USAGE_AMOUNT_sum', 'DKFFE_genFeat3_HDZYFZE_sum', 'DKYE_genFeat7_YEARPURINCM_sum', 'DKYE_genFeat5_YEARPURINCM_mean', 'XINGBIE_DKFFE_genFeat5_sum', 'ZHICHEN_in_NOUSAGE_prop', 'ZHICHEN_DKYE_genFeat4_mean', 'DKYE_genFeat4_GRYJCE_sum', 'DKYE_genFeat2_WITHDRAW_min', 'DKFFE_genFeat2_GRJCJS_min', 'DKFFE_genFeat4_GRZHSNJZYE_std', 'DKYE_genFeat5_GRZHSNJZYE_sum', 'ZHICHEN_WITHDRAW_sum', 'ZHICHEN_GRJCJS_min', 'DKYE_genFeat3_GRJCJS_sum', 'DKYE_genFeat3_TOP_USER_sum', 'DKYE_genFeat5_HDSYYF_std', 'DKYE_genFeat3_DKYE_std', 'DKYE_genFeat2_YEARPURINCM_mean', 'DKYE_genFeat7_DKFFE_min', 'XINGBIE_DKFFE_genFeat4_mean', 'DKFFE_genFeat2_GRZHYE_sum', 'DKFFE_genFeat5_HDZYFZE_sum', 'DKFFE_genFeat1_DKFFE_sum', 'DKFFE_genFeat4_TOP_USER_std', 'DKYE_genFeat7_WITHDRAW_max', 'DKFFE_genFeat4_TOP_USER1_std', 'ZHIYE_GRZHSNJZYE_max', 'DKFFE_genFeat3_HDZYF_max', 'DKFFE_genFeat5_USAGE_AMOUNT_sum', 'DKFFE_genFeat3_TOP_USER_sum', 'DKFFE_genFeat6_GRJCJS_max', 'ZHICHEN_WITHDRAW_SUBMIT_sum', 'DKFFE_genFeat3_WITHDRAW_sum', 'DKFFE_genFeat2_GRYJCE_min', 'DKFFE_genFeat4_HDYF_mean', 'DKFFE_genFeat5_HDZYF_std', 'DKFFE_genFeat1_DKFFE_std', 'DKYE_genFeat1_HDSYYF_max', 'DKFFE_genFeat1_YEARPURINCM_max', 'ZHICHEN_HDZYF_sum', 'ZHIYE_DKYE_genFeat7_sum', 'DKYE_genFeat6_GRYJCE_min', 'DKFFE_genFeat4_DKLL_min', 'DKFFE_genFeat4_GJJRATE_std', 'DKFFE_genFeat2_HDSYYFZE_max', 'DKFFE_genFeat3_TOP_USER1_max', 'DKYE_genFeat6_DKLL_sum', 'DKFFE_genFeat4_HDZYF_min', 'ZHIYE_TOP_USER_sum', 'DKFFE_genFeat4_WITHDRAW_SUBMIT_sum', 'XINGBIE_in_ZHICHEN_prop', 'ZHIYE_WITHDRAW_SUBMIT_min', 'DKFFE_genFeat2_USAGE_AMOUNT_std', 'DKYE_genFeat7_WITHDRAW_min', 'DKFFE_genFeat2_YEARPURINCM_mean', 'DKYE_genFeat3_GRZHYE_mean', 'DKFFE_genFeat7_TOP_USER_std', 'DKFFE_genFeat1_TOP_USER1_mean', 'DKFFE_genFeat7_GJJRATE_std', 'DKYE_genFeat4_TOP_USER_std', 'DKYE_genFeat7_TOP_USER_max', 'DKYE_genFeat1_HDSYYF_min', 'DKYE_genFeat3_YEARPURINCM_sum', 'DKFFE_genFeat1_WITHDRAW_std', 'DKFFE_genFeat1_GRJCJS_sum', 'DKYE_genFeat3_HDZYF_min', 'DKFFE_genFeat5_DKXE_mean', 'XINGBIE_HDZYF_std', 'XINGBIE_GRJCJS_min', 'DKFFE_genFeat1_WITHDRAW_SUBMIT_max', 'XINGBIE_HDYF_max', 'ZHICHEN_DKFFE_genFeat2_sum', 'DKYE_genFeat2_YEARPURINCM_std', 'DKFFE_genFeat5_TOP_USER_mean', 'XINGBIE_GJJRATE_max', 'DKYE_genFeat2_GRJCJS_mean', 'DKYE_genFeat3_DKXE_max', 'DKYE_genFeat3_HDSYYFZE_mean', 'DKFFE_genFeat7_GRZHDNGJYE_sum', 'DKFFE_genFeat3_TOP_USER_max', 'XINGBIE_DKYE_genFeat2_sum', 'ZHICHEN_HDSYYF_std', 'DKYE_genFeat7_YEARPURINCM_mean', 'ZHICHEN_USAGE_AMOUNT_sum', 'ZHICHEN_GJJRATE_std', 'NOUSAGE_in_DWJJLX_prop', 'DKFFE_genFeat3_USAGE_AMOUNT_std', 'DKFFE_genFeat5_YEARPURINCM_max', 'DKYE_genFeat4_DKYE_mean', 'DKYE_genFeat5_TOP_USER1_sum', 'DKYE_genFeat6_GRYJCE_sum', 'DKYE_genFeat6_WITHDRAW_SUBMIT_mean', 'DKFFE_genFeat7_WITHDRAW_sum', 'DKYE_genFeat6_YEARPURINCM_sum', 'DKFFE_genFeat5_GRJCJS_min', 'DKFFE_genFeat2_DKFFE_std', 'DKFFE_genFeat2_YEARPURINCM_sum', 'DKFFE_genFeat3_DKFFE_min', 'WITHDRAW_GRJCJS_sum', 'DKYE_genFeat7_TOP_USER_mean', 'DKFFE_genFeat4_GRJCJS_sum', 'DKFFE_genFeat4_HDSYYF_sum', 'DKFFE_genFeat1_HDSYYFZE_max', 'DKFFE_genFeat7_YEARPURINCM_min', 'DKYE_genFeat4_GRYJCE_min', 'DKYE_genFeat3_GRYJCE_std', 'DKYE_genFeat3_TOP_USER1_mean', 'DKFFE_genFeat1_HDSYYFZE_std', 'DKYE_genFeat1_GRZHDNGJYE_std', 'DKFFE_genFeat5_HDSYYF_sum', 'DKYE_genFeat3_DKFFE_std', 'DKFFE_genFeat2_USAGE_AMOUNT_max', 'ZHICHEN_HDYF_sum', 'ZHICHEN_HDSYYFZE_max', 'DKYE_genFeat5_DKLL_std', 'DKFFE_genFeat5_HDYF_sum', 'DKFFE_genFeat7_GJJRATE_max', 'DKFFE_genFeat1_DKFFE_max', 'DKFFE_genFeat2_HDZYFZE_mean', 'ZHIYE_in_WITHSIGN2_prop', 'DKYE_genFeat2_GRZHDNGJYE_mean', 'ZHIYE_YEARPURINCM_min', 'WITHDRAW_DKXE_sum', 'DKYE_genFeat4_DKFFE_std', 'DKFFE_genFeat6_HDZYFZE_std', 'XINGBIE_GJJRATE_std', 'ZHICHEN_HDSYYF_sum', 'DKYE_genFeat2_DKXE_mean', 'DKFFE_genFeat4_HDZYFZE_min', 'DKFFE_genFeat6_HDZYF_sum', 'DKFFE_genFeat4_GRYJCE_std', 'DKFFE_genFeat7_DKYE_std', 'DKFFE_genFeat1_DKXE_std', 'DKFFE_genFeat5_WITHDRAW_SUBMIT_max', 'DKFFE_genFeat1_DKXE_mean', 'DKFFE_genFeat4_DKYE_max', 'DKFFE_genFeat1_HDZYF_min', 'DKYE_genFeat5_HDSYYFZE_std', 'DKYE_genFeat1_WITHDRAW_SUBMIT_max', 'DKYE_genFeat7_HDSYYFZE_mean', 'DKFFE_genFeat4_DKXE_max', 'DKFFE_genFeat4_GRZHDNGJYE_std', 'DKFFE_genFeat1_DKYE_std', 'DKYE_genFeat5_HDSYYFZE_max', 'DKYE_genFeat4_HDZYFZE_std', 'DKYE_genFeat1_GJJRATE_mean', 'ZHICHEN_GRZHYE_mean', 'DKYE_genFeat4_DKYE_min', 'DKYE_genFeat6_HDSYYFZE_mean', 'DKYE_genFeat1_WITHDRAW_mean', 'DKFFE_genFeat5_GRJCJS_max', 'DKYE_genFeat3_WITHDRAW_min', 'ZHICHEN_TOP_USER_std', 'XINGBIE_DKYE_genFeat4_sum', 'DKYE_genFeat2_HDSYYFZE_mean', 'ZHIYE_HDZYF_sum', 'DKFFE_genFeat5_TOP_USER1_sum', 'DKYE_genFeat5_HDYF_std', 'DKFFE_genFeat5_GRYJCE_min', 'ZHIYE_DKFFE_genFeat1_sum', 'DKFFE_genFeat7_GRYJCE_sum', 'XINGBIE_count', 'ZHICHEN_DKYE_genFeat2_mean', 'DKYE_genFeat4_GRJCJS_max', 'ZHICHEN_HDZYFZE_max', 'DKFFE_genFeat3_WITHDRAW_SUBMIT_sum', 'DKYE_genFeat4_TOP_USER1_sum', 'DKFFE_genFeat7_GRZHDNGJYE_mean', 'DKYE_genFeat1_WITHDRAW_max', 'DKFFE_genFeat2_HDSYYFZE_sum', 'DKFFE_genFeat1_HDSYYF_mean', 'DKYE_genFeat6_HDSYYF_max', 'DKFFE_genFeat4_YEARPURINCM_sum', 'DKYE_genFeat3_HDYF_sum', 'DKYE_genFeat3_WITHDRAW_SUBMIT_std', 'DKFFE_genFeat7_HDYF_std', 'DKYE_genFeat3_GRJCJS_std', 'ZHICHEN_GRYJCE_min', 'DKYE_genFeat7_GRJCJS_max', 'XINGBIE_DKXE_mean', 'DKYE_genFeat7_DKLL_sum', 'DKYE_genFeat2_DKLL_std', 'DKYE_genFeat3_GJJRATE_std', 'DKYE_genFeat6_HDZYFZE_sum', 'DKYE_genFeat5_GRZHYE_std', 'DKYE_genFeat5_GRYJCE_max', 'XINGBIE_GRZHYE_max', 'DKFFE_genFeat6_TOP_USER1_sum', 'DKFFE_genFeat7_WITHDRAW_max', 'ZHIYE_DKYE_genFeat7_mean', 'DKYE_genFeat6_TOP_USER1_std', 'XINGBIE_HDZYFZE_sum', 'DKFFE_genFeat2_DKFFE_sum', 'DKFFE_genFeat7_HDSYYF_mean', 'DKYE_genFeat6_HDSYYFZE_sum', 'DKYE_genFeat1_TOP_USER_sum', 'DKFFE_genFeat3_GRJCJS_sum', 'DKFFE_genFeat7_HDSYYF_max', 'DKYE_genFeat4_YEARPURINCM_max', 'DKFFE_genFeat7_HDZYFZE_mean', 'DKYE_genFeat4_GRJCJS_mean', 'DKYE_genFeat2_GRZHSNJZYE_sum', 'DKYE_genFeat5_WITHDRAW_SUBMIT_min', 'DKYE_genFeat6_WITHDRAW_sum', 'DKYE_genFeat3_DKYE_mean', 'ZHIYE_GRJCJS_sum', 'DKYE_genFeat6_DKFFE_std', 'DKFFE_genFeat4_HDSYYFZE_max', 'DKYE_genFeat7_GRZHYE_max', 'DKYE_genFeat5_WITHDRAW_SUBMIT_std', 'ZHICHEN_GRYJCE_std', 'DKYE_genFeat7_USAGE_RATE_max', 'DKYE_genFeat2_USAGE_AMOUNT_sum', 'DKYE_genFeat1_HDZYFZE_std', 'DKYE_genFeat4_WITHDRAW_mean', 'DKFFE_genFeat4_GRJCJS_min', 'DKYE_genFeat1_USAGE_AMOUNT_max', 'DKYE_genFeat6_GRZHSNJZYE_mean', 'DKYE_genFeat3_GRYJCE_max', 'DKYE_genFeat5_YEARPURINCM_max', 'DKFFE_genFeat7_DKYE_max', 'DKFFE_genFeat4_GRZHYE_std', 'ZHIYE_DKFFE_genFeat3_sum', 'XINGBIE_USAGE_RATE_max', 'DKFFE_genFeat2_TOP_USER_mean', 'DKFFE_genFeat2_GJJRATE_mean', 'DKYE_genFeat2_GRYJCE_max', 'DKYE_genFeat3_HDSYYF_max', 'DKYE_genFeat1_DKXE_max', 'DKFFE_genFeat4_GRZHSNJZYE_mean', 'DKYE_genFeat7_TOP_USER1_max', 'GRYJCE_GRZHYE_sum', 'DKYE_genFeat7_HDSYYFZE_std', 'DKYE_genFeat1_HDSYYFZE_std', 'GRZHYE_GRZHSNJZYE_sum', 'ZHICHEN_DKFFE_max', 'DKYE_genFeat7_DKXE_max', 'DKFFE_genFeat3_GRZHDNGJYE_mean', 'DKYE_genFeat7_TOP_USER1_std', 'DKYE_genFeat5_HDZYF_mean', 'DKYE_genFeat7_WITHDRAW_mean', 'DKFFE_genFeat4_GRZHSNJZYE_sum', 'DKFFE_genFeat1_GRZHYE_sum', 'DKFFE_genFeat4_DKLL_sum', 'DKYE_genFeat3_USAGE_RATE_mean', 'DKFFE_genFeat4_HDZYF_std', 'DKFFE_genFeat7_DKXE_min', 'DKFFE_genFeat1_GRZHYE_std', 'DKFFE_genFeat1_WITHDRAW_SUBMIT_min', 'DKYE_genFeat3_GRZHDNGJYE_sum', 'DKYE_genFeat3_YEARPURINCM_min', 'ZHICHEN_TOP_USER1_max', 'DKYE_genFeat3_USAGE_RATE_std', 'DKFFE_genFeat4_DKYE_mean', 'DKFFE_genFeat5_WITHDRAW_SUBMIT_min', 'DKYE_genFeat4_DKLL_mean', 'DKYE_genFeat7_GRZHDNGJYE_mean', 'DKFFE_genFeat6_GRZHSNJZYE_mean', 'DKFFE_genFeat2_HDZYF_min', 'DKFFE_genFeat6_GJJRATE_sum', 'ZHICHEN_DKLL_sum', 'DKYE_genFeat5_GRYJCE_std', 'DKFFE_genFeat1_GRZHDNGJYE_max', 'DKFFE_genFeat6_USAGE_RATE_mean', 'GRYJCE_GRJCJS_sum', 'DKYE_genFeat7_HDYF_sum', 'DKYE_genFeat5_GRZHYE_sum', 'DKYE_genFeat7_HDSYYFZE_sum', 'DKYE_genFeat3_GRYJCE_min', 'DKYE_genFeat6_DKLL_mean', 'numsOf_GRZHSNJZYE_WITHDRAW_div', 'DKYE_genFeat1_HDSYYF_mean', 'DKFFE_genFeat2_WITHDRAW_min', 'DKYE_genFeat4_HDSYYF_sum', 'DKYE_genFeat7_DKLL_min', 'DKYE_genFeat6_GRZHDNGJYE_std', 'XINGBIE_DKYE_genFeat3_sum', 'DKFFE_genFeat5_WITHDRAW_SUBMIT_std', 'ZHICHEN_DKYE_genFeat5_sum', 'DKYE_genFeat5_HDZYF_max', 'DKFFE_genFeat1_WITHDRAW_SUBMIT_std', 'DKFFE_genFeat3_USAGE_AMOUNT_max', 'ZHICHEN_HDZYF_max', 'DKYE_genFeat5_DKLL_max', 'DKYE_genFeat7_WITHDRAW_SUBMIT_std', 'DKFFE_genFeat6_GJJRATE_mean', 'ZHICHEN_GRZHDNGJYE_max', 'DKYE_genFeat1_YEARPURINCM_std', 'DKYE_genFeat3_GRZHDNGJYE_mean', 'DKFFE_genFeat1_GRZHDNGJYE_sum', 'DKFFE_genFeat2_YEARPURINCM_min', 'DKYE_genFeat7_YEARPURINCM_std', 'DKFFE_genFeat4_WITHDRAW_std', 'DKFFE_genFeat2_DKFFE_mean', 'DKFFE_genFeat7_TOP_USER1_sum', 'DKYE_genFeat7_HDZYFZE_mean', 'DKYE_genFeat3_GRYJCE_sum', 'DKFFE_genFeat5_HDSYYF_max', 'DKFFE_genFeat2_HDSYYFZE_std', 'DKYE_genFeat5_USAGE_AMOUNT_std', 'DKFFE_genFeat2_HDZYF_mean', 'DKFFE_genFeat7_DKFFE_sum', 'DKFFE_genFeat4_TOP_USER_mean', 'DKYE_genFeat3_DKLL_min', 'DKYE_genFeat7_GRZHYE_std', 'DKYE_genFeat1_GRJCJS_max', 'DKFFE_genFeat6_GRZHDNGJYE_mean', 'XINGBIE_DKYE_genFeat5_mean', 'USAGEOVER_in_XINGBIE_prop', 'USAGEOVER_in_ZHICHEN_prop', 'ZHICHEN_WITHDRAW_mean', 'DKYE_genFeat7_TOP_USER_sum', 'DKYE_genFeat5_HDZYFZE_std', 'DKYE_genFeat1_GRZHDNGJYE_sum', 'XINGBIE_GRZHSNJZYE_sum', 'DKYE_genFeat2_GRZHDNGJYE_sum', 'ZHICHEN_YEARPURINCM_min', 'DKFFE_genFeat6_WITHDRAW_max', 'DKYE_genFeat2_DKYE_sum', 'DKFFE_genFeat3_GRYJCE_mean', 'ZHIYE_HDYF_sum', 'DKYE_genFeat1_HDSYYFZE_mean', 'DKFFE_genFeat6_USAGE_AMOUNT_mean', 'DKFFE_genFeat7_GRJCJS_sum', 'DKYE_genFeat6_DKFFE_min', 'DKFFE_genFeat3_HDSYYF_mean', 'DKYE_genFeat2_GJJRATE_mean', 'DKYE_USAGE_AMOUNT_sum', 'DKYE_genFeat3_HDSYYF_sum', 'DKYE_genFeat7_WITHDRAW_SUBMIT_min', 'ZHICHEN_DKYE_genFeat3_sum', 'ZHIYE_WITHDRAW_SUBMIT_max', 'DKYE_genFeat3_WITHDRAW_std', 'DKYE_genFeat1_DKLL_sum', 'DKYE_genFeat4_DKXE_max', 'DKYE_genFeat1_GJJRATE_std', 'DKFFE_genFeat6_GRJCJS_std', 'DKYE_genFeat1_GRJCJS_min', 'DKYE_genFeat7_TOP_USER_std', 'DKFFE_genFeat4_USAGE_AMOUNT_mean', 'XINGBIE_DKYE_genFeat6_sum', 'XINGBIE_GRYJCE_mean', 'DKYE_genFeat6_WITHDRAW_min', 'DKFFE_genFeat4_HDZYF_sum', 'DKYE_genFeat4_GRZHDNGJYE_mean', 'ZHICHEN_GRZHSNJZYE_mean', 'DKFFE_genFeat6_USAGE_RATE_sum', 'DKFFE_genFeat4_WITHDRAW_SUBMIT_mean', 'DKYE_genFeat1_DKXE_mean', 'DKYE_genFeat1_GRZHSNJZYE_std', 'DKYE_genFeat2_TOP_USER1_max', 'numsOf_USAGE_RATE_WITHDRAW_div', 'DKYE_genFeat4_GRZHDNGJYE_std', 'DKYE_genFeat1_DKLL_min', 'DKFFE_genFeat5_GRZHSNJZYE_mean', 'DKYE_genFeat4_GRZHYE_std', 'ZHICHEN_HDZYFZE_min', 'DKYE_genFeat1_WITHDRAW_SUBMIT_std', 'DKYE_genFeat3_DKXE_mean', 'DKYE_genFeat6_YEARPURINCM_mean', 'DKYE_genFeat2_WITHDRAW_SUBMIT_sum', 'DKFFE_genFeat7_WITHDRAW_SUBMIT_max', 'DKFFE_genFeat7_WITHDRAW_std', 'DKFFE_genFeat7_TOP_USER1_std', 'DKFFE_genFeat2_HDYF_std', 'DKFFE_genFeat3_HDZYF_std', 'DKYE_genFeat2_GRYJCE_min', 'DKYE_genFeat7_GRZHSNJZYE_std', 'DKYE_genFeat6_TOP_USER1_sum', 'ZHICHEN_GRZHSNJZYE_max', 'DKFFE_genFeat3_USAGE_AMOUNT_sum', 'DKYE_genFeat3_GRJCJS_mean', 'DKYE_genFeat2_HDSYYFZE_sum', 'DKYE_genFeat7_GRYJCE_std', 'DKYE_genFeat2_WITHDRAW_max', 'DKFFE_genFeat7_GRZHSNJZYE_sum', 'DKFFE_genFeat3_GJJRATE_std', 'DKYE_genFeat2_HDSYYFZE_max', 'DKYE_genFeat2_HDSYYF_sum', 'DKFFE_genFeat1_HDSYYFZE_sum', 'ZHIYE_GRYJCE_sum', 'DKFFE_genFeat1_GJJRATE_mean', 'DKYE_genFeat6_DKYE_std', 'DKFFE_genFeat1_HDZYF_mean', 'DKYE_genFeat7_YEARPURINCM_max', 'DKYE_genFeat6_DKLL_max', 'XINGBIE_0', 'ZHICHEN_DKFFE_genFeat4_mean', 'DKYE_genFeat5_YEARPURINCM_min', 'DKYE_genFeat3_HDZYFZE_std', 'DKFFE_genFeat4_GRYJCE_min', 'DKYE_genFeat7_GRZHSNJZYE_max', 'ZHICHEN_in_TOP_USER_prop', 'DKFFE_genFeat5_YEARPURINCM_mean', 'ZHIYE_in_GJJRATE_prop', 'ZHICHEN_GRJCJS_sum', 'DKYE_genFeat4_YEARPURINCM_sum', 'DKYE_genFeat2_DKLL_max', 'DKFFE_genFeat2_DKXE_sum', 'DKFFE_genFeat3_USAGE_RATE_sum', 'DKYE_genFeat1_USAGE_AMOUNT_mean', 'XINGBIE_DKYE_genFeat1_sum', 'XINGBIE_HDSYYF_min', 'DKYE_genFeat5_GRYJCE_sum', 'DKFFE_genFeat7_WITHDRAW_SUBMIT_sum', 'GRZHZT_count', 'DKFFE_genFeat6_TOP_USER1_std', 'ZHICHEN_count', 'DKYE_genFeat4_DKLL_sum', 'DKYE_genFeat6_GRZHDNGJYE_mean', 'DKYE_genFeat6_WITHDRAW_SUBMIT_std', 'HDSYYFZE_USAGE_AMOUNT_sum', 'DKYE_genFeat1_DKYE_sum', 'DKYE_genFeat2_HDZYF_min', 'DKYE_genFeat6_HDSYYF_std', 'DKYE_genFeat3_DKLL_sum', 'DKYE_genFeat2_DKXE_std', 'DKYE_genFeat4_DKXE_mean', 'DKYE_genFeat2_USAGE_RATE_sum', 'DKFFE_genFeat3_GRJCJS_min', 'XINGBIE_DKLL_mean', 'DKFFE_genFeat2_WITHDRAW_SUBMIT_std', 'DKFFE_genFeat3_DKFFE_std', 'DKYE_genFeat6_DKXE_sum', 'DKFFE_genFeat1_YEARPURINCM_mean', 'DKFFE_genFeat1_HDSYYF_max', 'DKFFE_genFeat5_HDSYYFZE_std', 'DKFFE_genFeat7_HDYF_mean', 'DKYE_genFeat5_YEARPURINCM_std', 'DKFFE_genFeat3_HDZYFZE_std', 'DKFFE_genFeat2_TOP_USER_max', 'ZHIYE_DKYE_genFeat1_sum', 'DKFFE_genFeat3_DKXE_std', 'DKYE_genFeat7_GRZHYE_mean', 'DKYE_genFeat1_GRYJCE_mean', 'ZHICHEN_0', 'DKYE_genFeat5_WITHDRAW_std', 'DKYE_genFeat1_HDYF_mean', 'DKFFE_genFeat6_YEARPURINCM_std', 'DKYE_genFeat5_USAGE_AMOUNT_max', 'DKYE_genFeat1_HDZYF_mean', 'DKFFE_genFeat4_HDYF_std', 'DKFFE_genFeat4_HDZYFZE_mean', 'DKFFE_genFeat4_HDZYF_max', 'DKFFE_genFeat7_GRJCJS_max', 'DKYE_genFeat6_GRZHSNJZYE_std', 'DKFFE_genFeat1_GRZHYE_mean', 'DKFFE_genFeat1_DKYE_sum', 'DKFFE_genFeat3_HDZYF_min', 'DKFFE_genFeat4_DKFFE_sum', 'DKFFE_genFeat4_GJJRATE_mean', 'ZHIYE_DKYE_genFeat3_sum', 'DKFFE_genFeat7_GRZHSNJZYE_mean', 'DKFFE_genFeat5_USAGE_AMOUNT_max', 'DKFFE_genFeat6_GRYJCE_mean', 'DKFFE_genFeat6_DKLL_min', 'DKYE_genFeat4_WITHDRAW_SUBMIT_max', 'DKFFE_genFeat7_YEARPURINCM_std', 'XINGBIE_USAGE_RATE_std', 'DKYE_genFeat3_GRZHSNJZYE_mean', 'DKYE_genFeat6_TOP_USER1_mean', 'DKYE_genFeat4_HDSYYFZE_min', 'DKYE_genFeat4_GRYJCE_std', 'ZHICHEN_DKYE_genFeat1_sum', 'DKFFE_genFeat5_USAGE_RATE_std', 'DKYE_genFeat3_TOP_USER1_max', 'DKYE_genFeat1_TOP_USER1_sum', 'DKFFE_genFeat1_DKLL_min', 'DKYE_genFeat6_DKYE_max', 'DKFFE_genFeat3_USAGE_RATE_mean', 'DKYE_genFeat2_HDZYFZE_std', 'DKFFE_genFeat1_WITHDRAW_max', 'DKFFE_genFeat6_WITHDRAW_SUBMIT_mean', 'DKYE_genFeat3_HDYF_mean', 'DKYE_genFeat1_WITHDRAW_SUBMIT_min', 'DKYE_genFeat5_USAGE_AMOUNT_sum', 'DKFFE_genFeat6_GRYJCE_sum', 'DKFFE_genFeat1_HDYF_sum', 'DKFFE_genFeat4_WITHDRAW_SUBMIT_max', 'ZHICHEN_TOP_USER1_std', 'ZHIYE_in_GJSIGN_prop', 'DKFFE_genFeat6_HDSYYF_std', 'ZHICHEN_in_DWSSHY_prop', 'DKYE_genFeat5_WITHDRAW_sum', 'DKYE_genFeat3_WITHDRAW_sum', 'ZHIYE_DKYE_genFeat6_sum', 'DKFFE_genFeat1_GRYJCE_mean', 'DKYE_genFeat5_WITHDRAW_SUBMIT_mean', 'DKFFE_genFeat1_DKFFE_min', 'DKYE_genFeat6_HDZYFZE_mean', 'DKFFE_genFeat2_GRZHSNJZYE_mean', 'DKYE_genFeat2_GRJCJS_std', 'XINGBIE_DKFFE_sum', 'DKYE_genFeat4_USAGE_AMOUNT_max', 'DKYE_genFeat5_DKFFE_sum', 'ZHICHEN_WITHDRAW_SUBMIT_min', 'DKFFE_genFeat3_WITHDRAW_mean', 'ZHICHEN_TOP_USER_sum', 'DKFFE_genFeat6_TOP_USER_mean', 'DKFFE_genFeat3_DKLL_max', 'DKYE_genFeat6_HDYF_sum', 'DKFFE_genFeat7_USAGE_RATE_mean', 'DKFFE_genFeat2_DKLL_mean', 'DKFFE_genFeat1_WITHDRAW_SUBMIT_mean', 'DKFFE_genFeat7_DKFFE_std', 'XINGBIE_HDZYF_max', 'DKFFE_genFeat5_GRJCJS_sum', 'ZHICHEN_GRJCJS_mean', 'DKYE_genFeat5_TOP_USER1_max', 'DKFFE_genFeat7_HDSYYF_std', 'XINGBIE_DKFFE_genFeat1_mean', 'DKFFE_genFeat5_DKLL_min', 'DKYE_genFeat3_TOP_USER1_std', 'DKYE_genFeat4_USAGE_RATE_mean', 'DKFFE_genFeat5_GJJRATE_mean', 'ZHICHEN_TOP_USER1_sum', 'DKYE_genFeat7_HDYF_mean', 'XINGBIE_DKFFE_genFeat2_sum', 'ZHICHEN_HDZYF_min', 'DKFFE_genFeat4_DKXE_mean', 'DKFFE_genFeat6_WITHDRAW_mean', 'ZHICHEN_WITHDRAW_max', 'DKFFE_genFeat1_TOP_USER_sum', 'DKYE_genFeat5_USAGE_RATE_mean', 'ZHICHEN_GJJRATE_sum', 'DKYE_genFeat3_HDZYFZE_min', 'DKFFE_genFeat1_HDZYF_std', 'DKFFE_genFeat5_HDZYFZE_min', 'DKYE_genFeat7_WITHDRAW_SUBMIT_sum', 'DKYE_genFeat6_TOP_USER_std', 'DKFFE_genFeat7_GJJRATE_mean', 'DKFFE_genFeat6_GRZHYE_mean', 'DKYE_genFeat1_DKYE_max', 'DKFFE_genFeat3_GRJCJS_mean', 'XINGBIE_GRJCJS_mean', 'DKYE_genFeat1_TOP_USER_std', 'DKFFE_genFeat2_WITHDRAW_SUBMIT_min', 'DKYE_genFeat2_GRZHSNJZYE_mean', 'DKYE_genFeat4_GRZHYE_mean', 'ZHICHEN_HDSYYFZE_sum', 'ZHICHEN_GRZHDNGJYE_sum', 'DKYE_genFeat7_WITHDRAW_sum', 'DKFFE_genFeat5_DKFFE_max', 'DKFFE_genFeat7_HDSYYFZE_std', 'DKYE_genFeat1_TOP_USER1_max', 'DKFFE_genFeat7_GRZHSNJZYE_std', 'DKYE_genFeat7_HDSYYFZE_min', 'DKFFE_genFeat6_TOP_USER1_max', 'DKFFE_genFeat4_YEARPURINCM_mean', 'ZHIYE_DKYE_genFeat2_sum', 'DKYE_genFeat4_HDZYF_max', 'DKFFE_genFeat2_USAGE_RATE_mean', 'DKFFE_genFeat6_GRYJCE_min', 'DKFFE_genFeat3_DKLL_std', 'DKFFE_genFeat7_TOP_USER_sum', 'DKYE_genFeat6_GJJRATE_mean', 'DKYE_genFeat1_WITHDRAW_SUBMIT_sum', 'DKFFE_genFeat6_YEARPURINCM_min', 'ZHICHEN_DKYE_genFeat7_mean', 'DKFFE_genFeat5_YEARPURINCM_std', 'DKFFE_genFeat1_TOP_USER1_max', 'DKFFE_genFeat5_HDZYFZE_mean', 'DKYE_genFeat5_HDYF_sum', 'DKFFE_genFeat6_DKFFE_sum', 'ZHICHEN_USAGE_AMOUNT_max', 'DKYE_genFeat7_HDZYF_sum', 'DKYE_genFeat3_HDZYF_max', 'DKFFE_genFeat7_HDZYF_sum', 'DKFFE_genFeat5_GRZHSNJZYE_sum', 'XINGBIE', 'ZHIYE_in_NOUSAGE_prop', 'ZHICHEN_DKYE_genFeat5_mean', 'DKYE_genFeat4_HDYF_max', 'XINGBIE_USAGE_AMOUNT_sum', 'ZHIYE_HDSYYF_sum', 'DKFFE_genFeat2_HDZYFZE_max', 'DKFFE_genFeat3_USAGE_RATE_std', 'DKFFE_genFeat5_HDZYF_min', 'ZHIYE_DKFFE_genFeat4_sum', 'DKFFE_genFeat7_WITHDRAW_SUBMIT_std', 'DKYE_genFeat3_WITHDRAW_SUBMIT_mean', 'DKYE_genFeat6_USAGE_AMOUNT_sum', 'DKYE_genFeat1_HDZYFZE_mean', 'DKYE_genFeat5_GRZHDNGJYE_std', 'numsOf_YEARPURINCM_WITHDRAW_div', 'DKFFE_genFeat3_HDZYFZE_min', 'DKFFE_genFeat1_GRYJCE_sum', 'DKFFE_genFeat5_TOP_USER1_max', 'DKYE_genFeat4_HDZYF_std', 'DKYE_genFeat6_DKLL_min', 'GRZHZT_in_XINGBIE_prop', 'DKYE_genFeat2_WITHDRAW_SUBMIT_mean', 'ZHICHEN_YEARPURINCM_sum', 'DKFFE_genFeat6_GJJRATE_std', 'DKFFE_genFeat7_DKYE_sum', 'DKFFE_genFeat1_TOP_USER_std', 'DKYE_genFeat2_GRZHSNJZYE_std', 'DKYE_genFeat1_DKFFE_sum', 'ZHIYE_YEARPURINCM_sum', 'DKFFE_genFeat1_TOP_USER_max', 'XINGBIE_DKYE_genFeat4_mean', 'DKFFE_genFeat5_USAGE_RATE_mean', 'DKFFE_genFeat6_HDSYYFZE_sum', 'DKFFE_genFeat7_DKLL_std', 'DKFFE_genFeat4_DKYE_std', 'DKYE_genFeat6_GRZHYE_std', 'DKFFE_genFeat1_GJJRATE_sum', 'DKFFE_genFeat3_HDYF_mean', 'ZHICHEN_1', 'DKYE_genFeat1_GRYJCE_max', 'DKYE_genFeat4_YEARPURINCM_std', 'DKYE_genFeat4_HDSYYFZE_mean', 'DKFFE_genFeat3_TOP_USER1_mean', 'XINGBIE_DKFFE_mean', 'DKFFE_genFeat2_DKFFE_max', 'DKYE_genFeat1_HDZYFZE_min', 'ZHICHEN_HDZYF_std', 'DKYE_genFeat3_GRZHDNGJYE_std', 'DKFFE_genFeat2_WITHDRAW_std', 'DKFFE_genFeat1_DKLL_std', 'DKFFE_genFeat2_HDSYYF_sum', 'ZHICHEN_GRYJCE_mean', 'DKFFE_genFeat3_HDZYFZE_max', 'DKYE_genFeat3_USAGE_AMOUNT_max', 'DKFFE_genFeat2_USAGE_AMOUNT_mean', 'DKYE_genFeat3_TOP_USER_mean', 'DKFFE_genFeat7_YEARPURINCM_max', 'DKYE_genFeat5_DKYE_mean', 'ZHICHEN', 'DKFFE_genFeat1_DKYE_max', 'DKYE_genFeat1_GJJRATE_sum', 'DKFFE_genFeat5_WITHDRAW_max', 'GRZHYE_GRJCJS_sum', 'DKFFE_genFeat7_USAGE_RATE_sum', 'XINGBIE_HDZYFZE_max', 'DKYE_genFeat7_USAGE_RATE_min', 'DKYE_genFeat7_HDSYYF_max', 'DKYE_genFeat1_GRZHYE_mean', 'DKYE_genFeat7_DKYE_sum', 'DKFFE_genFeat7_TOP_USER_mean', 'DKYE_genFeat7_GRYJCE_max', 'DKFFE_genFeat4_GRJCJS_max', 'DKYE_genFeat1_DKFFE_min', 'DKFFE_genFeat1_DKXE_sum', 'DKFFE_genFeat6_GRZHSNJZYE_std', 'DKYE_genFeat2_USAGE_RATE_mean', 'DKYE_genFeat2_DKLL_mean', 'DKYE_genFeat5_DKYE_std', 'DKFFE_genFeat3_GRYJCE_std', 'DKYE_genFeat6_GJJRATE_std', 'DKFFE_genFeat4_GJJRATE_sum', 'DKFFE_genFeat3_GRZHYE_sum', 'DKFFE_genFeat4_GRZHYE_sum', 'DKFFE_genFeat6_HDSYYFZE_mean', 'ZHICHEN_DKYE_genFeat6_mean', 'GRJCJS_GRZHYE_sum', 'DKYE_genFeat7_GRYJCE_mean', 'ZHICHEN_in_GRZHZT_prop', 'DKYE_genFeat2_TOP_USER1_sum', 'DKYE_genFeat1_HDZYF_std', 'DKYE_genFeat6_USAGE_RATE_std', 'DKFFE_genFeat6_DKLL_sum', 'DKYE_genFeat1_WITHDRAW_sum', 'XINGBIE_in_ZHIYE_prop', 'DKFFE_genFeat2_HDZYFZE_sum', 'DKFFE_genFeat5_DKYE_mean', 'DKYE_genFeat2_DKXE_max', 'DKYE_genFeat4_TOP_USER1_max', 'DKYE_genFeat2_HDSYYF_std', 'DKYE_genFeat3_HDSYYFZE_min', 'DKYE_genFeat6_GRZHSNJZYE_sum', 'DKYE_genFeat7_GRZHYE_sum', 'DKYE_genFeat5_GRZHDNGJYE_mean', 'DKYE_genFeat5_DKYE_sum', 'DKFFE_genFeat5_HDSYYFZE_mean', 'ZHICHEN_GRZHYE_std', 'DKYE_genFeat5_HDZYF_min', 'DKYE_genFeat7_GRZHSNJZYE_mean', 'DKFFE_genFeat1_DKFFE_mean', 'ZHICHEN_HDZYF_mean', 'DKFFE_genFeat5_HDZYF_mean', 'DKYE_genFeat6_GRJCJS_min', 'DKYE_genFeat5_GRJCJS_max', 'DKFFE_genFeat3_HDZYF_mean', 'DKFFE_genFeat7_TOP_USER_max', 'DKYE_genFeat4_HDZYFZE_min', 'DKFFE_genFeat3_GRZHSNJZYE_std', 'DKYE_genFeat7_YEARPURINCM_min', 'DKYE_genFeat2_TOP_USER_std', 'DKFFE_genFeat7_TOP_USER1_mean', 'DKYE_genFeat4_GRZHSNJZYE_mean', 'DKYE_genFeat7_DKXE_std', 'DKYE_genFeat3_GRZHSNJZYE_sum', 'DKYE_genFeat5_GRZHYE_mean', 'DKFFE_genFeat7_USAGE_AMOUNT_min', 'DKYE_genFeat3_DKYE_max', 'DKYE_genFeat4_HDSYYF_max', 'DKYE_genFeat4_TOP_USER1_mean', 'DKYE_genFeat7_DKXE_sum', 'DKFFE_genFeat2_GRZHSNJZYE_max', 'DKYE_genFeat5_HDSYYFZE_sum', 'DKYE_genFeat2_GRZHYE_std', 'DKYE_genFeat3_HDSYYFZE_std', 'DKYE_genFeat2_YEARPURINCM_min', 'DKFFE_genFeat7_WITHDRAW_SUBMIT_mean', 'DKYE_genFeat4_GRYJCE_mean', 'DKYE_genFeat1_TOP_USER1_mean', 'DKFFE_genFeat6_WITHDRAW_sum', 'DKYE_genFeat1_HDSYYFZE_min', 'DKYE_genFeat4_USAGE_RATE_sum', 'DKYE_genFeat4_HDYF_mean', 'DKFFE_genFeat5_WITHDRAW_sum', 'DKYE_genFeat1_DKFFE_mean', 'DKYE_genFeat4_YEARPURINCM_mean', 'XINGBIE_TOP_USER1_sum', 'DKFFE_genFeat2_TOP_USER1_max', 'ZHIYE_in_GRZHZT_prop', 'ZHICHEN_DKXE_std', 'DKYE_genFeat1_GRZHSNJZYE_mean', 'DKYE_genFeat3_DKYE_min', 'DKYE_genFeat7_GJJRATE_max', 'DKFFE_genFeat7_TOP_USER1_max', 'ZHIYE_HDSYYF_max', 'DKYE_genFeat3_HDSYYFZE_sum', 'DKYE_genFeat7_HDZYFZE_sum', 'DKYE_genFeat3_HDZYFZE_mean', 'ZHICHEN_GJJRATE_mean', 'DKYE_genFeat3_DKYE_sum', 'DKYE_genFeat1_DKLL_max', 'DKFFE_genFeat7_DKFFE_mean', 'DKFFE_genFeat4_USAGE_AMOUNT_std', 'DKYE_genFeat1_USAGE_AMOUNT_sum', 'DKYE_genFeat2_HDZYF_max', 'DKYE_genFeat5_GRJCJS_sum', 'DKYE_genFeat5_DKYE_min', 'DKFFE_genFeat3_HDZYF_sum', 'ZHICHEN_DKYE_genFeat1_mean', 'DKFFE_genFeat5_DKXE_sum', 'XINGBIE_DKYE_genFeat6_mean', 'DKYE_genFeat7_HDZYF_std', 'DKFFE_genFeat7_DKLL_mean', 'DKYE_genFeat6_HDSYYFZE_min', 'DKYE_genFeat1_GRZHDNGJYE_max', 'DKYE_genFeat4_WITHDRAW_sum', 'DKYE_genFeat5_USAGE_RATE_std', 'DKYE_genFeat7_WITHDRAW_SUBMIT_max', 'DKYE_genFeat7_GRJCJS_std', 'DKYE_genFeat2_HDZYF_sum', 'DKYE_genFeat6_TOP_USER1_max', 'DKFFE_genFeat5_YEARPURINCM_sum', 'DKFFE_genFeat1_GRYJCE_std', 'DKYE_genFeat3_TOP_USER_max', 'DKYE_genFeat3_USAGE_RATE_min', 'DKYE_genFeat7_DKLL_std', 'ZHICHEN_GRZHDNGJYE_std', 'DKFFE_genFeat5_GRZHYE_std', 'ZHIYE_DKYE_genFeat4_sum', 'DKYE_genFeat1_DKXE_sum', 'DKFFE_genFeat4_WITHDRAW_SUBMIT_std', 'DKFFE_genFeat6_HDZYF_std', 'DKFFE_genFeat6_WITHDRAW_SUBMIT_max', 'DKYE_genFeat7_HDZYF_max', 'DKFFE_genFeat4_WITHDRAW_SUBMIT_min', 'DKYE_genFeat7_HDZYF_mean', 'DKFFE_genFeat6_DKFFE_min', 'DKFFE_genFeat3_DKFFE_sum', 'DKYE_genFeat5_DKXE_mean', 'ZHICHEN_DKYE_genFeat4_sum', 'DKFFE_genFeat4_TOP_USER1_sum', 'DKFFE_genFeat1_HDZYFZE_sum', 'DKYE_genFeat4_WITHDRAW_min', 'DKFFE_genFeat7_HDZYFZE_max', 'DKFFE_genFeat7_YEARPURINCM_mean', 'XINGBIE_DKYE_sum', 'ZHICHEN_TOP_USER1_mean', 'DKYE_genFeat4_WITHDRAW_std', 'DKFFE_genFeat3_GRJCJS_max', 'DKFFE_genFeat2_DKLL_max', 'DKFFE_genFeat6_GRZHDNGJYE_sum', 'DKFFE_genFeat3_GRZHYE_mean', 'DKFFE_genFeat2_TOP_USER_std', 'DKYE_genFeat4_GRJCJS_std', 'DKFFE_genFeat1_HDZYF_max', 'DKFFE_genFeat3_DKXE_sum', 'DKFFE_genFeat3_GRZHDNGJYE_max', 'DKYE_genFeat3_GRZHYE_sum', 'DKFFE_genFeat4_DKFFE_max', 'DKYE_genFeat6_DKLL_std', 'DKFFE_genFeat7_HDZYF_mean', 'DKYE_genFeat2_WITHDRAW_SUBMIT_std', 'DKFFE_genFeat4_TOP_USER_sum', 'DKFFE_genFeat5_GJJRATE_sum', 'DKFFE_genFeat6_HDSYYF_mean', 'DKYE_genFeat3_DKLL_mean', 'DKYE_genFeat4_HDSYYFZE_std', 'DKFFE_genFeat6_HDSYYF_sum', 'DKFFE_genFeat3_DKFFE_mean', 'DKFFE_genFeat7_USAGE_AMOUNT_mean', 'DKYE_genFeat6_DKXE_max', 'XINGBIE_HDZYF_sum', 'DKYE_genFeat7_USAGE_AMOUNT_sum', 'XINGBIE_DKYE_genFeat1_mean', 'DKFFE_genFeat6_HDSYYFZE_max', 'ZHICHEN_DKFFE_genFeat2_mean', 'ZHICHEN_DKYE_sum', 'DKYE_genFeat7_USAGE_AMOUNT_mean', 'DKYE_genFeat4_HDSYYF_min', 'DKYE_genFeat6_WITHDRAW_SUBMIT_min', 'ZHICHEN_GRYJCE_max', 'DKYE_genFeat3_USAGE_AMOUNT_sum', 'XINGBIE_DKFFE_min', 'DKFFE_genFeat4_USAGE_RATE_mean', 'DKFFE_genFeat5_GRZHDNGJYE_std', 'DKYE_genFeat4_GRZHDNGJYE_sum', 'DKFFE_genFeat6_HDZYFZE_sum', 'ZHICHEN_HDSYYFZE_mean', 'DKYE_genFeat7_DKFFE_mean', 'DKFFE_genFeat5_DKXE_std', 'DKFFE_genFeat6_HDYF_std', 'DKYE_genFeat5_WITHDRAW_min', 'USAGEOVER_in_GRZHZT_prop', 'DKFFE_genFeat2_TOP_USER1_sum', 'ZHICHEN_DKLL_min', 'DKFFE_genFeat2_WITHDRAW_mean', 'DKYE_genFeat3_HDZYF_mean', 'DKYE_genFeat4_TOP_USER_mean', 'DKYE_genFeat1_TOP_USER_mean', 'numsOf_GRZHYE_YEARPURINCM_div', 'DKFFE_genFeat1_USAGE_AMOUNT_std', 'DKFFE_genFeat2_YEARPURINCM_std', 'ZHIYE_in_DWSSHY_prop', 'DKFFE_genFeat2_GRZHYE_std', 'DKYE_genFeat6_GJJRATE_sum', 'DKFFE_genFeat6_USAGE_AMOUNT_max', 'DKFFE_genFeat7_GRJCJS_std', 'ZHICHEN_DKYE_max', 'DKYE_genFeat7_DKFFE_std', 'DKYE_genFeat4_TOP_USER_max', 'DKYE_genFeat2_DKXE_sum', 'DKYE_genFeat5_GRZHSNJZYE_std', 'DKYE_genFeat1_HDZYF_sum', 'DKFFE_genFeat1_USAGE_RATE_mean', 'DKFFE_genFeat6_DKLL_mean', 'XINGBIE_DKFFE_std', 'XINGBIE_GRYJCE_std', 'DKYE_genFeat5_GRYJCE_min', 'DKFFE_genFeat1_HDZYFZE_mean', 'DKYE_genFeat4_GRZHYE_sum', 'ZHICHEN_DKYE_genFeat3_mean', 'DKFFE_genFeat5_TOP_USER_std', 'DKYE_genFeat7_GRYJCE_min', 'DKYE_genFeat2_TOP_USER_sum', 'ZHIYE_WITHDRAW_min', 'ZHICHEN_USAGE_RATE_mean', 'DKYE_genFeat5_GRZHDNGJYE_sum', 'DKYE_genFeat4_USAGE_AMOUNT_std', 'DKYE_genFeat3_DKFFE_sum', 'ZHIYE_DKYE_sum', 'DKYE_genFeat3_GRJCJS_min', 'DKFFE_genFeat4_YEARPURINCM_min', 'DKFFE_genFeat5_HDZYF_max', 'ZHICHEN_DKXE_mean', 'DKYE_genFeat7_HDYF_max', 'DKFFE_genFeat1_DKYE_mean', 'DKYE_genFeat4_DKLL_min', 'ZHICHEN_WITHDRAW_std', 'DKYE_genFeat1_HDSYYFZE_sum', 'DKYE_genFeat4_WITHDRAW_SUBMIT_sum', 'DKFFE_genFeat4_DKYE_sum', 'ZHICHEN_YEARPURINCM_mean', 'DKFFE_genFeat6_DKLL_std', 'numsOf_DKYE_WITHDRAW_div', 'DKYE_genFeat3_HDZYF_std', 'DKFFE_genFeat5_DKLL_mean', 'DKYE_genFeat1_WITHDRAW_min', 'DKFFE_genFeat7_YEARPURINCM_sum', 'DKFFE_genFeat6_HDYF_sum', 'DKFFE_genFeat4_HDSYYFZE_mean', 'DKFFE_genFeat5_DKYE_std', 'ZHICHEN_DKYE_std', 'DKYE_genFeat2_TOP_USER1_std', 'ZHICHEN_DKFFE_genFeat5_sum', 'DKFFE_genFeat6_YEARPURINCM_max', 'DKYE_genFeat5_GRJCJS_mean', 'XINGBIE_DKLL_std', 'ZHICHEN_GRZHSNJZYE_std', 'DKYE_genFeat3_DKLL_max', 'ZHICHEN_USAGE_RATE_min', 'DKYE_genFeat2_GJJRATE_sum', 'DKFFE_genFeat1_HDZYFZE_min', 'DKYE_genFeat4_USAGE_AMOUNT_sum', 'DKFFE_genFeat6_DKXE_mean', 'DKFFE_genFeat2_YEARPURINCM_max', 'DKYE_genFeat4_HDZYFZE_mean', 'DKFFE_genFeat2_TOP_USER1_mean', 'DKYE_genFeat3_WITHDRAW_max', 'DKYE_genFeat6_DKFFE_mean', 'DKYE_genFeat2_GRYJCE_mean', 'DKYE_genFeat5_HDZYFZE_mean', 'DKYE_genFeat7_DKXE_mean', 'ZHICHEN_DKLL_std', 'ZHIYE_ZHICHEN_count', 'DKYE_genFeat7_GRJCJS_min', 'XINGBIE_USAGE_RATE_min', 'ZHICHEN_DKYE_genFeat6_sum', 'XINGBIE_DKYE_genFeat5_sum', 'DKFFE_genFeat4_GRYJCE_sum', 'DKYE_genFeat1_YEARPURINCM_mean', 'DKYE_genFeat5_HDSYYF_sum', 'DKYE_genFeat4_YEARPURINCM_min', 'DKFFE_genFeat6_HDZYF_min', 'DKYE_genFeat6_USAGE_RATE_min', 'DKFFE_genFeat4_DKLL_max', 'DKFFE_genFeat7_HDYF_max', 'DKFFE_genFeat7_HDSYYFZE_max', 'DKYE_genFeat5_HDZYFZE_sum', 'DKFFE_genFeat5_USAGE_AMOUNT_mean', 'DKFFE_genFeat2_DKYE_std', 'DKFFE_genFeat3_HDSYYF_max', 'DKFFE_genFeat6_GRYJCE_std', 'DKFFE_genFeat4_DKFFE_std', 'GRZHZT_in_DWJJLX_prop', 'DKFFE_genFeat2_GRZHDNGJYE_std', 'ZHICHEN_WITHDRAW_SUBMIT_std', 'DKYE_genFeat1_USAGE_RATE_mean', 'DKFFE_genFeat2_GRYJCE_mean', 'DKFFE_genFeat3_GRZHDNGJYE_std', 'DKFFE_genFeat2_HDSYYFZE_mean', 'DKFFE_genFeat2_WITHDRAW_sum', 'DKFFE_genFeat4_HDSYYF_max', 'DKYE_genFeat6_HDZYF_min', 'DKFFE_genFeat1_HDZYF_sum', 'DKYE_genFeat2_HDSYYFZE_min', 'DKFFE_genFeat3_TOP_USER1_sum', 'DKFFE_genFeat7_DKYE_mean', 'DKFFE_genFeat7_USAGE_AMOUNT_max', 'DKFFE_genFeat7_GRZHSNJZYE_min', 'DKFFE_genFeat3_WITHDRAW_SUBMIT_max', 'DKYE_genFeat3_WITHDRAW_mean', 'DKFFE_genFeat5_GRYJCE_max', 'DKYE_genFeat2_HDZYFZE_min', 'DKYE_genFeat6_HDZYFZE_std', 'XINGBIE_DKYE_mean', 'DKYE_genFeat2_USAGE_RATE_std', 'DKFFE_genFeat4_USAGE_RATE_std', 'DKYE_genFeat2_DKFFE_min', 'DKFFE_genFeat5_YEARPURINCM_min', 'DKFFE_genFeat2_DKYE_max', 'DKYE_genFeat7_DKLL_max', 'DKYE_genFeat5_USAGE_RATE_sum', 'DKFFE_genFeat4_TOP_USER1_mean', 'DKFFE_genFeat6_DKYE_sum', 'ZHIYE_GRZHSNJZYE_std', 'DKFFE_genFeat3_GRYJCE_max', 'DKFFE_genFeat7_DKFFE_max', 'DKFFE_genFeat7_HDSYYFZE_mean', 'DKFFE_genFeat1_HDYF_mean', 'XINGBIE_in_USAGEOVER_prop', 'DKFFE_genFeat3_WITHDRAW_SUBMIT_std', 'DKFFE_genFeat2_GRZHSNJZYE_sum', 'DKYE_genFeat6_GRZHYE_mean', 'DKFFE_genFeat4_HDYF_sum', 'DKFFE_genFeat4_USAGE_AMOUNT_sum', 'XINGBIE_DKXE_sum', 'DKYE_genFeat4_DKYE_sum', 'DKYE_genFeat2_GJJRATE_std', 'ZHICHEN_GRJCJS_std', 'DKFFE_genFeat2_HDSYYF_std', 'DKFFE_genFeat3_GRZHDNGJYE_sum', 'DKFFE_genFeat4_HDZYF_mean', 'DKYE_genFeat1_GRJCJS_sum', 'DKYE_genFeat4_HDZYF_min', 'DKYE_genFeat5_HDSYYFZE_min', 'DKFFE_genFeat2_HDZYF_sum', 'DKYE_genFeat1_HDSYYF_std', 'DKFFE_genFeat1_GRZHSNJZYE_std', 'DKYE_genFeat1_HDSYYFZE_max', 'DKYE_genFeat6_DKYE_mean', 'GRJCJS_HDSYYF_sum', 'DKYE_genFeat2_GRZHYE_sum', 'DKYE_genFeat5_HDZYFZE_min', 'DKFFE_genFeat1_YEARPURINCM_min', 'DKYE_genFeat2_DKLL_min', 'XINGBIE_GRZHYE_std', 'DKFFE_genFeat4_TOP_USER_max', 'DKFFE_genFeat3_DKYE_std', 'DKFFE_genFeat3_HDZYFZE_mean', 'DKFFE_genFeat2_HDYF_sum', 'DKYE_genFeat6_YEARPURINCM_max', 'DKFFE_genFeat2_TOP_USER_sum', 'DKFFE_genFeat6_GRYJCE_max', 'DKYE_genFeat2_GRZHDNGJYE_std', 'DKFFE_genFeat6_YEARPURINCM_sum', 'DKFFE_genFeat1_USAGE_AMOUNT_mean', 'DKFFE_genFeat3_DKYE_sum', 'DKFFE_genFeat6_HDSYYF_max', 'ZHICHEN_in_GJJRATE_prop', 'DKFFE_genFeat5_GJJRATE_std', 'DKFFE_genFeat3_HDSYYFZE_std', 'ZHICHEN_DKYE_genFeat7_sum', 'DKYE_genFeat4_WITHDRAW_SUBMIT_mean', 'DKYE_genFeat5_GJJRATE_std', 'DKFFE_genFeat4_TOP_USER1_max', 'DKFFE_genFeat6_DKXE_std', 'ZHICHEN_HDSYYFZE_min', 'DKYE_genFeat7_GJJRATE_mean', 'DKFFE_genFeat5_DKXE_max', 'DKFFE_genFeat7_HDYF_sum', 'XINGBIE_GRJCJS_sum', 'DKFFE_genFeat7_GRYJCE_std', 'DKFFE_genFeat1_HDSYYF_std', 'DKFFE_genFeat6_GRZHSNJZYE_sum', 'DKFFE_genFeat6_DKXE_max', 'DKFFE_genFeat1_DKLL_max', 'DKYE_genFeat1_USAGE_AMOUNT_std', 'DKFFE_genFeat5_HDYF_std', 'DKYE_genFeat5_TOP_USER1_std', 'ZHICHEN_WITHDRAW_min', 'XINGBIE_HDSYYF_mean', 'DKYE_genFeat6_GRJCJS_max', 'DKFFE_genFeat6_GRJCJS_mean', 'DKFFE_genFeat2_HDZYF_max', 'DKFFE_genFeat3_DKXE_max', 'DKFFE_genFeat5_DKFFE_mean', 'DKYE_genFeat1_HDSYYF_sum', 'DKYE_genFeat5_HDZYF_std', 'DKFFE_genFeat2_DKXE_max', 'DKYE_genFeat5_TOP_USER_max', 'ZHICHEN_GRJCJS_max', 'DKFFE_genFeat3_GJJRATE_mean', 'DKYE_genFeat1_GRJCJS_std', 'DKFFE_genFeat7_WITHDRAW_min', 'DKYE_genFeat1_GRZHYE_std', 'DKYE_genFeat5_HDZYF_sum', 'DKFFE_genFeat7_GRYJCE_min', 'DKFFE_genFeat4_YEARPURINCM_max', 'DKFFE_genFeat2_GRZHDNGJYE_mean', 'ZHIYE_WITHDRAW_SUBMIT_sum', 'DKFFE_genFeat2_DKLL_std', 'DKFFE_genFeat3_HDYF_std', 'DKFFE_genFeat1_HDYF_std', 'DKYE_genFeat3_HDYF_std', 'DKYE_genFeat6_GRJCJS_mean', 'DKYE_genFeat7_WITHDRAW_SUBMIT_mean', 'DKYE_genFeat7_HDZYF_min', 'USAGE_RATE_HDYF_sum', 'DKFFE_genFeat5_GRZHDNGJYE_sum', 'DKFFE_genFeat1_WITHDRAW_sum', 'DKFFE_genFeat1_WITHDRAW_mean', 'DKYE_genFeat2_HDSYYF_max', 'DKFFE_genFeat7_USAGE_AMOUNT_sum', 'ZHICHEN_USAGE_AMOUNT_std', 'DKFFE_genFeat6_DKFFE_max', 'ZHIYE_TOP_USER1_sum', 'DKYE_genFeat1_TOP_USER_max', 'DKYE_genFeat2_GRJCJS_sum', 'DKFFE_genFeat2_USAGE_RATE_sum', 'DKFFE_genFeat4_HDSYYFZE_sum', 'DKYE_genFeat5_DKXE_max', 'DKFFE_genFeat5_GRZHDNGJYE_mean', 'DKYE_genFeat6_HDSYYFZE_std', 'DKYE_genFeat4_WITHDRAW_max', 'DKYE_genFeat2_GRYJCE_std', 'DKFFE_genFeat5_HDSYYF_mean', 'DKYE_genFeat1_DKYE_mean', 'DKFFE_genFeat5_TOP_USER_max', 'DKYE_genFeat5_HDSYYFZE_mean', 'DKFFE_genFeat5_DKFFE_std', 'DKFFE_genFeat5_WITHDRAW_min', 'DKFFE_genFeat1_GRZHDNGJYE_mean', 'DKYE_genFeat4_GJJRATE_std', 'DKFFE_genFeat6_HDYF_max', 'DKFFE_genFeat6_TOP_USER_max', 'DKYE_genFeat1_DKXE_std', 'XINGBIE_DKLL_min', 'XINGBIE_WITHDRAW_SUBMIT_max', 'ZHICHEN_GRZHYE_sum', 'DKYE_genFeat4_DKLL_std', 'DKYE_genFeat4_TOP_USER1_std', 'DKYE_genFeat6_TOP_USER_sum', 'DKFFE_genFeat3_GRZHYE_std', 'DKFFE_genFeat3_DKLL_min', 'DKYE_genFeat2_USAGE_RATE_min', 'DKFFE_genFeat4_HDZYFZE_std', 'DKYE_genFeat2_YEARPURINCM_max', 'DKYE_genFeat1_YEARPURINCM_max', 'DKFFE_genFeat3_WITHDRAW_std', 'DKFFE_genFeat7_HDYF_min', 'DKFFE_genFeat1_GRZHSNJZYE_mean', 'ZHIYE_GRZHSNJZYE_mean', 'DKFFE_genFeat7_HDSYYF_sum', 'DKYE_genFeat5_YEARPURINCM_sum', 'DKYE_genFeat2_WITHDRAW_mean', 'DKYE_genFeat4_DKXE_std', 'DKYE_genFeat6_USAGE_RATE_sum', 'XINGBIE_WITHDRAW_sum', 'XINGBIE_DKYE_genFeat2_mean', 'DKFFE_genFeat3_WITHDRAW_min', 'XINGBIE_DKYE_genFeat7_mean', 'ZHIYE_in_USAGEOVER_prop', 'DKYE_genFeat5_GRJCJS_min', 'DKYE_genFeat2_GRJCJS_max', 'DKFFE_genFeat1_USAGE_RATE_sum', 'DKYE_genFeat6_DKFFE_sum', 'DKYE_genFeat3_HDYF_max', 'DKYE_genFeat1_GRZHYE_sum', 'DKFFE_genFeat4_WITHDRAW_mean', 'DKFFE_genFeat1_DKLL_mean', 'DKYE_genFeat1_YEARPURINCM_sum', 'DKYE_genFeat5_DKFFE_mean', 'DKFFE_genFeat4_HDSYYF_std', 'ZHICHEN_GRZHYE_max', 'DKYE_genFeat2_GRYJCE_sum', 'GRJCJS_TOP_USER1_sum', 'DKFFE_genFeat6_GRZHYE_std', 'DKFFE_genFeat3_GRJCJS_std', 'DKYE_genFeat6_HDZYF_sum', 'DKFFE_genFeat2_WITHDRAW_SUBMIT_mean', 'DKFFE_genFeat5_HDZYFZE_max', 'DKFFE_genFeat1_GRZHDNGJYE_std', 'DKFFE_genFeat3_WITHDRAW_SUBMIT_min', 'DKYE_genFeat4_HDSYYF_mean', 'DKYE_genFeat3_WITHDRAW_SUBMIT_min', 'XINGBIE_HDZYF_mean', 'DKYE_genFeat4_HDSYYF_std', 'DKYE_genFeat1_HDZYF_max', 'WITHDRAW_DKXE_max', 'DKYE_genFeat2_HDSYYF_mean', 'DKYE_genFeat1_USAGE_RATE_sum', 'DKYE_genFeat1_DKFFE_std', 'DKYE_genFeat2_TOP_USER1_mean', 'DKYE_genFeat3_HDSYYF_mean', 'DKYE_genFeat5_DKXE_sum', 'ZHIYE_HDSYYFZE_sum', 'DKFFE_genFeat6_DKFFE_mean', 'ZHICHEN_in_GJSIGN_prop', 'ZHICHEN_GRZHSNJZYE_sum', 'DKFFE_genFeat5_GRZHYE_mean', 'DKYE_genFeat3_GJJRATE_mean', 'DKFFE_genFeat4_DKLL_std', 'DKYE_genFeat4_USAGE_RATE_std', 'ZHICHEN_GRZHDNGJYE_mean', 'DKFFE_genFeat3_HDYF_max', 'DKYE_genFeat7_HDSYYF_std', 'DKFFE_genFeat4_HDZYFZE_sum', 'DKFFE_genFeat1_USAGE_AMOUNT_max', 'DKFFE_genFeat6_DKLL_max', 'DKYE_genFeat3_HDSYYFZE_max', 'DKFFE_genFeat4_DKFFE_mean', 'DKYE_genFeat7_DKYE_min', 'ZHICHEN_DKFFE_std', 'DKFFE_genFeat2_DKLL_sum', 'DKFFE_genFeat6_HDZYFZE_min', 'DKFFE_genFeat6_TOP_USER_sum', 'DKYE_genFeat2_HDZYFZE_mean', 'DKYE_genFeat5_USAGE_RATE_min', 'DKYE_genFeat4_HDYF_sum', 'DKYE_genFeat7_USAGE_AMOUNT_max', 'XINGBIE_HDSYYFZE_min', 'DKYE_genFeat3_WITHDRAW_SUBMIT_max', 'DKYE_genFeat1_DKYE_min', 'DKYE_genFeat6_WITHDRAW_std', 'XINGBIE_HDSYYF_std', 'DKFFE_genFeat3_HDSYYFZE_mean', 'DKYE_genFeat4_GRZHSNJZYE_sum', 'DKYE_genFeat6_USAGE_RATE_mean', 'DKFFE_genFeat5_HDSYYFZE_sum', 'DKFFE_genFeat2_GRZHYE_mean', 'DKFFE_genFeat1_YEARPURINCM_sum', 'ZHICHEN_HDYF_std', 'DKFFE_genFeat4_HDSYYF_mean', 'DKFFE_genFeat1_DKLL_sum', 'DKYE_genFeat3_WITHDRAW_SUBMIT_sum', 'DKFFE_genFeat7_USAGE_AMOUNT_std', 'DKFFE_genFeat3_USAGE_AMOUNT_mean', 'DKYE_genFeat6_WITHDRAW_SUBMIT_max', 'ZHICHEN_DKXE_sum', 'DKYE_genFeat6_GRYJCE_std', 'DKYE_genFeat7_DKLL_mean', 'DKFFE_genFeat7_USAGE_RATE_std', 'DKYE_genFeat5_HDYF_mean', 'DKFFE_genFeat7_GRYJCE_mean', 'DKFFE_genFeat2_DKYE_mean', 'DKYE_genFeat7_GJJRATE_std', 'DKFFE_genFeat5_HDZYFZE_std', 'XINGBIE_GRZHDNGJYE_sum', 'DKFFE_genFeat3_YEARPURINCM_sum', 'DKFFE_genFeat5_WITHDRAW_SUBMIT_sum', 'ZHIYE_GRZHDNGJYE_sum', 'DKFFE_genFeat3_WITHDRAW_SUBMIT_mean', 'DKFFE_genFeat2_GRJCJS_mean', 'DKYE_genFeat3_USAGE_AMOUNT_std', 'DKYE_genFeat3_DKFFE_min', 'DKFFE_genFeat2_HDYF_max', 'XINGBIE_GJJRATE_sum', 'XINGBIE_HDSYYF_max', 'DKYE_genFeat1_TOP_USER1_std', 'DKFFE_genFeat1_GRZHSNJZYE_max', 'ZHIYE_in_WITHSIGN_prop', 'DKFFE_genFeat7_GRJCJS_mean', 'DKFFE_genFeat2_GRZHYE_max', 'ZHICHEN_in_XINGBIE_prop', 'DKYE_genFeat2_WITHDRAW_SUBMIT_min', 'DKYE_genFeat3_GRZHYE_std', 'DKFFE_genFeat7_GRZHDNGJYE_std', 'DKFFE_genFeat4_GRYJCE_mean', 'DKFFE_genFeat3_YEARPURINCM_std', 'ZHICHEN_DKFFE_genFeat4_sum', 'XINGBIE_GRZHYE_min', 'DKFFE_genFeat2_HDYF_mean', 'DKYE_genFeat6_TOP_USER_mean', 'DKYE_genFeat5_TOP_USER1_mean', 'DKFFE_genFeat7_HDSYYFZE_sum', 'ZHIYE_WITHDRAW_sum', 'DKFFE_genFeat5_WITHDRAW_mean', 'DKFFE_genFeat5_USAGE_AMOUNT_std', 'XINGBIE_TOP_USER_mean', 'DKYE_genFeat4_HDZYFZE_sum', 'ZHIYE_DKYE_genFeat5_sum', 'ZHICHEN_HDYF_max', 'DKYE_genFeat6_DKYE_min', 'DKYE_genFeat1_DKYE_std', 'DKFFE_genFeat5_DKLL_max', 'DKFFE_genFeat7_WITHDRAW_mean', 'ZHICHEN_YEARPURINCM_std', 'DKFFE_genFeat7_HDZYF_min', 'ZHIYE_HDZYF_max', 'DKYE_genFeat7_GRZHSNJZYE_min', 'DKYE_genFeat6_USAGE_AMOUNT_mean', 'DKFFE_genFeat7_DKLL_sum', 'DKYE_genFeat2_WITHDRAW_sum', 'DKFFE_genFeat7_HDZYFZE_min', 'ZHICHEN_in_WITHSIGN2_prop', 'DKYE_genFeat7_GRJCJS_mean', 'DKYE_genFeat2_HDSYYF_min', 'DKYE_genFeat6_GRJCJS_std', 'DKFFE_genFeat5_GRYJCE_std', 'DKFFE_genFeat3_YEARPURINCM_max', 'DKYE_genFeat7_GRZHSNJZYE_sum', 'DKYE_genFeat3_HDZYF_sum', 'ZHICHEN_in_USAGEOVER_prop', 'DKYE_genFeat1_HDYF_sum', 'DKYE_genFeat2_DKYE_mean', 'ZHIYE_USAGE_AMOUNT_sum', 'DKYE_genFeat3_YEARPURINCM_max', 'DKFFE_genFeat4_HDZYFZE_max', 'DKYE_genFeat7_DKYE_std', 'DKFFE_genFeat2_GJJRATE_sum', 'DKFFE_genFeat2_HDZYFZE_std', 'DKFFE_genFeat6_WITHDRAW_std', 'DKFFE_genFeat7_USAGE_RATE_max', 'DWJJLX_DKFFE_genFeat7_mean', 'DKFFE_genFeat5_GRZHSNJZYE_std', 'DKYE_genFeat5_DKFFE_min', 'ZHICHEN_YEARPURINCM_max', 'DKYE_genFeat1_WITHDRAW_SUBMIT_mean', 'DKFFE_genFeat1_GRJCJS_std', 'DKYE_genFeat2_HDYF_std', 'DKFFE_genFeat6_HDZYFZE_mean', 'DKFFE_genFeat1_GRZHYE_max', 'DKYE_genFeat5_GRJCJS_std', 'DKFFE_genFeat6_GRZHDNGJYE_std', 'DKFFE_genFeat4_GRYJCE_max', 'DKYE_genFeat3_YEARPURINCM_mean', 'DKYE_genFeat6_DKXE_mean', 'DKYE_genFeat7_GRZHDNGJYE_max', 'XINGBIE_DKYE_genFeat3_mean', 'DKFFE_genFeat5_USAGE_RATE_sum', 'DKFFE_genFeat5_DKLL_std', 'XINGBIE_DKFFE_genFeat3_sum', 'DKFFE_genFeat3_DKLL_mean', 'XINGBIE_YEARPURINCM_sum', 'ZHICHEN_DKFFE_mean', 'DKYE_genFeat6_DKXE_std', 'XINGBIE_HDYF_mean', 'DKFFE_genFeat2_HDZYF_std', 'DKFFE_genFeat1_HDSYYF_sum', 'ZHICHEN_HDZYFZE_std', 'DKFFE_genFeat5_DKYE_sum', 'DKYE_genFeat6_YEARPURINCM_std', 'DKFFE_genFeat5_DKLL_sum', 'XINGBIE_USAGE_AMOUNT_std', 'XINGBIE_DKFFE_genFeat1_sum', 'DKYE_genFeat3_TOP_USER1_sum', 'DKFFE_genFeat2_HDSYYF_mean', 'DKFFE_genFeat6_WITHDRAW_SUBMIT_min', 'ZHICHEN_DKFFE_min', 'ZHIYE_GRZHSNJZYE_sum', 'ZHICHEN_USAGE_AMOUNT_mean', 'DKYE_genFeat1_GRZHSNJZYE_sum', 'XINGBIE_DKFFE_genFeat5_mean', 'DKYE_genFeat2_DKLL_sum', 'GRJCJS_HDZYF_sum', 'DKYE_genFeat2_TOP_USER_max', 'DKFFE_genFeat2_HDSYYF_max', 'ZHIYE_HDYF_max', 'DKFFE_genFeat6_DKYE_max', 'DKYE_genFeat2_GRJCJS_min', 'DKYE_genFeat7_HDZYFZE_min', 'DKYE_genFeat4_HDYF_std', 'DKYE_genFeat6_HDZYFZE_min', 'GRZHZT_in_ZHIYE_prop', 'XINGBIE_1', 'DKFFE_genFeat6_DKXE_sum', 'DKFFE_genFeat3_DKFFE_max', 'DKFFE_genFeat6_HDSYYFZE_std', 'WITHDRAW_SUBMIT_DKXE_sum', 'XINGBIE_GRZHDNGJYE_max', 'DKFFE_genFeat5_DKFFE_sum', 'DKFFE_genFeat6_WITHDRAW_SUBMIT_sum', 'DKFFE_genFeat6_USAGE_AMOUNT_std', 'XINGBIE_DKYE_genFeat7_sum', 'DKFFE_genFeat3_YEARPURINCM_mean', 'DKFFE_genFeat2_WITHDRAW_SUBMIT_sum', 'DKYE_genFeat4_GJJRATE_mean', 'ZHICHEN_USAGE_RATE_std', 'DKFFE_genFeat1_HDZYFZE_max', 'DKYE_genFeat6_HDYF_std', 'DKFFE_genFeat6_HDZYFZE_max', 'DKFFE_genFeat4_GRZHYE_mean', 'DKYE_genFeat7_DKYE_max', 'DKFFE_genFeat5_HDYF_max', 'DKYE_genFeat7_GRZHDNGJYE_sum', 'ZHICHEN_HDSYYF_mean', 'DKFFE_genFeat3_GRZHSNJZYE_sum', 'DKFFE_genFeat2_WITHDRAW_max', 'DKFFE_genFeat7_DKLL_max', 'DKFFE_genFeat5_DKFFE_min', 'DKYE_genFeat2_HDYF_sum', 'DKFFE_genFeat3_HDSYYFZE_max', 'DKFFE_genFeat3_DKLL_sum', 'DKYE_genFeat6_HDYF_max', 'DKFFE_genFeat2_GRYJCE_std', 'DKYE_genFeat1_GRYJCE_sum', 'DKFFE_genFeat7_DKXE_mean', 'DKFFE_genFeat1_USAGE_AMOUNT_sum', 'DKFFE_genFeat7_HDZYF_std', 'XINGBIE_DKLL_sum', 'XINGBIE_YEARPURINCM_max', 'DKFFE_genFeat1_USAGE_RATE_std', 'ZHICHEN_HDSYYF_max', 'DKYE_genFeat4_DKXE_sum', 'DKFFE_genFeat4_WITHDRAW_min', 'DKFFE_genFeat6_HDZYF_max', 'DKFFE_genFeat7_GRZHYE_mean', 'DKYE_genFeat4_USAGE_RATE_min', 'DKYE_genFeat2_USAGE_AMOUNT_mean', 'DKYE_genFeat2_USAGE_AMOUNT_max', 'DKFFE_genFeat7_DKFFE_min', 'DKYE_genFeat5_DKLL_mean', 'DKYE_genFeat7_USAGE_RATE_sum', 'XINGBIE_GRZHSNJZYE_max', 'DKFFE_genFeat1_GRJCJS_mean', 'DKYE_genFeat5_HDSYYF_mean', 'DKYE_genFeat4_DKFFE_mean', 'DKYE_genFeat4_HDZYF_sum', 'ZHIYE_DKLL_sum', 'ZHICHEN_GJJRATE_max', 'XINGBIE_DKYE_min', 'ZHIYE_HDZYFZE_sum', 'DKYE_genFeat7_HDSYYF_mean', 'DKFFE_genFeat3_HDYF_sum', 'DKYE_genFeat6_GRZHDNGJYE_sum', 'XINGBIE_TOP_USER_sum', 'DKYE_genFeat5_WITHDRAW_SUBMIT_max', 'DKYE_genFeat3_GRZHSNJZYE_std', 'DKYE_genFeat6_GRZHYE_sum', 'DKYE_genFeat2_HDZYFZE_sum', 'DKYE_genFeat7_GRYJCE_sum', 'DKYE_genFeat4_DKLL_max', 'XINGBIE_WITHDRAW_SUBMIT_min', 'ZHICHEN_DKFFE_genFeat3_sum', 'DKFFE_genFeat1_TOP_USER_mean', 'XINGBIE_2', 'DKFFE_genFeat3_HDSYYF_sum', 'DKYE_genFeat5_USAGE_AMOUNT_mean', 'ZHICHEN_HDZYFZE_sum', 'DKYE_genFeat2_HDSYYFZE_std', 'ZHICHEN_HDSYYFZE_std', 'DKFFE_genFeat4_GRZHDNGJYE_sum', 'DKFFE_genFeat5_GRZHYE_sum', 'ZHICHEN_DKFFE_genFeat1_mean', 'DKYE_genFeat3_GRYJCE_mean', 'DKFFE_genFeat6_WITHDRAW_min', 'DKFFE_genFeat2_USAGE_AMOUNT_sum', 'DKYE_genFeat6_HDZYF_mean', 'ZHICHEN_WITHDRAW_SUBMIT_mean', 'XINGBIE_GRJCJS_max', 'DKFFE_genFeat5_TOP_USER_sum', 'ZHIYE_GRZHYE_sum', 'DKFFE_genFeat4_HDSYYFZE_std', 'DKYE_genFeat4_DKFFE_min', 'DKFFE_genFeat5_GRJCJS_std', 'DKFFE_genFeat3_HDSYYFZE_sum', 'DKYE_genFeat5_DKXE_std', 'DKYE_genFeat6_USAGE_AMOUNT_max', 'DKYE_genFeat2_HDYF_max', 'ZHICHEN_DKLL_mean', 'DKYE_genFeat5_TOP_USER_sum', 'DKFFE_genFeat6_HDZYF_mean', 'DKFFE_genFeat7_GRJCJS_min', 'DKFFE_genFeat3_TOP_USER_std', 'DKFFE_genFeat4_GRJCJS_mean', 'DKFFE_genFeat5_TOP_USER1_mean', 'ZHIYE_count', 'ZHICHEN_GRYJCE_sum', 'DKYE_genFeat3_DKXE_std', 'DKYE_genFeat4_GJJRATE_sum', 'DKFFE_genFeat2_DKXE_std', 'DKFFE_genFeat2_GRJCJS_sum', 'DKYE_genFeat1_HDZYFZE_sum', 'DKFFE_genFeat3_GRYJCE_sum', 'DKYE_genFeat3_GJJRATE_sum', 'DKFFE_genFeat4_DKLL_mean', 'DKYE_genFeat1_HDYF_max', 'ZHICHEN_TOP_USER_mean', 'DKYE_genFeat4_GRZHSNJZYE_std', 'DKYE_genFeat5_WITHDRAW_mean', 'DKYE_genFeat2_GRZHYE_mean', 'XINGBIE_HDYF_sum', 'DKYE_genFeat4_USAGE_AMOUNT_mean', 'DKFFE_genFeat7_DKXE_max', 'DKFFE_genFeat2_HDZYFZE_min', 'DKYE_genFeat3_DKLL_std', 'ZHICHEN_DKYE_min', 'WITHSIGN2_in_XINGBIE_prop', 'DKYE_genFeat7_USAGE_AMOUNT_min', 'DKFFE_genFeat5_WITHDRAW_std', 'DKFFE_genFeat1_HDYF_max', 'DKFFE_genFeat4_GRZHDNGJYE_mean', 'DKYE_genFeat5_GJJRATE_mean', 'DKFFE_genFeat1_WITHDRAW_min', 'DKYE_genFeat6_YEARPURINCM_min', 'DKFFE_genFeat3_DKYE_mean', 'DKYE_genFeat6_DKYE_sum', 'DKYE_genFeat4_DKYE_max', 'DKFFE_genFeat3_HDSYYF_std', 'DKYE_genFeat7_WITHDRAW_std', 'DKYE_genFeat7_USAGE_RATE_mean', 'DWJJLX_ZHIYE_count', 'DKYE_genFeat4_DKYE_std', 'DKFFE_genFeat4_DKXE_sum', 'DKYE_genFeat1_GRYJCE_std', 'DKYE_genFeat7_DKFFE_sum', 'DKFFE_genFeat7_HDZYFZE_std', 'DKFFE_genFeat5_DKYE_max', 'WITHDRAW_SUBMIT_GRYJCE_sum', 'ZHICHEN_DKFFE_genFeat1_sum', 'DKYE_genFeat1_GRZHDNGJYE_mean', 'DKFFE_genFeat1_GRZHSNJZYE_sum', 'DKFFE_genFeat4_WITHDRAW_sum', 'DKFFE_genFeat5_HDZYF_sum', 'DKYE_genFeat4_DKFFE_sum', 'DKYE_genFeat7_HDZYFZE_std', 'DKYE_genFeat6_WITHDRAW_SUBMIT_sum', 'XINGBIE_DKFFE_genFeat2_mean', 'DKYE_genFeat4_GRYJCE_max', 'DKFFE_genFeat4_WITHDRAW_max', 'numsOf_GRZHDNGJYE_HDYF_div', 'DKYE_genFeat3_YEARPURINCM_std', 'DKYE_genFeat5_TOP_USER_mean', 'DKYE_genFeat3_USAGE_AMOUNT_mean', 'DKYE_genFeat6_TOP_USER_max', 'XINGBIE_DKFFE_max', 'ZHICHEN_WITHDRAW_SUBMIT_max', 'DKYE_genFeat4_GRJCJS_min', 'DKFFE_genFeat4_YEARPURINCM_std', 'ZHIYE_DKFFE_genFeat2_sum', 'DKFFE_genFeat2_DKLL_min', 'DKFFE_genFeat4_USAGE_AMOUNT_max', 'DKFFE_genFeat4_USAGE_RATE_sum', 'DKYE_genFeat4_HDSYYFZE_sum', 'DKYE_genFeat7_HDYF_std', 'DKFFE_genFeat1_YEARPURINCM_std', 'DKYE_genFeat2_WITHDRAW_std', 'DKYE_genFeat7_TOP_USER1_mean', 'DKFFE_genFeat3_GRYJCE_min', 'DKFFE_genFeat2_GRYJCE_max', 'DKYE_genFeat2_YEARPURINCM_sum', 'DKFFE_genFeat4_HDYF_max', 'DKYE_genFeat4_WITHDRAW_SUBMIT_min', 'DKYE_genFeat2_DKFFE_mean', 'DKFFE_genFeat2_GRZHDNGJYE_sum', 'DKYE_genFeat3_USAGE_RATE_sum', 'DKFFE_genFeat4_GRJCJS_std', 'DKFFE_genFeat7_DKXE_std', 'ZHICHEN_HDZYFZE_mean', 'DKYE_genFeat5_GRZHSNJZYE_mean', 'DKFFE_genFeat5_GRYJCE_mean', 'DKYE_genFeat7_HDSYYF_sum', 'DKYE_genFeat1_WITHDRAW_std', 'DKYE_genFeat5_DKFFE_std', 'DKYE_genFeat5_GRYJCE_mean', 'XINGBIE_HDYF_min', 'DKYE_genFeat2_WITHDRAW_SUBMIT_max', 'DKFFE_genFeat3_DKXE_mean', 'DKFFE_genFeat4_DKFFE_min', 'DKFFE_genFeat7_GJJRATE_sum', 'DKYE_genFeat5_DKYE_max', 'DKFFE_genFeat1_GRYJCE_max', 'DKFFE_genFeat5_HDSYYF_std', 'DKFFE_genFeat1_GRJCJS_min', 'DKFFE_genFeat6_GRJCJS_sum', 'DKFFE_genFeat2_WITHDRAW_SUBMIT_max', 'DKFFE_genFeat1_TOP_USER1_std', 'ZHIYE_0', 'DKFFE_genFeat1_WITHDRAW_SUBMIT_sum', 'DKFFE_genFeat6_USAGE_RATE_std', 'DKYE_genFeat1_DKLL_mean', 'DKYE_genFeat1_USAGE_RATE_std', 'DKYE_genFeat2_USAGE_AMOUNT_std', 'DKYE_genFeat2_HDZYF_mean', 'DKFFE_genFeat7_WITHDRAW_SUBMIT_min', 'DKFFE_genFeat3_WITHDRAW_max', 'DKFFE_genFeat1_GJJRATE_max', 'XINGBIE_HDSYYF_sum', 'DKFFE_genFeat2_DKFFE_min', 'DKFFE_genFeat6_GRJCJS_min', 'DKFFE_genFeat3_GJJRATE_sum', 'ZHIYE_USAGE_RATE_sum', 'DKYE_genFeat3_TOP_USER_std', 'DKYE_genFeat1_GRJCJS_mean', 'DKFFE_genFeat6_YEARPURINCM_mean', 'DKFFE_genFeat1_HDSYYFZE_mean', 'ZHICHEN_HDYF_mean', 'DKFFE_genFeat2_DKYE_sum', 'DKFFE_genFeat6_HDYF_mean', 'DKYE_genFeat2_DKYE_max', 'DKFFE_genFeat5_HDSYYFZE_max', 'ZHIYE_in_XINGBIE_prop', 'DKYE_genFeat1_DKLL_std', 'DKFFE_genFeat3_GRZHSNJZYE_mean', 'XINGBIE_HDSYYFZE_sum', 'DKFFE_genFeat6_DKYE_std', 'DKYE_genFeat5_HDYF_max', 'DKYE_genFeat5_WITHDRAW_SUBMIT_sum', 'DKFFE_genFeat2_GRYJCE_sum', 'DKYE_genFeat2_DKFFE_std', 'DKYE_genFeat7_USAGE_AMOUNT_std', 'DKFFE_genFeat5_GRYJCE_sum', 'DKFFE_genFeat7_GRZHYE_std', 'ZHICHEN_DKFFE_genFeat3_mean', 'DKFFE_genFeat5_WITHDRAW_SUBMIT_mean', 'ZHICHEN_HDSYYF_min', 
                'DKYE_genFeat3_DKXE_sum']+['--DANYIZHI_QUEREN',
                 'missing_rate',
  'DWJJLX_TOP_USER1_min',
  'DWSSHY_DKYE_min',
  'DWSSHY_HDSYYF_min',
  'DWSSHY_HDSYYFZE_min',
  'DWSSHY_USAGE_RATE_min',
  'DWSSHY_TOP_USER_max',
  'DWSSHY_TOP_USER_min',
  'DWSSHY_TOP_USER1_min',
  'GRZHZT_TOP_USER_min',
  'GRZHZT_TOP_USER1_min',
  'USAGEOVER_GRZHYE_min',
  'USAGEOVER_GRYJCE_max',
  'USAGEOVER_DKFFE_max',
  'USAGEOVER_DKLL_min',
  'USAGEOVER_HDZYFZE_max',
  'USAGEOVER_TOP_USER_max',
  'USAGEOVER_TOP_USER_min',
  'USAGEOVER_TOP_USER1_min',
  'NOUSAGE_GRZHSNJZYE_min',
  'NOUSAGE_GRYJCE_max',
  'NOUSAGE_GJJRATE_min',
  'NOUSAGE_TOP_USER_max',
  'NOUSAGE_TOP_USER_min',
  'NOUSAGE_TOP_USER1_min',
  'TOP_USER_GRZHYE_min',
  'TOP_USER_GRYJCE_max',
  'TOP_USER_DKYE_min',
  'TOP_USER_DKLL_min',
  'TOP_USER_HDSYYF_min',
  'TOP_USER_HDSYYFZE_min',
  'TOP_USER_USAGE_RATE_min',
  'TOP_USER_TOP_USER_std',
  'WITHSIGN_GRZHYE_min',
  'WITHSIGN_GRZHSNJZYE_min',
  'WITHSIGN_GRYJCE_max',
  'WITHSIGN_DKFFE_max',
  'WITHSIGN_DKYE_min',
  'WITHSIGN_DKLL_max',
  'WITHSIGN_DKLL_min',
  'WITHSIGN_HDSYYF_min',
  'WITHSIGN_HDYF_min',
  'WITHSIGN_HDZYFZE_max',
  'WITHSIGN_HDSYYFZE_min',
  'WITHSIGN_USAGE_RATE_max',
  'WITHSIGN_USAGE_RATE_min',
  'WITHSIGN_TOP_USER_max',
  'WITHSIGN_TOP_USER_min',
  'WITHSIGN_TOP_USER1_min',
  'WITHSIGN_USAGE_AMOUNT_max',
  'WITHSIGN_USAGE_AMOUNT_min',
  'DKFFE_TOP_USER_std',
  'DKFFE_TOP_USER1_std',
  'DKLL_TOP_USER_min',
  'DKLL_TOP_USER1_min',
  'HDZYFZE_DKFFE_std',
  'HDZYFZE_TOP_USER_std',
  'HDZYFZE_TOP_USER1_std',
  'HDSYYFZE_DKYE_std']+['--BEITICHU_SURE','DKFFE_genFeat7_GRYJCE_max', 'XINGBIE_DKXE_std', 'XINGBIE_GJJRATE_min', 'ZHIYE_GRJCJS_std', 'XINGBIE_HDZYFZE_min', 'XINGBIE_HDZYFZE_std', 'XINGBIE_HDZYFZE_mean', 'XINGBIE_HDYF_std', 'ZHIYE_DKFFE_max', 'ZHIYE_DKFFE_min', 'XINGBIE_DKYE_std', 'ZHIYE_HDZYF_mean', 'ZHIYE_DKXE_sum', 'ZHIYE_2', 'ZHIYE_1', 'XINGBIE_GRZHSNJZYE_mean', 'NOUSAGE_in_XINGBIE_prop', 'ZHICHEN_DKXE_max', 'WITHSIGN2_in_ZHIYE_prop', 'WITHSIGN2_ZHIYE_count', 'USAGEOVER_ZHIYE_count', 
                                                  'XINGBIE_HDSYYFZE_mean']+ ['--DANYIZHI_QUEREN','missing_rate', 'GJJRATE_GRZHSNJZYE_min',
  'GJJRATE_GJJRATE_std',
  'GJSIGN_GRZHYE_min',
  'GJSIGN_GRZHSNJZYE_min',
  'GJSIGN_DKFFE_max',
  'GJSIGN_DKYE_min',
  'GJSIGN_DKLL_min',
  'GJSIGN_HDSYYF_min',
  'GJSIGN_HDZYFZE_max',
  'GJSIGN_HDSYYFZE_min',
  'GJSIGN_USAGE_RATE_min',
  'GJSIGN_GJJRATE_min',
  'GJSIGN_TOP_USER_max',
  'GJSIGN_TOP_USER_min',
  'GJSIGN_TOP_USER1_min']+  ['---DANYIZHI_QUEREN','ZHIWU',
  'missing_rate',
  'XINGBIE_GRZHSNJZYE_min',
  'XINGBIE_GRYJCE_max',
  'XINGBIE_TOP_USER_min',
  'XINGBIE_TOP_USER1_min',
  'ZHIYE_GRZHSNJZYE_min',
  'ZHIYE_HDYF_min',
  'ZHIYE_USAGE_RATE_max',
  'ZHIYE_GJJRATE_min',
  'ZHIYE_TOP_USER_min',
  'ZHIYE_TOP_USER1_min',
  'ZHIYE_USAGE_AMOUNT_min',
  'ZHICHEN_GRZHYE_min',
  'ZHICHEN_GRZHSNJZYE_min',
  'ZHICHEN_DKLL_max',
  'ZHICHEN_HDYF_min',
  'ZHICHEN_USAGE_RATE_max',
  'ZHICHEN_GJJRATE_min',
  'ZHICHEN_TOP_USER_max',
  'ZHICHEN_TOP_USER_min',
  'ZHICHEN_TOP_USER1_min',
  'ZHICHEN_USAGE_AMOUNT_min',
                             'ZHIWU']+['--BEITICHU_QUEREN','DKYE_genFeat2_HDYF_min',
 'DKFFE_genFeat4_TOP_USER1_min', 'DKFFE_genFeat4_GJJRATE_min', 'DKFFE_genFeat5_USAGE_RATE_min', 'DKFFE_genFeat5_GJJRATE_max', 'DKFFE_genFeat5_GJJRATE_min', 'DKFFE_genFeat6_GRZHDNGJYE_min', 'DKFFE_genFeat6_USAGE_RATE_min', 'DKFFE_genFeat6_USAGE_RATE_max', 'DKFFE_genFeat6_HDSYYFZE_min', 'DKFFE_genFeat6_HDYF_min', 'DKFFE_genFeat6_HDSYYF_min', 'DKFFE_genFeat6_DKYE_min', 'DKFFE_genFeat6_GRZHDNGJYE_max', 'ZHIYE', 'DKFFE_genFeat6_GRZHSNJZYE_min', 'DKFFE_genFeat6_GRZHSNJZYE_max', 'DKFFE_genFeat6_GRZHYE_min', 'DKFFE_genFeat6_GRZHYE_max', 'DKFFE_genFeat5_USAGE_AMOUNT_min', 'DKFFE_genFeat5_TOP_USER1_min', 'DKFFE_genFeat3_HDSYYF_min', 'DKFFE_genFeat3_GRZHDNGJYE_min', 'DKYE_genFeat4_GJJRATE_min', 'DKYE_genFeat5_USAGE_AMOUNT_min', 'DKYE_genFeat6_GRZHYE_min', 'DKYE_genFeat6_GRZHSNJZYE_max', 'DKYE_genFeat6_GRZHSNJZYE_min', 'DKYE_genFeat6_GRZHDNGJYE_max', 'DKYE_genFeat6_GRZHDNGJYE_min', 'DKYE_genFeat6_DKFFE_max', 'DKFFE_genFeat4_TOP_USER_min', 'DKFFE_genFeat4_USAGE_AMOUNT_min', 'DKFFE_genFeat3_DKYE_min', 'DKFFE_genFeat5_GRZHYE_max', 'DKFFE_genFeat3_HDYF_min', 'DKFFE_genFeat3_HDSYYFZE_min', 'DKFFE_genFeat3_USAGE_RATE_max', 'DKFFE_genFeat3_USAGE_RATE_min', 'DKFFE_genFeat3_GJJRATE_max', 'DKFFE_genFeat3_GJJRATE_min', 'DKFFE_genFeat3_TOP_USER_min', 'DKFFE_genFeat3_TOP_USER1_min', 'DKFFE_genFeat3_USAGE_AMOUNT_min', 'DKFFE_genFeat4_GRZHYE_max', 'DKFFE_genFeat4_GRZHYE_min', 'DKFFE_genFeat4_GRZHSNJZYE_max', 'DKFFE_genFeat4_GRZHSNJZYE_min', 'DKFFE_genFeat4_GRZHDNGJYE_max', 'DKFFE_genFeat4_GRZHDNGJYE_min', 'DKFFE_genFeat6_TOP_USER1_min', 'DKFFE_genFeat6_TOP_USER_min', 'DKFFE_genFeat6_GJJRATE_min', 'DKFFE_genFeat6_GJJRATE_max', 'DKFFE_genFeat5_USAGE_RATE_max', 'DKFFE_genFeat5_HDSYYFZE_min', 'DKFFE_genFeat5_HDYF_min', 'DKFFE_genFeat5_HDSYYF_min', 'DKFFE_genFeat5_DKYE_min', 'DKFFE_genFeat5_GRZHDNGJYE_min', 'DKFFE_genFeat5_GRZHDNGJYE_max', 'DKFFE_genFeat5_GRZHSNJZYE_min', 'DKFFE_genFeat5_GRZHSNJZYE_max', 'DKFFE_genFeat5_GRZHYE_min', 'DKYE_genFeat6_HDYF_min', 'DKYE_genFeat6_HDZYFZE_max', 'DKYE_genFeat6_USAGE_RATE_max', 'DKYE_genFeat6_GJJRATE_max', 'DKFFE_genFeat2_HDSYYFZE_min', 'DKFFE_genFeat2_USAGE_RATE_max', 'DKFFE_genFeat2_USAGE_RATE_min', 'DKFFE_genFeat2_GJJRATE_max', 'DKFFE_genFeat2_GJJRATE_min', 'DKFFE_genFeat2_TOP_USER_min', 'DKFFE_genFeat2_TOP_USER1_min', 'DKFFE_genFeat2_USAGE_AMOUNT_min', 'DKFFE_genFeat3_GRZHYE_max', 'DKFFE_genFeat3_GRZHYE_min', 'DKFFE_genFeat3_GRZHSNJZYE_max', 'DKFFE_genFeat2_GRZHSNJZYE_min', 'DKFFE_genFeat2_GRZHYE_min', 'DKFFE_genFeat1_DKXE_min', 'DKFFE_genFeat1_HDSYYF_min', 'DKYE_genFeat7_TOP_USER_min', 'DKYE_genFeat7_TOP_USER1_min', 'DKFFE_genFeat1_GRZHYE_min', 'DKFFE_genFeat1_GRZHSNJZYE_min', 'DKFFE_genFeat1_GRZHDNGJYE_min', 'DKFFE_genFeat1_DKYE_min', 'DKFFE_genFeat1_HDYF_min', 'DKFFE_genFeat1_USAGE_AMOUNT_min', 'DKFFE_genFeat1_HDSYYFZE_min', 'DKFFE_genFeat1_USAGE_RATE_max', 'DKFFE_genFeat1_USAGE_RATE_min', 'DKFFE_genFeat1_GJJRATE_min', 'DKFFE_genFeat1_TOP_USER_min', 'DKFFE_genFeat1_TOP_USER1_min', 'DKFFE_genFeat2_HDYF_min', 'DKFFE_genFeat2_HDSYYF_min', 'DKFFE_genFeat2_DKYE_min', 'DKYE_genFeat5_GRZHYE_min', 'DKYE_genFeat6_GJJRATE_min', 'DKYE_genFeat6_TOP_USER_min', 'DKYE_genFeat6_TOP_USER1_min', 'DKYE_genFeat6_USAGE_AMOUNT_min', 'DKYE_genFeat7_GRZHYE_min', 'DKYE_genFeat6_GRZHYE_max', 'DKYE_genFeat5_TOP_USER1_min', 'DKFFE_genFeat3_GRZHSNJZYE_min', 'DKYE_genFeat5_TOP_USER_min', 'DKYE_genFeat4_TOP_USER_min', 'DKYE_genFeat4_TOP_USER1_min', 'DKYE_genFeat4_USAGE_AMOUNT_min', 'DKYE_genFeat5_GRZHYE_max', 'DKYE_genFeat5_GRZHSNJZYE_max', 'DKFFE_genFeat2_GRZHDNGJYE_min', 'DKYE_genFeat5_GRZHSNJZYE_min', 'DKYE_genFeat5_GRZHDNGJYE_max', 'DKFFE_genFeat4_GJJRATE_max', 'DKYE_genFeat5_DKFFE_max', 'DKYE_genFeat5_HDYF_min', 'DKYE_genFeat5_HDZYFZE_max', 'DKYE_genFeat5_USAGE_RATE_max', 'DKYE_genFeat5_GJJRATE_max', 'DKYE_genFeat5_GJJRATE_min', 'DKYE_genFeat7_GRZHDNGJYE_min', 'DKYE_genFeat7_DKFFE_max', 'DKYE_genFeat7_HDZYFZE_max', 'DKYE_genFeat7_GJJRATE_min', 'DKFFE_genFeat4_DKYE_min', 'DKFFE_genFeat4_HDSYYF_min', 'DKYE_genFeat2_GRZHDNGJYE_min', 'DKYE_genFeat3_TOP_USER_min', 'DKYE_genFeat3_GJJRATE_max', 'DKYE_genFeat3_USAGE_RATE_max', 'DKYE_genFeat3_HDZYFZE_max', 'DKYE_genFeat3_DKFFE_max', 'DKYE_genFeat3_GRZHDNGJYE_min', 'DKYE_genFeat3_GRZHDNGJYE_max', 'DKYE_genFeat3_GRZHSNJZYE_min', 'DKYE_genFeat3_GRZHSNJZYE_max', 'DKYE_genFeat3_GRZHYE_min', 'DKYE_genFeat3_GRZHYE_max', 'DKYE_genFeat2_USAGE_AMOUNT_min', 'DKYE_genFeat2_TOP_USER1_min', 'DKYE_genFeat2_TOP_USER_min', 'DKYE_genFeat1_GRZHDNGJYE_min', 'ZHIYE_YEARPURINCM_mean', 'DKYE_genFeat1_GRZHSNJZYE_min', 'ZHIYE_GRZHDNGJYE_max', 'ZHIYE_GRZHYE_max', 'ZHIYE_GRZHYE_std', 'ZHIYE_GRZHYE_mean', 'ZHIYE_GRJCJS_min', 'ZHIYE_GRJCJS_max', 'ZHIYE_GRJCJS_mean', 'XINGBIE_DKXE_max', 'XINGBIE_WITHDRAW_SUBMIT_std', 'XINGBIE_WITHDRAW_SUBMIT_mean', 'XINGBIE_WITHDRAW_mean', 'XINGBIE_TOP_USER1_std', 'XINGBIE_TOP_USER1_mean', 'DKYE_genFeat3_GJJRATE_min', 'DKYE_genFeat3_TOP_USER1_min', 'DKFFE_genFeat4_HDSYYFZE_min', 'DKYE_genFeat3_USAGE_AMOUNT_min', 'DKYE_genFeat2_DKFFE_max', 'DKYE_genFeat2_GRZHDNGJYE_max', 'DKFFE_genFeat4_USAGE_RATE_min', 'DKYE_genFeat2_GRZHSNJZYE_min', 'DKYE_genFeat2_GRZHSNJZYE_max', 'DKYE_genFeat2_GRZHYE_min', 'DKYE_genFeat2_GRZHYE_max', 'DKYE_genFeat1_USAGE_AMOUNT_min', 'DKYE_genFeat1_TOP_USER1_min', 'DKYE_genFeat1_TOP_USER_min', 'DKYE_genFeat1_GJJRATE_min', 'DKYE_genFeat1_GJJRATE_max', 'DKYE_genFeat2_USAGE_RATE_max', 'DKYE_genFeat1_USAGE_RATE_max', 'DKYE_genFeat1_HDZYFZE_max', 'DKYE_genFeat1_HDYF_min', 'DKYE_genFeat1_DKFFE_max', 'DKYE_genFeat2_HDZYFZE_max', 'DKYE_genFeat3_HDYF_min', 'DKYE_genFeat2_GJJRATE_max', 'DKYE_genFeat2_GJJRATE_min', 'DKYE_genFeat4_HDYF_min', 'DKYE_genFeat4_DKFFE_max', 'DKYE_genFeat4_GRZHDNGJYE_min', 'DKYE_genFeat4_GRZHDNGJYE_max', 'DKYE_genFeat4_GRZHSNJZYE_min', 'DKYE_genFeat4_GRZHSNJZYE_max', 'DKYE_genFeat4_GRZHYE_min', 'DKYE_genFeat4_GRZHYE_max', 'XINGBIE_TOP_USER_max', 'XINGBIE_TOP_USER_std', 'XINGBIE_GJJRATE_mean', 'XINGBIE_USAGE_RATE_mean', 'DKYE_genFeat4_USAGE_RATE_max', 'ZHIYE_DKLL_min', 'ZHIYE_DKLL_mean', 'ZHIYE_DKYE_min', 'ZHIYE_DKYE_max', 'ZHIYE_DKYE_mean', 'ZHIYE_DKFFE_std', 'ZHIYE_DKFFE_mean', 'ZHIYE_GRYJCE_min', 'ZHIYE_GRYJCE_max', 'DKYE_genFeat4_HDZYFZE_max', 'DKYE_genFeat5_GRZHDNGJYE_min', 'DKYE_genFeat4_GJJRATE_max', 'DKFFE_genFeat4_HDYF_min', 'DKFFE_genFeat4_USAGE_RATE_max', 'DKFFE_genFeat7_TOP_USER1_min', 'DKFFE_genFeat7_TOP_USER_min', 'DKFFE_genFeat7_GJJRATE_min', 'DKFFE_genFeat7_USAGE_RATE_min', 'DKFFE_genFeat7_HDSYYFZE_min', 'DKFFE_genFeat7_HDSYYF_min', 'DKFFE_genFeat7_DKLL_min', 'DKFFE_genFeat7_DKYE_min', 'DKFFE_genFeat7_GRZHDNGJYE_min', 'DKFFE_genFeat7_GRZHDNGJYE_max', 'DKFFE_genFeat7_GRZHSNJZYE_max', 'DKFFE_genFeat7_GRZHYE_min', 'DKFFE_genFeat7_GRZHYE_max', 'DKFFE_genFeat6_USAGE_AMOUNT_min', 'ZHIYE_YEARPURINCM_max', 'ZHIYE_HDZYF_min', 'ZHIYE_HDSYYF_mean', 'DKYE_genFeat1_GRZHSNJZYE_max', 'XINGBIE_HDSYYFZE_std', 'XINGBIE_HDZYF_min', 'XINGBIE_YEARPURINCM_min', 'XINGBIE_GRYJCE_min', 'XINGBIE_GRZHDNGJYE_min', 'XINGBIE_GRZHDNGJYE_std', 'XINGBIE_GRZHSNJZYE_std', 'ZHIYE_in_ZHICHEN_prop', 'WITHSIGN2_in_ZHICHEN_prop', 'WITHSIGN2_ZHICHEN_count', 'USAGEOVER_ZHICHEN_count', 'ZHIYE_GRZHYE_min', 'ZHIYE_GRZHDNGJYE_min', 'ZHIYE_GRYJCE_std', 'ZHIYE_HDYF_mean', 'DKYE_genFeat1_GRZHYE_min', 'DKYE_genFeat1_GRZHYE_max', 'ZHICHEN_DKXE_min', 'ZHICHEN_GRZHDNGJYE_min', 'ZHIYE_DKXE_max', 'ZHIYE_USAGE_AMOUNT_max', 'ZHIYE_USAGE_AMOUNT_std', 'ZHIYE_USAGE_AMOUNT_mean', 'ZHIYE_WITHDRAW_max', 'ZHIYE_TOP_USER1_max', 'ZHIYE_GJJRATE_max', 'ZHIYE_HDZYFZE_min', 'ZHIYE_HDZYFZE_max', 
                                       'DKFFE_genFeat5_TOP_USER_min'] +['--BEITICHU_QUEREN','GJJRATE_HDSYYF_max', 'DWJJLX_GRJCJS_max', 'YEARPURINCM_WITHDRAW_SUBMIT_std', 'WITHSIGN2_in_WITHSIGN_prop', 'YEARPURINCM_WITHDRAW_std', 'YEARPURINCM_TOP_USER1_std', 'YEARPURINCM_TOP_USER_min', 'YEARPURINCM_TOP_USER_std', 'YEARPURINCM_GJJRATE_min', 'YEARPURINCM_HDSYYFZE_std', 'YEARPURINCM_HDZYF_std', 'YEARPURINCM_DKLL_min', 'WITHSIGN_in_GJSIGN_prop', 'YEARPURINCM_DKLL_max', 'HDSYYFZE_DKFFE_std', 'YEARPURINCM_DKYE_std', 'YEARPURINCM_GRYJCE_std', 'HDSYYFZE_HDZYFZE_std', 'HDYF_TOP_USER1_min', 'HDSYYFZE_HDZYFZE_min', 'HDYF_TOP_USER1_std', 'HDYF_TOP_USER_std', 'HDSYYF_WITHDRAW_SUBMIT_std', 'DWJJLX_GRZHYE_min', 'DWJJLX_GRJCJS_sum', 'DKYE_genFeat7_DKXE_min', 'WITHSIGN2_in_GJSIGN_prop', 'DKYE_genFeat5_DKXE_min', 'DKYE_genFeat4_DKXE_min', 'DKYE_genFeat3_DKXE_min', 'DKYE_genFeat2_DKXE_min', 'DKYE_genFeat1_DKXE_min', 'WITHSIGN2_WITHDRAW_SUBMIT_min', 'WITHSIGN2_USAGE_AMOUNT_min', 'WITHSIGN2_USAGE_AMOUNT_max', 'WITHSIGN2_TOP_USER1_min', 'WITHSIGN2_TOP_USER_min', 'WITHSIGN2_TOP_USER_max', 'WITHSIGN2_GJJRATE_min', 'WITHSIGN2_USAGE_RATE_min', 'DKLL_GRYJCE_min', 'DKLL_YEARPURINCM_std', 'DKLL_YEARPURINCM_max', 'HDSYYF_DKLL_std', 'HDSYYF_DKYE_std', 'HDSYYF_DKFFE_std', 'HDSYYF_GRYJCE_std', 'GJSIGN_in_WITHSIGN2_prop', 'HDSYYF_USAGE_AMOUNT_std', 'HDYF_GRYJCE_std', 'HDSYYF_TOP_USER1_min', 'HDYF_DKFFE_std', 'DKLL_GJJRATE_max', 'DKLL_GJJRATE_std', 'DKLL_GJJRATE_mean', 'DKLL_USAGE_RATE_min', 'DKLL_USAGE_RATE_max', 'DKLL_USAGE_RATE_std', 'NOUSAGE_GJSIGN_count', 'DKLL_USAGE_RATE_mean', 'DKLL_HDSYYFZE_min', 'DKLL_HDSYYFZE_max', 'DKLL_HDSYYFZE_std', 'DKLL_HDSYYFZE_mean', 'DKLL_HDZYFZE_min', 'DKLL_HDZYFZE_std', 'DKLL_HDZYFZE_mean', 'DKLL_HDYF_min', 'DKLL_HDYF_std', 'DKLL_HDYF_mean', 'DKLL_HDSYYF_max', 'DKLL_HDSYYF_std', 'DKLL_HDSYYF_mean', 'YEARPURINCM_GRJCJS_std', 'YEARPURINCM_GRZHSNJZYE_std', 'GJSIGN_HDYF_max', 'HDSYYF_USAGE_RATE_std', 'HDYF_DKYE_std', 'HDSYYF_TOP_USER1_std', 'HDSYYF_TOP_USER1_mean', 'HDSYYF_TOP_USER_min', 'HDSYYF_TOP_USER_max', 'HDSYYF_TOP_USER_std', 'HDSYYF_GJJRATE_max', 'HDSYYF_GJJRATE_std', 'HDYF_HDSYYF_std', 'HDSYYF_HDSYYFZE_min', 'HDYF_TOP_USER_min', 'HDYF_HDZYFZE_mean', 'HDSYYF_HDSYYFZE_std', 'HDYF_HDSYYFZE_std', 'HDYF_USAGE_RATE_std', 'HDSYYF_HDZYFZE_std', 'HDSYYF_HDYF_std', 'HDSYYF_HDZYF_std', 'HDYF_GJJRATE_min', 'HDYF_TOP_USER_mean', 'DKYE_genFeat6_DKXE_min', 'DKFFE_genFeat1_DKXE_max', 'GJJRATE_HDZYF_max', 'DWJJLX_WITHDRAW_SUBMIT_sum', 'WITHSIGN_in_NOUSAGE_prop', 'DWJJLX_HDYF_min', 'DWJJLX_HDYF_sum', 'DKLL_GRYJCE_std', 'DKLL_GRYJCE_mean', 'DKLL_GRZHDNGJYE_min', 'TOP_USER1_DKFFE_std', 'DKLL_GRZHDNGJYE_max', 'DKLL_GRZHDNGJYE_sum', 'DKLL_GRZHSNJZYE_min', 'DKLL_GRZHSNJZYE_std', 'DKLL_GRZHYE_max', 'DKLL_GRZHYE_mean', 'DKLL_GRJCJS_min', 'DKYE_TOP_USER_std', 'DKYE_USAGE_RATE_std', 'DWJJLX_GRZHSNJZYE_max', 'DKYE_DKFFE_std', 'DWJJLX_GRZHSNJZYE_min', 'DKFFE_TOP_USER_sum', 'TOP_USER1_TOP_USER_std', 'USAGE_AMOUNT_DKLL_max', 'DWJJLX_USAGE_AMOUNT_min', 'DKFFE_genFeat2_DKXE_min', 'DWJJLX_USAGE_AMOUNT_max', 'DWJJLX_HDZYFZE_max', 'NOUSAGE_WITHSIGN_count', 'DWJJLX_HDSYYFZE_sum', 'DWJJLX_HDSYYFZE_std', 'DWJJLX_HDSYYFZE_min', 'DWJJLX_USAGE_RATE_max', 'DWJJLX_USAGE_RATE_min', 'DWJJLX_GJJRATE_sum', 'DWJJLX_GJJRATE_mean', 'TOP_USER_in_NOUSAGE_prop', 'DWJJLX_GJJRATE_min', 'NOUSAGE_in_WITHSIGN_prop', 'DWJJLX_TOP_USER_sum', 'USAGE_AMOUNT_DKLL_min', 'DWJJLX_TOP_USER_max', 'DWJJLX_TOP_USER_min', 'DWJJLX_TOP_USER1_sum', 'DWJJLX_WITHDRAW_sum', 'DWJJLX_WITHDRAW_max', 'DWJJLX_USAGE_AMOUNT_mean', 'DWJJLX_USAGE_AMOUNT_std', 'DKFFE_HDSYYFZE_max', 'DWJJLX_GRZHDNGJYE_min', 'DWJJLX_GRYJCE_sum', 'DWJJLX_DKFFE_max', 'XINGBIE_ZHIYE_count', 'GJSIGN_TOP_USER1_mean', 'GJSIGN_TOP_USER1_sum', 'GJSIGN_TOP_USER_sum', 'GJSIGN_GJJRATE_mean', 'GJSIGN_HDSYYFZE_max', 'GJSIGN_HDSYYFZE_sum', 'GJSIGN_HDZYFZE_mean', 'WITHSIGN2_HDSYYFZE_min', 'DKLL_GRYJCE_max', 'WITHSIGN2_USAGE_RATE_max', 'WITHSIGN2_GJJRATE_max', 'DWSSHY_DKLL_sum', 'DWSSHY_DKLL_max', 'DWSSHY_DKLL_min', 'DWSSHY_GJJRATE_min', 'USAGEOVER_in_WITHSIGN_prop', 'ZHICHEN_DKFFE_genFeat5_mean', 'DKFFE_genFeat6_DKXE_min', 'DKFFE_genFeat5_DKXE_min', 'DKFFE_genFeat3_DKXE_min', 'USAGEOVER_NOUSAGE_count', 'GJSIGN_USAGE_AMOUNT_sum', 'GJSIGN_USAGE_AMOUNT_min', 'WITHSIGN2_HDSYYF_min', 'DWJJLX_DKYE_min', 'DWJJLX_DKLL_sum', 'DWJJLX_HDSYYF_min', 'DWJJLX_WITHDRAW_SUBMIT_std', 'DWSSHY_GRJCJS_max', 'DWSSHY_GRZHSNJZYE_min', 'DWSSHY_GRYJCE_max', 'WITHSIGN2_HDZYFZE_max', 'WITHSIGN2_HDYF_min', 'WITHSIGN2_DKLL_min', 'GJSIGN_WITHDRAW_SUBMIT_std', 'WITHSIGN2_DKYE_min', 'WITHSIGN2_DKFFE_max', 'WITHSIGN2_GRZHSNJZYE_min', 'WITHSIGN2_GRZHSNJZYE_max', 'WITHSIGN2_GRZHYE_min', 'WITHSIGN2_GRZHYE_max', 'GJSIGN_WITHDRAW_SUBMIT_min', 'GJSIGN_WITHDRAW_SUBMIT_max', 'USAGEOVER_WITHSIGN_count', 'DKLL_HDZYF_min', 'DKLL_HDZYF_std', 'DKLL_HDZYF_mean', 'GJJRATE_GRZHSNJZYE_mean', 'GJJRATE_WITHDRAW_std', 'GJJRATE_WITHDRAW_mean', 'GJJRATE_TOP_USER1_min', 'GRZHZT_YEARPURINCM_min', 'GJJRATE_TOP_USER1_std', 'GJJRATE_TOP_USER_min', 'GJJRATE_TOP_USER_max', 'GJJRATE_TOP_USER_sum', 'GJJRATE_WITHDRAW_min', 'GJJRATE_GJJRATE_min', 'GJJRATE_USAGE_RATE_std', 'GRZHZT_HDSYYF_mean', 'GRZHZT_HDYF_std', 'GJJRATE_HDSYYFZE_std', 'numsOf_TOP_USER_TOP_USER1_div', 'GJJRATE_GRZHYE_min', 'GRZHZT_HDYF_min', 'GRZHZT_HDZYFZE_max', 'GJJRATE_HDZYFZE_mean', 'GJJRATE_USAGE_RATE_max', 'GJJRATE_HDYF_min', 'GJJRATE_WITHDRAW_max', 'USAGEOVER_GRZHDNGJYE_min', 'DKLL_YEARPURINCM_min', 'GJJRATE_GRJCJS_max', 'TOP_USER_HDSYYF_std', 'WITHSIGN_HDSYYF_max', 'numsOf_DKYE_HDSYYFZE_multi', 'WITHSIGN_HDSYYF_mean', 'DWSSHY_12', 'WITHSIGN_HDZYF_min', 'WITHSIGN_HDZYF_max', 'DWJJLX_5', 'WITHSIGN_YEARPURINCM_min', 'WITHSIGN_YEARPURINCM_std', 'WITHSIGN_GRZHSNJZYE_max', 'WITHSIGN_GRZHSNJZYE_mean', 'WITHSIGN_GRZHYE_max', 'WITHSIGN_GRJCJS_max', 'GJSIGN_YEARPURINCM_max', 'TOP_USER_HDZYF_min', 'TOP_USER_HDSYYF_sum', 'TOP_USER_HDSYYF_mean', 'TOP_USER_WITHDRAW_SUBMIT_mean', 'numsOf_DKLL_GJJRATE_multi', 'GRZHZT_WITHDRAW_SUBMIT_min', 'GJJRATE_USAGE_AMOUNT_sum', 'XINGBIE_in_GRZHZT_prop', 'DWSSHY_USAGE_AMOUNT_min', 'GJSIGN_YEARPURINCM_std', 'GRZHZT_USAGE_RATE_mean', 'GRZHZT_TOP_USER_max', 'GJJRATE_GRYJCE_max', 'GRZHZT_USAGE_AMOUNT_mean', 'GRZHZT_USAGE_AMOUNT_std', 'GJJRATE_GRYJCE_mean', 'GJJRATE_GRZHDNGJYE_min', 'GRZHZT_USAGE_AMOUNT_max', 'GJJRATE_GRZHDNGJYE_max', 'GRZHZT_USAGE_AMOUNT_min', 'GRZHZT_WITHDRAW_SUBMIT_std', 'numsOf_HDSYYFZE_USAGE_AMOUNT_add', 'GJJRATE_GRZHDNGJYE_std', 'GRZHZT_GJJRATE_std', 'GRZHZT_USAGE_RATE_max', 'TOP_USER_USAGE_RATE_sum', 'GJJRATE_HDSYYF_std', 'GRZHZT_USAGE_RATE_std', 'GJJRATE_HDYF_sum', 'GJJRATE_HDZYF_sum', 'GJJRATE_HDSYYF_sum', 'GRZHZT_GJJRATE_min', 'GJJRATE_YEARPURINCM_std', 'GRZHZT_TOP_USER_mean', 'GJJRATE_WITHDRAW_SUBMIT_std', 'GJSIGN_YEARPURINCM_sum', 'GJSIGN_GRYJCE_mean', 'GJSIGN_GRZHSNJZYE_std', 'WITHSIGN_in_GRZHZT_prop', 'GJSIGN_GRZHSNJZYE_mean', 'GJSIGN_GRJCJS_mean', 'GRZHZT_DKFFE_max', 'GJJRATE_USAGE_AMOUNT_mean', 'GJJRATE_WITHDRAW_SUBMIT_max', 'TOP_USER_in_GRZHZT_prop', 'age_genFeat2', 'GJJRATE_WITHDRAW_SUBMIT_mean', 'GJJRATE_WITHDRAW_SUBMIT_sum', 'GJJRATE_USAGE_AMOUNT_min', 'GJJRATE_USAGE_AMOUNT_max', 'GJJRATE_USAGE_AMOUNT_std', 'GRZHZT_NOUSAGE_count', 'age_genFeat1', 'GRZHZT_in_WITHSIGN_prop', 'GRZHZT_HDSYYFZE_max', 'TOP_USER_WITHDRAW_SUBMIT_sum', 'TOP_USER_HDSYYF_max', 'TOP_USER_WITHDRAW_mean', 'TOP_USER_GRYJCE_min', 'DKLL_TOP_USER1_max', 'DKLL_TOP_USER1_std', 'DKLL_TOP_USER1_mean', 'NOUSAGE_in_WITHSIGN2_prop', 'GJSIGN_in_NOUSAGE_prop', 'DKLL_TOP_USER_max', 'GJSIGN_YEARPURINCM_min', 'DKFFE_genFeat4_DKXE_min', 'GJSIGN_in_GRZHZT_prop', 'WITHSIGN_TOP_USER_std', 'WITHSIGN_TOP_USER_mean', 'WITHSIGN_GJJRATE_min', 'TOP_USER_GRZHSNJZYE_sum', 'WITHSIGN_GJJRATE_max', 'WITHSIGN_GJJRATE_std', 'numsOf_DKFFE_HDZYFZE_multi', 'WITHSIGN_USAGE_RATE_mean', 'WITHSIGN_TOP_USER1_std', 'DWSSHY_19', 'DWSSHY_18', 'TOP_USER_GRYJCE_mean', 'DKLL_WITHDRAW_mean', 'DKLL_WITHDRAW_std', 'TOP_USER_ZHIYE_count', 'HDSYYFZE_TOP_USER_max', 'DKLL_GJJRATE_min', 'HDSYYFZE_USAGE_RATE_std', 'DKLL_TOP_USER_std', 'NOUSAGE_in_GJSIGN_prop', 'DKLL_WITHDRAW_SUBMIT_min', 'DKLL_WITHDRAW_SUBMIT_max', 'DKLL_WITHDRAW_SUBMIT_std', 'DKLL_WITHDRAW_SUBMIT_mean', 'HDSYYFZE_TOP_USER_std', 'HDSYYFZE_TOP_USER_min', 'DKLL_WITHDRAW_max', 'HDSYYFZE_TOP_USER1_std', 'DKLL_WITHDRAW_SUBMIT_sum', 'HDSYYFZE_TOP_USER1_min', 'DKLL_USAGE_AMOUNT_min', 'DKLL_USAGE_AMOUNT_max', 'DKLL_USAGE_AMOUNT_std', 'HDSYYFZE_USAGE_AMOUNT_std', 'DKLL_USAGE_AMOUNT_mean', 'DKLL_WITHDRAW_min', 'TOP_USER_GRYJCE_std', 'TOP_USER_DKLL_sum', 'TOP_USER_HDSYYFZE_mean', 'TOP_USER_YEARPURINCM_max', 'WITHSIGN_WITHDRAW_SUBMIT_min', 'WITHSIGN_HDZYFZE_min', 'DWSSHY_13', 'WITHSIGN_HDYF_max', 'TOP_USER_USAGE_AMOUNT_max', 'TOP_USER_HDYF_sum', 'TOP_USER_HDYF_max', 'TOP_USER_HDZYFZE_sum', 'TOP_USER_USAGE_AMOUNT_std', 'TOP_USER_HDZYFZE_mean', 'TOP_USER_USAGE_AMOUNT_sum', 'TOP_USER_WITHDRAW_max', 'TOP_USER_USAGE_AMOUNT_min', 'TOP_USER_WITHDRAW_std', 'TOP_USER_WITHDRAW_sum', 'TOP_USER_GJJRATE_min', 'TOP_USER_GJJRATE_max', 'DWSSHY_6', 'TOP_USER_HDZYFZE_max', 'TOP_USER_GJJRATE_sum', 'TOP_USER_HDSYYFZE_sum', 'TOP_USER_GRZHYE_sum', 'WITHSIGN_USAGE_AMOUNT_std', 'TOP_USER_8', 'USAGEOVER_GRZHSNJZYE_max', 'DWJJLX_4', 'TOP_USER_HDZYF_sum', 'WITHSIGN_HDSYYFZE_max', 'WITHSIGN_HDSYYFZE_mean', 'WITHSIGN_TOP_USER1_max', 'WITHSIGN_USAGE_AMOUNT_mean', 'XINGBIE_GRZHYE_mean', 'XINGBIE_YEARPURINCM_mean', 'USAGEOVER_GRZHYE_max', 'TOP_USER_HDSYYFZE_max', 'GJJRATE_0', 'USAGEOVER_GJJRATE_min', 'NOUSAGE_GRZHDNGJYE_max', 'NOUSAGE_GRZHDNGJYE_min', 'WITHSIGN_WITHDRAW_std', 'NOUSAGE_GJJRATE_max', 'WITHSIGN_WITHDRAW_SUBMIT_max', 'WITHSIGN_WITHDRAW_SUBMIT_std', 'GJJRATE_1', 'WITHSIGN_WITHDRAW_SUBMIT_mean', 
                                                                        'DKLL_GRZHSNJZYE_max']+ ['--BEITICHU_QUEREN','WITHSIGN', 'TOP_USER_USAGE_AMOUNT_mean', 'NOUSAGE_WITHSIGN2_count', 'GJJRATE_6', 'TOP_USER1_HDZYFZE_min', 'GJJRATE_7', 'GJJRATE_YEARPURINCM_min', 'DKLL_YEARPURINCM_mean', 'DKLL_HDZYFZE_max', 'HDYF_DKLL_std', 'HDSYYF_WITHDRAW_std', 'HDSYYF_TOP_USER_mean', 'HDSYYF_HDSYYFZE_max', 'HDSYYFZE_DKLL_min', 'YEARPURINCM_HDZYFZE_std', 'HDSYYF_GRZHDNGJYE_std', 'YEARPURINCM_TOP_USER_max', 'TOP_USER_4', 'TOP_USER_2', 'USAGE_AMOUNT_HDSYYFZE_std', 'HDSYYFZE_DKLL_max', 'DWSSHY_WITHDRAW_SUBMIT_sum', 'DWSSHY_17', 'HDSYYFZE_TOP_USER_mean',
                                                                                                 'DKLL_DKFFE_std'] +['--QIANGXIANXINGXIANGGUAN','numsOf_USAGE_RATE_WITHDRAW_diff', 'HDYF_DKLL_mean', 'GRJCJS_DKLL_sum', 'DKYE_TOP_USER1_sum', 'numsOf_GRJCJS_TOP_USER1_add', 'numsOf_GRJCJS_DKFFE_div', 'WITHSIGN_DKXE_std', 'numsOf_HDSYYF_HDYF_div', 'WITHDRAW_USAGE_AMOUNT_sum', 'numsOf_HDSYYF_TOP_USER1_add', 'numsOf_TOP_USER1_WITHDRAW_diff', 'HDSYYF_HDZYFZE_max', 'HDYF_USAGE_RATE_max', 'numsOf_DKFFE_HDSYYF_div', 'TOP_USER1_YEARPURINCM_sum', 'USAGEOVER_in_DWJJLX_prop', 'USAGE_RATE_HDSYYF_sum', 'DKYE_USAGE_RATE_mean', 'numsOf_HDYF_HDZYFZE_diff', 'DKYE_HDZYFZE_sum', 'USAGE_RATE_HDZYFZE_sum', 'numsOf_GRZHDNGJYE_WITHDRAW_div', 'DWJJLX_in_GJSIGN_prop', 'numsOf_GRZHYE_DKLL_div', 'HDSYYFZE_HDZYFZE_sum', 'GRZHSNJZYE_TOP_USER1_sum', 'WITHDRAW_SUBMIT_GRJCJS_sum', 'GRZHDNGJYE_DKFFE_sum', 'GJJRATE_ZHIYE_count', 'USAGE_AMOUNT_HDSYYF_sum', 'USAGEOVER_in_DWSSHY_prop', 'TOP_USER1_DKXE_sum', 'HDSYYF_WITHDRAW_SUBMIT_sum', 'HDZYFZE_DKYE_mean', 'HDZYFZE_GRYJCE_sum', 'numsOf_GRYJCE_WITHDRAW_diff', 'numsOf_DKLL_HDSYYFZE_div', 'XINGBIE_DKFFE_genFeat7_mean', 'HDZYFZE_GRZHSNJZYE_sum', 'DKYE_GRZHYE_sum', 'numsOf_DKFFE_WITHDRAW_multi', 'YEARPURINCM_USAGE_RATE_mean', 'numsOf_DKYE_HDYF_multi', 'GRZHYE_WITHDRAW_sum', 'GJJRATE_DKXE_std', 'numsOf_GRZHDNGJYE_TOP_USER1_add', 'GRZHZT_GRJCJS_min', 'numsOf_GRZHYE_DKYE_multi', 'numsOf_GRZHYE_DKFFE_diff', 'NOUSAGE_in_ZHICHEN_prop', 'numsOf_GRZHDNGJYE_TOP_USER1_diff', 'DWSSHY_in_NOUSAGE_prop', 'numsOf_GRZHSNJZYE_HDSYYF_div', 'DKYE_USAGE_AMOUNT_min', 'numsOf_GRJCJS_YEARPURINCM_multi', 'NOUSAGE_DKXE_std', 'USAGEOVER_DKXE_max', 'GRZHYE_GRZHDNGJYE_sum', 'YEARPURINCM_HDZYFZE_mean', 'HDSYYF_TOP_USER1_max', 'HDSYYF_HDZYFZE_min', 'numsOf_GRJCJS_DKYE_div', 'HDZYF_DKFFE_mean', 'numsOf_GRZHSNJZYE_DKXE_add', 'numsOf_DKLL_DKXE_diff', 'DWSSHY_DKFFE_std', 'HDSYYFZE_USAGE_RATE_min', 'ZHIYE_DKYE_genFeat6_mean', 'numsOf_GRJCJS_HDZYF_add', 'WITHDRAW_DKYE_min', 'GRYJCE_DKFFE_max', 'ZHIYE_HDZYF_std', 'numsOf_GRZHDNGJYE_DKFFE_multi', 'numsOf_GRZHYE_DKYE_div', 'GRZHZT_DKXE_sum', 'DWSSHY_DKYE_genFeat1_mean', 'HDYF_DKXE_mean', 'numsOf_GRZHYE_HDSYYF_div', 'USAGE_RATE_DKYE_mean', 'YEARPURINCM_WITHDRAW_SUBMIT_mean', 'USAGE_RATE_GRZHSNJZYE_sum', 'DKFFE_GRYJCE_max', 'numsOf_GRYJCE_HDZYFZE_div', 'numsOf_USAGE_AMOUNT_DKXE_add', 'numsOf_YEARPURINCM_DKXE_diff', 'HDZYFZE_DKFFE_sum', 'numsOf_HDZYF_TOP_USER1_add', 'numsOf_GRZHSNJZYE_DKXE_diff', 'DWSSHY_in_GRZHZT_prop', 'DKXE_HDZYFZE_sum', 'YEARPURINCM_GRZHDNGJYE_max', 'DKFFE_HDZYF_max', 'GRZHSNJZYE_DKXE_mean', 'numsOf_HDZYF_USAGE_RATE_div', 'numsOf_GRZHYE_DKFFE_multi', 'numsOf_GRJCJS_USAGE_AMOUNT_diff', 'numsOf_GRZHYE_GRZHSNJZYE_diff', 'GRZHZT_GJJRATE_count', 'ZHIYE_DKYE_genFeat2_mean', 'numsOf_GRJCJS_YEARPURINCM_div', 'GRZHYE_GRYJCE_sum', 'numsOf_HDSYYF_HDZYFZE_add', 'DWSSHY_GRZHYE_sum', 'HDSYYFZE_HDYF_mean', 'numsOf_GRZHSNJZYE_DKFFE_add', 'DKXE_GRJCJS_mean', 'numsOf_DKLL_WITHDRAW_SUBMIT_diff', 'YEARPURINCM_USAGE_AMOUNT_max', 'GRJCJS_DKYE_mean', 'numsOf_DKYE_TOP_USER1_add', 'GRZHZT_DKXE_std', 'HDYF_USAGE_AMOUNT_min', 'DKYE_DKXE_min', 'numsOf_DKLL_HDYF_multi', 'numsOf_GRYJCE_WITHDRAW_SUBMIT_diff', 'numsOf_GRYJCE_HDYF_div', 'DKFFE_GRYJCE_sum', 'numsOf_GRJCJS_HDZYF_div', 'YEARPURINCM_GRJCJS_max', 'GRZHSNJZYE_USAGE_AMOUNT_sum', 'numsOf_YEARPURINCM_DKXE_add', 'ZHIYE_DKYE_genFeat4_mean', 'HDZYF_GRJCJS_sum', 'GRZHDNGJYE_DKFFE_mean', 'GRYJCE_HDZYF_sum', 'USAGE_RATE_DKFFE_sum', 'numsOf_GRJCJS_HDYF_div', 'GRZHYE_HDYF_sum', 'GRZHYE_DKYE_mean', 'WITHDRAW_SUBMIT_HDSYYF_sum', 'numsOf_HDZYFZE_WITHDRAW_diff', 'GRYJCE_DKFFE_mean', 'numsOf_GRJCJS_HDSYYFZE_div', 'GJSIGN_in_ZHICHEN_prop', 'WITHDRAW_DKFFE_sum', 'numsOf_GRZHDNGJYE_HDSYYF_add', 'HDZYFZE_HDZYF_max', 'DKYE_GRZHSNJZYE_sum', 'YEARPURINCM_WITHDRAW_mean', 'numsOf_DKYE_DKXE_add', 'DWJJLX_GRZHZT_count', 'GRZHSNJZYE_GRZHDNGJYE_sum', 'GRZHDNGJYE_DKYE_max', 'numsOf_DKYE_HDSYYFZE_div', 'GRYJCE_HDZYFZE_sum', 'WITHDRAW_GRZHDNGJYE_mean', 'numsOf_HDYF_DKXE_diff', 'numsOf_GRZHSNJZYE_HDSYYF_add', 'HDSYYFZE_GRZHYE_sum', 'numsOf_GRYJCE_TOP_USER1_diff', 'WITHDRAW_DKFFE_mean', 'GRYJCE_HDZYF_mean', 'YEARPURINCM_GRZHDNGJYE_min', 'numsOf_GRJCJS_USAGE_AMOUNT_add', 'HDSYYFZE_HDYF_max', 'numsOf_DKYE_DKXE_multi', 'GRJCJS_HDZYFZE_sum', 'HDSYYF_HDZYFZE_mean', 'DWSSHY_DKFFE_genFeat1_mean', 'YEARPURINCM_GRZHDNGJYE_mean', 'GRZHSNJZYE_HDZYF_sum', 'numsOf_DKFFE_DKXE_diff', 'GRZHYE_TOP_USER1_sum', 'DKYE_HDYF_min', 'HDYF_GRYJCE_sum', 'numsOf_YEARPURINCM_HDSYYF_div', 'GRZHSNJZYE_DKYE_min', 'numsOf_GJJRATE_WITHDRAW_SUBMIT_add', 'YEARPURINCM_GRJCJS_min', 'HDSYYF_USAGE_AMOUNT_min', 'USAGE_AMOUNT_HDZYFZE_sum', 'GRYJCE_HDSYYF_sum', 'numsOf_GRJCJS_HDZYF_diff', 'WITHDRAW_TOP_USER1_sum', 'GRZHYE_DKYE_max', 'HDYF_TOP_USER1_mean', 'HDSYYF_HDSYYFZE_mean', 'numsOf_DKLL_WITHDRAW_div', 'GRZHYE_DKYE_min', 'WITHDRAW_GRZHYE_sum', 'WITHDRAW_YEARPURINCM_sum', 'WITHDRAW_USAGE_RATE_sum', 'numsOf_GRJCJS_GRZHDNGJYE_multi', 'GRZHZT_in_GJJRATE_prop', 'numsOf_GRYJCE_HDSYYFZE_div', 'WITHDRAW_GRZHDNGJYE_sum', 'GRYJCE_TOP_USER1_sum', 'HDYF_GRZHSNJZYE_sum', 'DWJJLX_GRYJCE_mean', 'numsOf_DKLL_HDSYYF_multi', 'DWSSHY_HDSYYFZE_sum', 'TOP_USER_GRZHDNGJYE_sum', 'GRZHSNJZYE_USAGE_RATE_mean', 'HDZYF_DKLL_sum', 'numsOf_GRJCJS_GRZHYE_diff', 'numsOf_HDSYYF_WITHDRAW_div', 'DKLL_DKXE_sum', 'GRYJCE_WITHDRAW_sum', 'DKYE_HDZYF_sum', 'numsOf_DKFFE_DKXE_add', 'DKFFE_GRZHDNGJYE_min', 'numsOf_HDZYF_WITHDRAW_diff', 'NOUSAGE_DKXE_max', 'ZHIYE_HDSYYF_min', 'USAGE_AMOUNT_USAGE_RATE_sum', 'numsOf_GRZHDNGJYE_DKXE_diff', 'numsOf_DKLL_USAGE_AMOUNT_multi', 'GRZHSNJZYE_USAGE_AMOUNT_mean', 'numsOf_TOP_USER1_DKXE_add', 'GRZHDNGJYE_HDZYF_sum', 'HDSYYF_GRYJCE_sum', 'GJJRATE_DKXE_sum', 'HDSYYFZE_USAGE_AMOUNT_max', 'NOUSAGE_in_ZHIYE_prop', 'DKYE_HDYF_mean', 'numsOf_GRYJCE_WITHDRAW_add', 'HDSYYF_WITHDRAW_sum', 'HDSYYFZE_HDZYF_sum', 'numsOf_GRJCJS_DKXE_diff', 'DWJJLX_DKYE_mean', 'ZHIYE_DKFFE_genFeat2_mean', 'YEARPURINCM_GJJRATE_max', 'HDSYYFZE_USAGE_AMOUNT_min', 'numsOf_DKLL_DKXE_multi', 'ZHIYE_DKFFE_genFeat7_sum', 'WITHDRAW_SUBMIT_HDZYF_sum', 'DWJJLX_in_NOUSAGE_prop', 'TOP_USER1_WITHDRAW_SUBMIT_min', 'numsOf_DKFFE_USAGE_AMOUNT_multi', 'TOP_USER1_GRZHDNGJYE_sum', 'GRZHYE_HDZYF_sum', 'ZHIYE_DKFFE_genFeat4_mean', 'WITHSIGN2_DKXE_sum', 'numsOf_DKYE_TOP_USER_multi', 'numsOf_GRJCJS_WITHDRAW_div', 'numsOf_DKYE_DKXE_div', 'GRZHSNJZYE_HDYF_sum', 'DKYE_DKXE_sum', 'HDSYYF_YEARPURINCM_sum', 'HDYF_GRYJCE_mean', 'GRJCJS_DKYE_min', 'GRYJCE_DKLL_sum', 'GRJCJS_HDZYF_max', 'HDSYYF_GRJCJS_sum', 'GRYJCE_DKYE_min', 'HDSYYF_DKXE_sum', 'HDSYYFZE_TOP_USER1_sum', 'WITHDRAW_HDYF_sum', 'numsOf_GRJCJS_USAGE_AMOUNT_multi', 'HDZYF_DKFFE_max', 'numsOf_DKYE_TOP_USER1_multi', 'GRJCJS_USAGE_RATE_sum', 'numsOf_GRYJCE_WITHDRAW_SUBMIT_add', 'numsOf_HDYF_GJJRATE_diff', 'GRZHZT_DKLL_std', 'GRZHYE_DKFFE_sum', 'GRZHZT_GRJCJS_max', 'numsOf_DKFFE_DKXE_multi', 'numsOf_YEARPURINCM_HDYF_div', 'HDZYFZE_GJJRATE_sum', 'DKYE_USAGE_RATE_max', 'DKYE_DKXE_max', 'YEARPURINCM_HDSYYF_mean', 'DKXE_DKFFE_sum', 'YEARPURINCM_HDZYF_mean', 'DKYE_WITHDRAW_SUBMIT_mean', 'numsOf_HDZYFZE_DKXE_diff', 'WITHSIGN_DKXE_sum', 'numsOf_USAGE_RATE_TOP_USER1_add', 'GJJRATE_in_ZHIYE_prop', 'HDZYFZE_GRJCJS_mean', 'GJJRATE_in_ZHICHEN_prop', 'GJJRATE_in_XINGBIE_prop', 'YEARPURINCM_USAGE_AMOUNT_min', 'WITHDRAW_SUBMIT_GRZHSNJZYE_sum', 'WITHDRAW2', 'numsOf_DKFFE_WITHDRAW_div', 'numsOf_HDYF_TOP_USER1_diff', 'XINGBIE_TOP_USER1_max', 'GRYJCE_HDZYF_max', 'YEARPURINCM_DKXE_mean', 'numsOf_GRJCJS_GRZHSNJZYE_diff', 'numsOf_GRYJCE_WITHDRAW_div', 'GRZHDNGJYE_DKFFE_min', 'USAGE_RATE_DKYE_min', 'numsOf_GRZHDNGJYE_DKLL_div', 'USAGE_RATE_GRYJCE_sum', 'HDSYYFZE_HDYF_sum', 'numsOf_DKFFE_USAGE_RATE_div', 'numsOf_DKYE_GJJRATE_div', 'USAGE_RATE_GRJCJS_sum', 'USAGE_AMOUNT_DKYE_max', 'GRZHSNJZYE_WITHDRAW_SUBMIT_sum', 'numsOf_DKYE_GJJRATE_multi', 'numsOf_GRZHYE_HDZYF_add', 'USAGEOVER_in_GJJRATE_prop', 'numsOf_HDYF_WITHDRAW_multi', 'numsOf_GRZHDNGJYE_USAGE_AMOUNT_diff', 'numsOf_USAGE_RATE_WITHDRAW_SUBMIT_diff', 'GRZHYE_USAGE_AMOUNT_sum', 'numsOf_GRJCJS_HDSYYF_diff', 'numsOf_DKLL_USAGE_AMOUNT_div', 'numsOf_HDSYYF_WITHDRAW_add', 'numsOf_HDYF_TOP_USER_add', 'DKFFE_GRJCJS_mean', 'numsOf_GRZHYE_DKYE_add', 'XINGBIE_DKFFE_genFeat6_sum', 'numsOf_HDYF_DKXE_add', 'DKYE_WITHDRAW_sum', 'numsOf_GRZHDNGJYE_DKFFE_diff', 'numsOf_HDSYYF_WITHDRAW_diff', 'numsOf_GRYJCE_DKXE_diff', 'HDZYFZE_GRZHYE_sum', 'numsOf_DKLL_DKXE_add', 'numsOf_DKFFE_USAGE_RATE_diff', 'GJSIGN_in_ZHIYE_prop', 'HDZYF_HDSYYF_mean', 'numsOf_HDSYYF_HDZYFZE_diff', 'CSNY', 'HDSYYF_DKXE_mean', 'numsOf_GRZHSNJZYE_GRZHDNGJYE_add', 'ZHIYE_DKFFE_genFeat1_mean', 'DKXE_DKYE_mean', 'GRZHDNGJYE_DKYE_sum', 'GJJRATE_in_USAGEOVER_prop', 'numsOf_DKLL_WITHDRAW_SUBMIT_multi', 'TOP_USER_1', 'TOP_USER_GRYJCE_sum', 'DKXE_DKFFE_max', 'HDZYFZE_HDZYF_sum', 'ZHIYE_USAGE_RATE_min', 'WITHDRAW_SUBMIT_HDSYYF_max', 'numsOf_GRZHYE_HDYF_div', 'DWJJLX_in_USAGEOVER_prop', 'GRYJCE_DKYE_sum', 'DWSSHY_HDZYF_std', 'numsOf_HDZYF_WITHDRAW_add', 'numsOf_GRZHYE_DKXE_add', 'numsOf_GRZHYE_DKXE_diff', 'DKXE_GRJCJS_sum', 'DWJJLX_HDSYYF_max', 'numsOf_DKFFE_GJJRATE_multi', 'HDZYF_DKYE_min', 'numsOf_DKYE_WITHDRAW_multi', 'GRJCJS_GRZHSNJZYE_sum', 'numsOf_GRJCJS_HDYF_diff', 'numsOf_GRZHDNGJYE_DKXE_add', 'TOP_USER1_GRJCJS_mean', 'numsOf_DKLL_YEARPURINCM_multi', 'USAGE_RATE_DKFFE_min', 'YEARPURINCM_USAGE_AMOUNT_sum', 'numsOf_DKYE_WITHDRAW_SUBMIT_multi', 'WITHDRAW_GRZHDNGJYE_max', 'numsOf_GRJCJS_GRZHSNJZYE_add', 'numsOf_DKFFE_DKLL_div', 'numsOf_HDZYF_TOP_USER1_diff', 'GRZHYE_HDSYYF_sum', 'HDSYYFZE_DKLL_sum', 'numsOf_GRZHSNJZYE_DKLL_multi', 'numsOf_TOP_USER_WITHDRAW_div', 'numsOf_HDZYF_HDSYYF_div', 'DWSSHY_count', 'WITHDRAW_DKYE_sum', 'WITHDRAW_SUBMIT_HDYF_sum', 'numsOf_GRJCJS_HDSYYF_div', 'HDYF_WITHDRAW_sum', 'WITHDRAW_DKYE_max', 'numsOf_DKFFE_HDYF_add', 'GRZHYE_WITHDRAW_SUBMIT_sum', 'DKYE_HDYF_sum', 'GRJCJS_DKYE_max', 'DKYE_TOP_USER_sum', 'numsOf_DKYE_TOP_USER_div', 'DWJJLX_DKYE_genFeat7_mean', 'GRJCJS_WITHDRAW_sum', 'USAGE_AMOUNT_GRYJCE_sum', 'DKYE_TOP_USER1_mean', 'USAGEOVER_in_GJSIGN_prop', 'YEARPURINCM_HDZYF_sum', 'numsOf_HDZYF_DKXE_add', 'DKFFE_HDYF_sum', 'DKYE_GJJRATE_sum', 'WITHDRAW_DKYE_mean', 'numsOf_GRZHDNGJYE_DKFFE_add', 'TOP_USER1_WITHDRAW_SUBMIT_sum', 'numsOf_HDSYYFZE_WITHDRAW_diff', 'numsOf_HDSYYFZE_TOP_USER_div', 'numsOf_GRZHDNGJYE_DKLL_multi', 'USAGE_RATE_TOP_USER_sum', 'numsOf_WITHDRAW_USAGE_AMOUNT_add', 'numsOf_GRJCJS_DKXE_add', 'numsOf_GJJRATE_WITHDRAW_SUBMIT_diff', 'DKFFE_HDSYYF_min', 'HDYF_DKXE_sum', 'DWSSHY_DKYE_genFeat7_mean', 'numsOf_USAGE_AMOUNT_DKXE_diff', 'DKYE_HDYF_max', 'numsOf_GRZHDNGJYE_DKYE_div', 'numsOf_GRYJCE_DKFFE_multi', 'numsOf_DKFFE_DKXE_div', 'XINGBIE_in_WITHSIGN_prop', 'numsOf_DKYE_USAGE_AMOUNT_diff', 'numsOf_HDSYYFZE_WITHDRAW_add', 'GRZHDNGJYE_WITHDRAW_sum', 'numsOf_GRYJCE_DKLL_multi', 'numsOf_TOP_USER_DKXE_add', 'numsOf_DKYE_YEARPURINCM_diff', 'numsOf_GRZHSNJZYE_GRYJCE_add', 'ZHICHEN_DKFFE_genFeat6_mean', 'numsOf_GRZHYE_DKFFE_add', 'numsOf_WITHDRAW_SUBMIT_DKXE_add', 'numsOf_GRZHDNGJYE_HDSYYF_diff', 'USAGEOVER_DKXE_sum', 'numsOf_GRJCJS_DKYE_multi', 'numsOf_HDYF_HDZYFZE_add', 'numsOf_HDYF_WITHDRAW_diff', 'DKYE_DKFFE_sum', 'numsOf_DKFFE_YEARPURINCM_add', 'HDSYYFZE_GJJRATE_sum', 'ZHICHEN_DKFFE_genFeat6_sum', 'DWSSHY_DKYE_std', 'numsOf_GRYJCE_DKLL_div', 'HDZYF_DKYE_max', 'numsOf_HDZYFZE_DKXE_add', 'HDZYFZE_WITHDRAW_SUBMIT_sum', 'DKFFE_GRZHSNJZYE_min', 'WITHDRAW_SUBMIT_DKYE_sum', 'GRZHDNGJYE_WITHDRAW_mean', 'WITHDRAW_GRYJCE_sum', 'GRYJCE_DKYE_max', 'YEARPURINCM_GRYJCE_max', 'WITHDRAW_SUBMIT_USAGE_AMOUNT_sum', 'numsOf_GRZHSNJZYE_HDYF_div', 'WITHSIGN_ZHIYE_count', 'DWJJLX_DKFFE_mean', 'numsOf_WITHDRAW_USAGE_AMOUNT_diff', 'numsOf_GRYJCE_HDZYFZE_add', 'numsOf_DKLL_TOP_USER_diff', 'DKYE_DKXE_mean', 'GRZHSNJZYE_HDSYYF_sum', 'GRZHSNJZYE_GRZHYE_sum', 'DKFFE_GRZHYE_sum', 'HDZYF_DKYE_sum', 'DKXE_DKFFE_mean', 'numsOf_GRZHYE_DKLL_multi', 'WITHSIGN2_DKXE_std', 'USAGE_AMOUNT_DKLL_sum', 'numsOf_GRJCJS_USAGE_RATE_div', 'numsOf_GRZHYE_WITHDRAW_div', 'USAGE_AMOUNT_TOP_USER1_sum', 'USAGE_AMOUNT_GJJRATE_sum', 'numsOf_GRZHSNJZYE_DKYE_multi', 'WITHSIGN_ZHICHEN_count', 'ZHIYE_DKFFE_genFeat5_sum', 'numsOf_DKFFE_YEARPURINCM_diff', 'numsOf_DKFFE_HDSYYF_multi', 'numsOf_HDSYYFZE_DKXE_diff', 'HDYF_HDZYF_sum', 'DWSSHY_DKFFE_min', 'ZHIYE_DKYE_genFeat5_mean', 'DKXE_HDSYYFZE_sum', 'GJJRATE_YEARPURINCM_mean', 'DWJJLX_DKXE_std', 'WITHDRAW_HDSYYF_max', 'numsOf_HDZYF_HDSYYFZE_div', 'numsOf_HDSYYF_HDZYFZE_multi', 'YEARPURINCM_DKXE_min', 'YEARPURINCM_WITHDRAW_max', 'numsOf_DKLL_WITHDRAW_multi', 'NOUSAGE_in_TOP_USER_prop', 'numsOf_HDSYYF_USAGE_RATE_multi', 'XINGBIE_DKFFE_genFeat6_mean', 'GRYJCE_DKFFE_sum', 'GRZHYE_DKFFE_min', 'GRZHDNGJYE_DKYE_min', 'YEARPURINCM_USAGE_RATE_max', 'numsOf_GRZHSNJZYE_GRYJCE_diff', 'numsOf_GRZHSNJZYE_HDSYYFZE_div', 'ZHIYE_DKLL_std', 'DWSSHY_DKYE_genFeat5_mean', 'GRZHSNJZYE_GRJCJS_sum', 'numsOf_GRZHDNGJYE_GRYJCE_add', 'GRJCJS_DKFFE_max', 'numsOf_GRZHDNGJYE_USAGE_AMOUNT_add', 'numsOf_GRZHDNGJYE_HDSYYFZE_div', 'numsOf_DKYE_YEARPURINCM_multi', 'GRZHDNGJYE_HDZYF_max', 'ZHIYE_DKYE_genFeat3_mean', 'YEARPURINCM_WITHDRAW_min', 'DKXE_HDZYF_sum', 'GRJCJS_HDYF_sum', 'numsOf_HDSYYF_DKXE_add', 'DKYE_GRJCJS_sum', 'ZHIYE_DKFFE_genFeat3_mean', 'XINGBIE_GRZHDNGJYE_mean', 'GRZHZT_GJSIGN_count', 'XINGBIE_USAGE_AMOUNT_min', 'GRZHSNJZYE_HDZYF_max', 'TOP_USER1_DKXE_max', 'numsOf_GRZHYE_DKYE_diff', 'numsOf_DKYE_YEARPURINCM_add', 'WITHSIGN2_DKXE_max', 'DWSSHY_DKFFE_mean', 'GRZHSNJZYE_USAGE_RATE_sum', 'numsOf_USAGE_RATE_TOP_USER1_diff', 'USAGEOVER_DKXE_std', 'GRZHYE_HDZYF_max', 'USAGE_AMOUNT_HDZYF_sum', 'numsOf_USAGE_RATE_DKXE_add', 'numsOf_TOP_USER1_WITHDRAW_div', 'numsOf_GRYJCE_TOP_USER1_add', 'DKXE_DKYE_sum', 'USAGEOVER_in_TOP_USER_prop', 'DWSSHY_DKYE_genFeat6_mean', 'HDSYYFZE_DKXE_mean', 'numsOf_GRJCJS_DKFFE_diff', 'DKFFE_GJJRATE_sum', 'XINGBIE_USAGE_AMOUNT_max', 'ZHIYE_GRYJCE_mean', 'HDZYF_DKYE_mean', 'DKLL_DKXE_std', 'WITHSIGN_in_XINGBIE_prop', 'numsOf_HDSYYF_WITHDRAW_SUBMIT_add', 'DWJJLX_in_ZHIYE_prop', 'GJSIGN_ZHIYE_count', 'GRZHSNJZYE_DKYE_max', 'NOUSAGE_DKXE_sum', 'WITHDRAW_SUBMIT_DKYE_mean', 'GRZHSNJZYE_WITHDRAW_sum', 'HDSYYFZE_GRYJCE_sum', 'numsOf_WITHDRAW_DKXE_add', 'WITHDRAW_SUBMIT_YEARPURINCM_sum', 'DKXE_HDZYF_max', 'GRJCJS_GJJRATE_sum', 'HDZYF_HDSYYF_max', 'numsOf_DKFFE_HDYF_multi', 'YEARPURINCM_USAGE_RATE_min', 'WITHSIGN_in_ZHICHEN_prop', 'numsOf_TOP_USER_DKXE_diff', 'GRZHYE_USAGE_RATE_sum', 'USAGE_RATE_DKYE_sum', 'numsOf_DKLL_TOP_USER_div', 'HDSYYF_HDZYF_sum', 'numsOf_HDZYFZE_WITHDRAW_div', 'numsOf_DKFFE_HDYF_div', 'GRYJCE_DKFFE_min', 'HDSYYF_GRZHSNJZYE_sum', 'numsOf_WITHDRAW_DKXE_diff', 'DKXE_GRZHYE_sum', 'GRZHSNJZYE_GRJCJS_mean', 'GJJRATE_in_WITHSIGN_prop', 'HDZYFZE_HDSYYF_sum', 'DWSSHY_HDZYF_sum', 'NOUSAGE_DKXE_min', 'GRZHYE_DKYE_sum', 'USAGE_RATE_HDYF_max', 'DKXE_DKFFE_min', 'GRJCJS_HDZYF_mean', 'numsOf_GRZHSNJZYE_HDZYF_add', 'DWSSHY_ZHIYE_count', 'numsOf_HDYF_TOP_USER1_add', 'numsOf_GRZHDNGJYE_WITHDRAW_SUBMIT_multi', 'numsOf_GRJCJS_GJJRATE_div', 'GRZHDNGJYE_HDZYFZE_sum', 'GRZHZT_WITHSIGN_count', 'GRYJCE_GRZHSNJZYE_sum', 'numsOf_GRYJCE_HDSYYF_div', 'numsOf_DKFFE_HDSYYFZE_diff', 'XINGBIE_in_NOUSAGE_prop', 'HDSYYFZE_TOP_USER_sum', 'GRZHSNJZYE_DKYE_sum', 'GRJCJS_DKFFE_mean', 'YEARPURINCM_HDSYYFZE_mean', 'DWSSHY_DKYE_genFeat3_mean', 'NOUSAGE_DKXE_mean', 'ZHIYE_DKLL_max', 'numsOf_GRYJCE_DKXE_add', 'numsOf_HDZYFZE_HDSYYFZE_div', 'numsOf_GRZHSNJZYE_DKFFE_multi', 'numsOf_HDYF_HDSYYFZE_div', 'numsOf_HDSYYF_TOP_USER1_diff', 'WITHDRAW_GRZHSNJZYE_sum', 'DKYE_HDSYYF_sum', 'numsOf_DKFFE_WITHDRAW_SUBMIT_div', 'USAGE_RATE_GRZHYE_sum', 'numsOf_GRYJCE_USAGE_RATE_multi', 'GJJRATE_ZHICHEN_count', 'DKXE', 'ZHIYE_DKFFE_genFeat6_sum', 'USAGEOVER_DKXE_min', 'WITHSIGN_WITHSIGN2_count', 'HDSYYF_HDZYF_mean', 'numsOf_GRJCJS_GRZHYE_add', 'numsOf_DKLL_TOP_USER_add', 'numsOf_GRZHDNGJYE_GRYJCE_diff', 'HDSYYF_GRZHYE_sum', 'GRZHZT_DKXE_mean', 'numsOf_USAGE_RATE_DKXE_diff', 'USAGE_AMOUNT_TOP_USER1_mean', 'numsOf_GRZHYE_GRYJCE_add', 'HDZYFZE_HDSYYFZE_max', 'WITHDRAW_SUBMIT_DKYE_max', 'numsOf_DKYE_DKLL_div', 'USAGE_AMOUNT_HDSYYFZE_sum', 'GRZHSNJZYE_GRYJCE_mean', 'YEARPURINCM_USAGE_AMOUNT_mean', 'numsOf_GRZHSNJZYE_DKLL_div', 'XINGBIE_in_WITHSIGN2_prop', 'GJSIGN_in_XINGBIE_prop', 'numsOf_GRZHSNJZYE_DKYE_add', 'GJJRATE_YEARPURINCM_sum', 'DKXE_DKYE_min', 'numsOf_GRJCJS_TOP_USER1_diff', 'numsOf_GJJRATE_WITHDRAW_div', 'GRZHYE_YEARPURINCM_sum', 'numsOf_DKFFE_WITHDRAW_SUBMIT_multi', 'numsOf_GRJCJS_GRZHYE_multi', 'numsOf_GRZHYE_HDSYYFZE_div', 'numsOf_GJJRATE_DKXE_add', 'numsOf_GRZHSNJZYE_DKFFE_div', 'WITHSIGN_DKXE_min', 'numsOf_GRJCJS_HDYF_add', 'numsOf_DKYE_USAGE_RATE_multi', 'DKFFE_WITHDRAW_SUBMIT_sum', 'TOP_USER1_HDZYFZE_max', 'DWSSHY_HDZYF_mean', 'GJJRATE_GRJCJS_mean', 'GRZHZT_HDZYF_mean', 'GJSIGN_DKXE_std', 'ZHIYE_DKFFE_genFeat7_mean', 'DWSSHY_DKYE_genFeat4_mean', 'GRZHSNJZYE_YEARPURINCM_sum', 'numsOf_GRJCJS_DKFFE_multi', 'numsOf_GRZHSNJZYE_HDZYF_diff', 'numsOf_DKYE_DKXE_diff', 'DKYE_WITHDRAW_SUBMIT_sum', 'DKXE_DKYE_max', 'YEARPURINCM_GRYJCE_min', 'HDZYF_DKFFE_min', 'numsOf_YEARPURINCM_HDSYYFZE_div', 'DWJJLX_ZHICHEN_count', 'DKFFE_DKXE_min', 'numsOf_HDZYFZE_WITHDRAW_add', 'numsOf_GRZHYE_WITHDRAW_add', 'numsOf_DKFFE_DKYE_add', 'USAGEOVER_DKXE_mean', 'numsOf_GRZHDNGJYE_DKYE_add', 'XINGBIE_in_GJSIGN_prop', 'numsOf_DKYE_TOP_USER1_diff', 'USAGE_AMOUNT_GRZHSNJZYE_sum', 'numsOf_GRJCJS_GRYJCE_diff', 'numsOf_TOP_USER1_WITHDRAW_add', 'HDYF_USAGE_AMOUNT_max', 'HDZYFZE_WITHDRAW_sum', 'DKYE_GRYJCE_sum', 'ZHIYE_DKYE_genFeat1_mean', 'DKFFE_GRZHYE_min', 'HDSYYF_HDYF_max', 'numsOf_GRZHDNGJYE_DKFFE_div', 'DWJJLX_DKFFE_genFeat5_mean', 'numsOf_TOP_USER1_DKXE_diff', 'XINGBIE_DKFFE_genFeat7_sum', 'GRYJCE_HDSYYFZE_sum', 'HDYF_USAGE_RATE_min', 'USAGE_RATE_DKYE_max', 'numsOf_HDSYYF_DKXE_diff', 'numsOf_TOP_USER_TOP_USER1_diff', 'numsOf_DKLL_TOP_USER1_multi', 'USAGE_AMOUNT_DKYE_min', 'numsOf_GRJCJS_DKLL_div', 'USAGE_AMOUNT_TOP_USER_sum', 'numsOf_HDYF_WITHDRAW_add', 'numsOf_GRZHSNJZYE_DKFFE_diff', 'numsOf_HDSYYFZE_DKXE_add', 'YEARPURINCM_TOP_USER1_mean', 'HDZYF_DKFFE_sum', 'HDSYYFZE_DKYE_sum', 'ZHIYE_HDSYYFZE_min', 'numsOf_HDZYF_HDSYYF_add', 'USAGE_AMOUNT_GRZHYE_sum', 'numsOf_GRYJCE_DKYE_multi', 'GRJCJS_DKFFE_min', 'numsOf_GRZHDNGJYE_DKYE_diff', 'numsOf_DKLL_TOP_USER_multi', 'GRZHDNGJYE_DKYE_mean', 'numsOf_DKFFE_USAGE_AMOUNT_add', 'numsOf_HDZYF_WITHDRAW_SUBMIT_multi', 'numsOf_GRZHYE_DKFFE_div', 'ZHICHEN_DKFFE_genFeat7_sum', 'GRZHSNJZYE_GRYJCE_sum', 'HDSYYF_GRYJCE_min', 'WITHDRAW_HDZYF_sum', 'USAGE_AMOUNT_USAGE_RATE_mean', 'numsOf_HDZYF_WITHDRAW_div', 'NOUSAGE_ZHIYE_count', 'WITHDRAW_SUBMIT_WITHDRAW_max', 'GRYJCE_DKYE_mean', 'GJJRATE_in_NOUSAGE_prop', 'USAGE_AMOUNT_GRJCJS_sum', 'XINGBIE_DKYE_max', 'numsOf_GRZHDNGJYE_DKYE_multi', 'ZHIYE_DKFFE_genFeat5_mean', 'numsOf_DKFFE_HDSYYF_diff', 'GRZHZT_in_TOP_USER_prop', 'USAGE_RATE_DKXE_mean', 'numsOf_GRJCJS_DKLL_multi', 'DWSSHY_HDZYF_max', 'WITHDRAW_GRZHDNGJYE_min', 'numsOf_DKFFE_WITHDRAW_diff', 'GRZHDNGJYE_GRJCJS_sum', 'numsOf_DKLL_HDZYF_multi', 'numsOf_DKLL_HDSYYFZE_multi', 'WITHSIGN2_in_GRZHZT_prop', 'USAGE_AMOUNT_DKYE_sum', 'numsOf_HDZYF_USAGE_AMOUNT_div', 'WITHSIGN2_in_USAGEOVER_prop', 'HDZYFZE_DKLL_mean', 'numsOf_GRJCJS_GRZHSNJZYE_multi', 'numsOf_DKFFE_HDYF_diff', 'numsOf_HDZYF_DKXE_diff', 'numsOf_USAGE_RATE_WITHDRAW_SUBMIT_add', 'GRYJCE_GRJCJS_mean', 'ZHIYE_DKYE_std', 'numsOf_GRZHSNJZYE_DKYE_diff', 'TOP_USER_ZHICHEN_count', 'YEARPURINCM_DKXE_max', 'GJJRATE_in_GJSIGN_prop', 'HDYF_USAGE_AMOUNT_mean', 'WITHDRAW_HDSYYF_sum', 'HDSYYFZE_USAGE_AMOUNT_mean', 'numsOf_DKYE_HDYF_div', 'numsOf_GRZHYE_HDSYYF_add', 'numsOf_DKFFE_YEARPURINCM_multi', 'WITHDRAW_HDZYF_max', 'USAGEOVER_in_WITHSIGN2_prop', 'WITHDRAW_SUBMIT_DKYE_min', 'USAGE_RATE_DKLL_sum', 'DKYE_DKLL_sum', 'DWSSHY_DKFFE_genFeat3_mean', 'GRZHDNGJYE_HDSYYFZE_sum', 'DKYE_USAGE_AMOUNT_mean', 'DKXE_DKLL_sum', 'USAGE_AMOUNT_DKFFE_sum', 'numsOf_DKLL_WITHDRAW_SUBMIT_add', 'DWSSHY_DKYE_max', 'numsOf_TOP_USER1_WITHDRAW_SUBMIT_add', 'ZHIYE_DKFFE_genFeat6_mean', 'DKFFE_WITHDRAW_sum', 'TOP_USER_DKXE_max', 'ZHIYE_YEARPURINCM_std', 'YEARPURINCM_HDYF_mean', 'numsOf_GJJRATE_DKXE_diff', 'DKXE_USAGE_RATE_sum', 'GRJCJS_HDSYYFZE_sum', 'numsOf_DKFFE_HDZYF_multi', 'numsOf_HDYF_WITHDRAW_div', 'DWJJLX_count', 'numsOf_DKYE_HDSYYF_multi', 'WITHSIGN_in_ZHIYE_prop', 'WITHDRAW_SUBMIT_USAGE_RATE_sum', 'DKYE_GRZHDNGJYE_sum', 'numsOf_GRZHDNGJYE_HDSYYF_div', 'numsOf_GRZHYE_GRZHDNGJYE_add', 'numsOf_GRZHSNJZYE_GRZHDNGJYE_diff', 'WITHSIGN2_DKXE_mean', 'TOP_USER_in_XINGBIE_prop', 'numsOf_DKFFE_DKYE_multi',
                                                                                                                     'numsOf_DKYE_HDZYFZE_diff', 'GED_GRY', 'HDSYYFZE_DKXE_min', 
                                                                                                 'TOP_USER_in_ZHIYE_prop'] + ['--QIANGXIANXINGXIANGGUAN-95','numsOf_HDZYF_TOP_USER1_div', 'GJJRATE_GRYJCE_std', 'GJSIGN_in_DWJJLX_prop', 'numsOf_GRZHYE_HDZYF_multi', 'GRZHYE_WITHDRAW_mean', 'USAGE_AMOUNT_TOP_USER1_min', 'DWSSHY_ZHICHEN_count', 'numsOf_GRZHSNJZYE_WITHDRAW_diff', 'YEARPURINCM_TOP_USER_mean', 'YEARPURINCM_DKLL_sum', 'GRJCJS_GRYJCE_mean', 'numsOf_GRJCJS_YEARPURINCM_add', 'DWSSHY_GRZHYE_mean', 'DKYE_GRZHYE_mean', 'DKFFE_GRJCJS_max', 'WITHDRAW_SUBMIT_HDZYF_min', 'WITHSIGN2_in_DWSSHY_prop', 'numsOf_GRZHYE_HDSYYFZE_add', 'WITHDRAW_YEARPURINCM_max', 'numsOf_HDZYF_HDYF_add', 'numsOf_YEARPURINCM_USAGE_AMOUNT_add', 'GRZHSNJZYE_WITHDRAW_SUBMIT_max', 'numsOf_HDSYYFZE_TOP_USER1_diff', 'TOP_USER1_HDZYFZE_mean', 'GRZHYE_GRJCJS_max', 'XINGBIE_in_DWJJLX_prop', 'DKYE_TOP_USER_min', 'WITHDRAW_SUBMIT_GRJCJS_max', 'GRYJCE_USAGE_RATE_sum', 'DKFFE_YEARPURINCM_max', 'numsOf_USAGE_RATE_GJJRATE_add', 'GRZHDNGJYE_HDZYF_mean', 'TOP_USER1_HDSYYFZE_max', 'HDSYYFZE_GJJRATE_min', 'YEARPURINCM_HDZYFZE_max', 'WITHDRAW_SUBMIT_GRZHSNJZYE_max', 'numsOf_GRJCJS_HDSYYF_multi', 'GRZHSNJZYE_HDSYYF_min', 'HDSYYF_GRZHDNGJYE_mean', 'DWSSHY_USAGE_RATE_mean', 'TOP_USER1_WITHDRAW_min', 'USAGE_RATE_TOP_USER1_min', 'USAGE_RATE_GRJCJS_mean', 'GRZHSNJZYE_GRZHYE_min', 'DKFFE_HDYF_min', 'numsOf_GRJCJS_YEARPURINCM_diff', 'numsOf_HDZYF_DKXE_div', 'ZHIYE_HDSYYFZE_mean', 'numsOf_GRZHDNGJYE_HDSYYF_multi', 'HDSYYFZE_HDSYYF_min', 'DKXE_GRYJCE_sum', 'GRZHYE_GRZHDNGJYE_min', 'numsOf_GRZHYE_USAGE_AMOUNT_add', 'numsOf_GRZHSNJZYE_HDSYYFZE_diff', 'numsOf_YEARPURINCM_USAGE_AMOUNT_diff', 'TOP_USER1_GRZHSNJZYE_max', 'DKXE_GRYJCE_mean', 'GJJRATE_YEARPURINCM_max', 'numsOf_GRZHSNJZYE_HDZYF_multi', 'GRYJCE_HDYF_sum', 'GRZHZT', 'DKFFE_WITHDRAW_max', 'TOP_USER_DKXE_min', 'GRZHYE_HDZYF_mean', 'DWSSHY_WITHDRAW_mean', 'GJSIGN_DKXE_sum', 'DWJJLX_GRZHYE_mean', 'DKYE_USAGE_RATE_min', 'HDZYF_DKXE_min', 'WITHDRAW_SUBMIT_GRZHYE_max', 'USAGE_AMOUNT_HDZYF_max', 'USAGE_RATE_GRJCJS_max', 'ZHIYE_HDZYFZE_mean', 'WITHDRAW_GRJCJS_mean', 'DWJJLX_GRJCJS_mean', 'YEARPURINCM_HDZYF_max', 'XINGBIE_HDSYYFZE_max', 'TOP_USER1_GRZHDNGJYE_max', 'DKYE_GRJCJS_max', 'DKYE_WITHDRAW_min', 'WITHDRAW_YEARPURINCM_min', 'DWJJLX_TOP_USER_std', 'DKYE_GRZHYE_max', 'HDZYF_HDZYFZE_sum', 'HDSYYF_GRZHSNJZYE_mean', 'USAGE_RATE_GRZHDNGJYE_mean', 'USAGE_AMOUNT_GRJCJS_min', 'HDSYYFZE_GRYJCE_mean', 'HDSYYFZE_DKXE_max', 'HDSYYF_HDYF_mean', 'YEARPURINCM_HDYF_min', 'HDZYF_GRYJCE_mean', 'numsOf_YEARPURINCM_HDZYF_multi', 'GRZHZT_WITHSIGN2_count', 'WITHSIGN2_DKXE_min', 'GRZHSNJZYE_TOP_USER1_mean', 'DKYE_GJJRATE_mean', 'numsOf_GRZHDNGJYE_YEARPURINCM_add', 'HDSYYFZE_GRYJCE_min', 'DKYE_GRYJCE_mean', 'numsOf_HDYF_TOP_USER_diff', 'DWJJLX_in_WITHSIGN2_prop', 'USAGE_RATE_USAGE_AMOUNT_mean', 'GRZHDNGJYE_GRYJCE_mean', 'USAGE_RATE_TOP_USER1_sum', 'GRZHDNGJYE_HDZYF_min', 'DKYE_HDSYYF_mean', 'GRZHYE_GRJCJS_mean', 'HDSYYFZE_DKXE_sum', 'numsOf_GRJCJS_HDSYYFZE_multi', 'DKYE_GRJCJS_min', 'HDSYYFZE_WITHDRAW_SUBMIT_sum', 'numsOf_HDZYFZE_GJJRATE_multi', 'numsOf_HDSYYFZE_USAGE_AMOUNT_multi', 'XINGBIE_WITHDRAW_std', 'GRZHSNJZYE_GRZHYE_mean', 'HDSYYFZE_WITHDRAW_sum', 'numsOf_GRJCJS_GRZHDNGJYE_diff', 'USAGE_AMOUNT_HDZYFZE_mean', 'TOP_USER1_HDZYF_mean', 'GRZHYE_YEARPURINCM_mean', 'HDSYYF_GRJCJS_min', 'GJJRATE_in_TOP_USER_prop', 'DKYE_YEARPURINCM_sum', 'GRZHYE_GRJCJS_min', 'HDYF_WITHDRAW_SUBMIT_min', 'TOP_USER1_GRYJCE_mean', 'DKXE_TOP_USER1_sum', 'USAGE_AMOUNT_HDZYF_mean', 'DKXE_GRZHSNJZYE_sum', 'GRZHDNGJYE_HDYF_sum', 'DKYE_GRZHDNGJYE_min', 'USAGE_RATE_YEARPURINCM_mean', 'GRZHYE_DKXE_mean', 'TOP_USER_YEARPURINCM_std', 'numsOf_HDSYYF_HDYF_diff', 'HDSYYFZE_GRZHDNGJYE_sum', 'WITHDRAW_SUBMIT_HDZYF_mean', 'DWJJLX_in_WITHSIGN_prop', 'USAGE_RATE_TOP_USER1_mean', 'HDZYF_GRYJCE_sum', 'WITHDRAW_GRYJCE_mean', 'XINGBIE_ZHICHEN_count', 'GRZHSNJZYE_HDZYF_mean', 'DWSSHY_in_ZHIYE_prop', 'HDSYYF_GJJRATE_min', 'DWSSHY_HDSYYFZE_mean', 'GRYJCE_GRZHDNGJYE_sum', 'HDSYYF_GRJCJS_max', 'TOP_USER1_GRZHDNGJYE_min', 'HDYF_TOP_USER_max', 'DKYE_GJJRATE_max', 'HDZYFZE_GRZHDNGJYE_min', 'numsOf_HDZYF_WITHDRAW_SUBMIT_div', 'GRZHDNGJYE_GRYJCE_min', 'DWSSHY_GRYJCE_sum', 'HDZYF_GRZHYE_sum', 'numsOf_GRJCJS_HDSYYF_add', 'DWSSHY_USAGE_RATE_sum', 'GRZHYE_HDZYF_min', 'ZHIYE_GRZHDNGJYE_mean', 'DWJJLX_GRZHSNJZYE_std', 'GRYJCE_GRJCJS_min', 'GRJCJS_HDSYYF_max', 'DKYE_USAGE_AMOUNT_max', 'USAGE_RATE_HDZYF_min', 'TOP_USER_WITHDRAW_min', 'numsOf_TOP_USER_WITHDRAW_SUBMIT_add', 'HDSYYFZE_YEARPURINCM_mean', 'numsOf_HDZYFZE_HDSYYFZE_add', 'numsOf_GRZHSNJZYE_TOP_USER1_diff', 'GRZHSNJZYE_DKLL_mean', 'ZHIYE_GRZHDNGJYE_std', 'numsOf_GRJCJS_WITHDRAW_multi', 'GRZHYE_GRZHSNJZYE_mean', 'USAGE_RATE_YEARPURINCM_max', 'numsOf_YEARPURINCM_TOP_USER1_add', 'WITHDRAW_WITHDRAW_SUBMIT_mean', 'DWSSHY_HDSYYF_mean', 'GRZHSNJZYE_TOP_USER_min', 'YEARPURINCM_WITHDRAW_SUBMIT_max', 'HDSYYFZE_GRJCJS_min', 'numsOf_GRZHDNGJYE_GRYJCE_div', 'HDZYFZE_HDZYF_mean', 'GRJCJS_GRYJCE_max', 'WITHSIGN_in_TOP_USER_prop', 'HDYF_HDZYFZE_max', 'numsOf_GRZHDNGJYE_WITHDRAW_add', 'numsOf_GRJCJS_TOP_USER_div', 'HDSYYFZE_GRZHYE_min', 'HDYF_DKXE_min', 'GRZHSNJZYE_GJJRATE_min', 'HDSYYF_GRJCJS_mean', 'DKXE_HDZYF_mean', 'numsOf_USAGE_RATE_WITHDRAW_SUBMIT_multi', 'DWJJLX_in_XINGBIE_prop', 'HDSYYF_GRZHYE_mean', 'HDSYYFZE_GRZHSNJZYE_min', 'DWJJLX_1', 'numsOf_HDZYFZE_GJJRATE_div', 'DWJJLX_USAGE_RATE_mean', 'WITHDRAW_DKXE_mean', 'DKLL_GRJCJS_mean', 'WITHDRAW_HDZYF_mean', 'numsOf_GRJCJS_WITHDRAW_SUBMIT_div', 'HDZYFZE_GRJCJS_max', 'GRJCJS_TOP_USER_sum', 'TOP_USER_in_USAGEOVER_prop', 'HDSYYFZE_GRJCJS_max', 'WITHDRAW_SUBMIT_GRJCJS_min', 'numsOf_GRZHYE_HDSYYFZE_diff', 'GRZHSNJZYE_HDSYYFZE_mean', 'WITHDRAW_HDSYYFZE_mean', 'GRZHYE_GRYJCE_mean', 'YEARPURINCM_WITHDRAW_sum', 'GJJRATE_GRZHYE_max', 'GRYJCE_HDYF_max', 'HDZYF_DKXE_mean', 'GRJCJS_GRZHDNGJYE_sum', 'HDSYYFZE_HDZYFZE_mean', 'HDYF_HDSYYF_min', 'DKYE_YEARPURINCM_max', 'DKYE_WITHDRAW_SUBMIT_max', 'GRZHSNJZYE_TOP_USER1_min', 'GRZHSNJZYE_USAGE_RATE_max', 'numsOf_GRYJCE_HDYF_diff', 'numsOf_GRJCJS_DKXE_multi', 'TOP_USER_in_WITHSIGN_prop', 'DKYE_HDZYF_max', 'numsOf_GRZHYE_TOP_USER1_diff', 'numsOf_GRYJCE_WITHDRAW_multi', 'TOP_USER1_HDYF_max', 'TOP_USER_in_WITHSIGN2_prop', 'HDSYYFZE_GJJRATE_mean', 'HDZYF_GRJCJS_min', 'numsOf_YEARPURINCM_HDSYYFZE_diff', 'HDZYF_GRZHSNJZYE_sum', 'WITHDRAW_TOP_USER1_mean', 'USAGE_RATE_HDZYF_mean', 'DWSSHY_HDZYFZE_sum', 'DKFFE_HDZYF_mean', 'TOP_USER1_GRJCJS_max', 'GRZHZT_ZHIYE_count', 'GJJRATE_GRZHDNGJYE_mean', 'DKYE_YEARPURINCM_min', 'USAGE_RATE_GJJRATE_sum', 'USAGE_AMOUNT_DKLL_mean', 'DWSSHY_GRZHSNJZYE_mean', 'GRZHYE_GRZHDNGJYE_mean', 'YEARPURINCM_HDSYYFZE_min', 'numsOf_GRZHDNGJYE_WITHDRAW_diff', 'HDZYFZE_GRZHYE_max', 'USAGE_AMOUNT_HDZYF_min', 'DWSSHY_in_USAGEOVER_prop', 'WITHDRAW_GRZHSNJZYE_max', 'DKXE_GRJCJS_min', 'numsOf_HDZYF_TOP_USER_div', 'TOP_USER_WITHDRAW_SUBMIT_min', 'numsOf_GRZHDNGJYE_DKXE_div', 'USAGE_RATE_GRYJCE_min', 'GRZHDNGJYE_USAGE_RATE_sum', 'GRZHSNJZYE_TOP_USER_mean', 'numsOf_GRZHYE_HDZYFZE_diff', 'ZHIYE_GJJRATE_std', 'DWJJLX_HDSYYF_std', 'WITHSIGN2_in_DWJJLX_prop', 'ZHIYE_HDYF_std', 'HDSYYFZE_DKLL_mean', 'GRJCJS_GRZHDNGJYE_min', 'GRYJCE_GRZHDNGJYE_min', 'DWSSHY_GRZHSNJZYE_std', 'numsOf_HDZYF_GJJRATE_multi', 'WITHDRAW_SUBMIT_YEARPURINCM_max', 'DWSSHY_in_ZHICHEN_prop', 'HDYF_GRZHYE_mean', 'GRZHSNJZYE_GJJRATE_mean', 'HDYF_GJJRATE_mean', 'YEARPURINCM_TOP_USER1_min', 'HDSYYFZE_GRZHSNJZYE_mean', 'USAGE_RATE_HDSYYF_mean', 'YEARPURINCM_HDYF_max', 'USAGE_RATE_GJJRATE_min', 'DWSSHY_GRJCJS_sum', 'numsOf_GRJCJS_WITHDRAW_SUBMIT_diff', 'numsOf_GRZHSNJZYE_HDZYFZE_add', 'HDZYFZE_HDSYYFZE_mean', 'HDZYF_TOP_USER1_mean', 'HDYF_GRJCJS_max', 'DKXE_GRJCJS_max', 'USAGE_RATE_HDSYYFZE_mean', 'YEARPURINCM_TOP_USER1_max', 'DWSSHY_TOP_USER_sum', 'USAGE_RATE_GRZHDNGJYE_min', 'DKYE_HDZYFZE_max', 'WITHDRAW_USAGE_RATE_min', 'DWJJLX_YEARPURINCM_std', 'DKYE_GRZHSNJZYE_min', 'GRZHYE_GRZHSNJZYE_min', 'numsOf_HDZYF_TOP_USER1_multi', 'USAGE_RATE_HDZYF_sum', 'HDYF_GRZHDNGJYE_mean', 'YEARPURINCM_HDZYF_min', 'HDYF_HDZYF_max', 'DKFFE_GRZHYE_max', 'DKYE_TOP_USER1_max', 'YEARPURINCM_DKLL_mean', 'HDSYYF_GRZHYE_min', 'USAGE_RATE_GRJCJS_min', 'GRJCJS_DKXE_min', 'DKYE_GJJRATE_min', 'HDSYYFZE_HDSYYF_mean', 'WITHSIGN_in_USAGEOVER_prop', 'numsOf_HDZYFZE_HDSYYFZE_multi', 'GJJRATE_GRJCJS_std', 'DKYE_TOP_USER1_min', 'TOP_USER1_HDSYYF_max', 'HDZYF_GRJCJS_mean', 'HDSYYFZE_HDZYF_min', 'TOP_USER_USAGE_RATE_mean', 'WITHDRAW_HDZYFZE_mean', 'USAGE_AMOUNT_USAGE_RATE_max', 'GRYJCE_HDSYYF_mean', 'DWSSHY_GRZHYE_std', 'WITHSIGN_in_DWSSHY_prop', 'USAGE_RATE_HDSYYFZE_sum', 'WITHDRAW_GRZHYE_max', 'DKYE_YEARPURINCM_mean', 'DKYE_WITHDRAW_max', 'TOP_USER_YEARPURINCM_mean', 'DKFFE_GRZHDNGJYE_max', 'YEARPURINCM_HDSYYF_min', 'HDSYYFZE_HDZYF_mean', 'HDYF_GRJCJS_mean', 'numsOf_HDZYF_TOP_USER_multi', 'numsOf_HDSYYF_USAGE_RATE_div', 'DKYE_USAGE_RATE_sum', 'USAGE_RATE_WITHDRAW_SUBMIT_min', 'numsOf_GRYJCE_HDZYF_add', 'YEARPURINCM_GRJCJS_sum', 'numsOf_DKLL_HDZYFZE_multi', 'WITHDRAW_HDYF_max', 'numsOf_GRZHYE_TOP_USER1_add', 'GRZHSNJZYE_USAGE_AMOUNT_min', 'DWSSHY_GRYJCE_mean', 'GJJRATE_TOP_USER_mean', 'GRZHSNJZYE_GRJCJS_max', 'USAGE_RATE_GJJRATE_mean', 'GRJCJS_HDYF_max', 'HDYF_GRZHSNJZYE_mean', 'HDSYYFZE_GRJCJS_mean', 'GRZHYE_USAGE_AMOUNT_mean', 'TOP_USER1_GRZHYE_max', 'DKYE_GRJCJS_mean', 'numsOf_USAGE_RATE_TOP_USER_add', 'GRZHYE_TOP_USER1_mean', 'HDYF_GRJCJS_min', 'GRZHYE_HDYF_max', 'numsOf_GRJCJS_GJJRATE_multi', 'GRZHZT_TOP_USER_count', 'HDSYYFZE_TOP_USER1_mean', 'YEARPURINCM_GRZHDNGJYE_sum', 'DWJJLX_in_ZHICHEN_prop', 'YEARPURINCM_HDSYYF_max', 'WITHDRAW_HDZYF_min', 'numsOf_GRZHYE_USAGE_AMOUNT_diff', 'numsOf_GRJCJS_WITHDRAW_diff', 'GJJRATE_GRZHDNGJYE_sum', 'WITHDRAW_SUBMIT_GRJCJS_mean', 'GRZHZT_WITHDRAW_mean', 'GRZHSNJZYE_GRJCJS_min', 'DWSSHY_GRZHZT_count', 'numsOf_YEARPURINCM_HDSYYFZE_add', 'GRZHSNJZYE_YEARPURINCM_mean', 'numsOf_HDSYYFZE_USAGE_RATE_multi', 'YEARPURINCM_HDZYFZE_min', 'HDZYF_GRYJCE_min', 'numsOf_GRZHDNGJYE_HDZYF_multi', 'HDSYYFZE_GRZHDNGJYE_min', 'DKYE_HDSYYF_max', 'GRZHSNJZYE_GRZHDNGJYE_mean', 'numsOf_GRZHSNJZYE_WITHDRAW_add', 'HDZYF_WITHDRAW_sum', 'GRZHDNGJYE_GRYJCE_sum', 'NOUSAGE_GJJRATE_count', 'DKYE_DKLL_mean', 'DKFFE_GRZHDNGJYE_sum', 'HDSYYF_GJJRATE_mean', 'DKYE_GRZHSNJZYE_max', 'USAGE_RATE_GRYJCE_mean', 'DKYE_HDSYYF_min', 'numsOf_HDZYFZE_WITHDRAW_SUBMIT_multi', 'HDZYF_TOP_USER1_min', 'numsOf_DKLL_GJJRATE_diff', 'GRZHSNJZYE_HDZYFZE_mean', 'numsOf_YEARPURINCM_TOP_USER1_diff', 'GRZHDNGJYE_WITHDRAW_SUBMIT_mean', 'GRZHSNJZYE_HDYF_min', 'HDSYYFZE_GRZHDNGJYE_mean', 'numsOf_GRZHSNJZYE_TOP_USER1_add', 'GJJRATE_HDYF_mean', 'GRZHDNGJYE_WITHDRAW_SUBMIT_sum', 'numsOf_YEARPURINCM_HDZYFZE_add', 'DKFFE_HDSYYF_max', 'TOP_USER_HDZYF_mean', 'GRZHZT_XINGBIE_count', 'numsOf_GRJCJS_HDZYFZE_multi', 'numsOf_DKLL_GJJRATE_add', 'WITHDRAW_USAGE_AMOUNT_mean', 'DWSSHY_GRJCJS_std', 'HDSYYF_GRYJCE_mean', 'TOP_USER_YEARPURINCM_sum', 'GRZHYE_DKXE_min', 'numsOf_HDZYFZE_TOP_USER1_multi', 'HDYF_HDSYYFZE_max', 'TOP_USER_in_DWJJLX_prop', 'numsOf_DKLL_GJJRATE_div', 'DKYE_WITHDRAW_SUBMIT_min', 'GRZHYE_DKLL_mean', 'GRZHDNGJYE_GRZHYE_mean', 'HDSYYFZE_GJJRATE_max', 'DKYE_GRYJCE_max', 'DWJJLX_GRZHSNJZYE_mean', 'numsOf_HDZYF_HDZYFZE_multi', 'numsOf_HDZYF_GJJRATE_div', 'DWSSHY_GRZHDNGJYE_max', 'HDSYYFZE_HDYF_min', 'DKXE_GJJRATE_sum', 'TOP_USER_GRZHSNJZYE_mean', 'HDZYF_GRYJCE_max', 'DKFFE_HDSYYFZE_mean', 'GRYJCE_HDSYYF_max', 'USAGE_AMOUNT_TOP_USER_mean', 'GJJRATE_TOP_USER1_mean', 'YEARPURINCM_WITHDRAW_SUBMIT_min', 'DKYE_GRZHDNGJYE_max', 'DWSSHY_GRZHSNJZYE_sum', 'NOUSAGE_ZHICHEN_count', 'USAGE_RATE_WITHDRAW_SUBMIT_mean', 'USAGE_AMOUNT_GRJCJS_max', 'numsOf_DKYE_HDZYFZE_div', 'DKYE_GRZHYE_min', 'GRZHSNJZYE_WITHDRAW_SUBMIT_min', 'DKYE_HDSYYFZE_sum', 'WITHSIGN2_in_TOP_USER_prop', 'HDYF_HDSYYF_mean', 'ZHIYE_3', 'HDYF_YEARPURINCM_mean', 'GRZHSNJZYE_WITHDRAW_mean', 'XINGBIE_YEARPURINCM_std', 'numsOf_DKFFE_YEARPURINCM_div', 'DKFFE_GRZHSNJZYE_max', 'DKYE_GRYJCE_min', 'GJJRATE_HDYF_std', 'HDYF_TOP_USER1_max', 'HDYF_DKLL_max', 'GRZHYE_USAGE_RATE_mean', 'TOP_USER1_GRJCJS_min', 'numsOf_GRZHSNJZYE_HDZYFZE_diff', 'HDYF_GRZHSNJZYE_min', 'DWJJLX_WITHDRAW_mean', 'numsOf_GRZHYE_HDZYFZE_add', 'USAGE_RATE_WITHDRAW_mean', 'numsOf_HDSYYF_HDZYFZE_div', 'HDYF_WITHDRAW_SUBMIT_sum', 'GRZHSNJZYE_HDYF_mean', 'HDSYYFZE_HDSYYF_sum', 'USAGE_RATE_GRZHDNGJYE_sum', 'GRZHDNGJYE_HDSYYF_sum', 'DKYE_TOP_USER_mean', 'USAGE_RATE_HDSYYF_min', 'numsOf_GRJCJS_USAGE_AMOUNT_div', 'TOP_USER_GRZHDNGJYE_mean', 'numsOf_HDSYYF_HDYF_add', 'HDSYYFZE_WITHDRAW_SUBMIT_mean', 'HDYF_GRZHYE_min', 'GRZHDNGJYE_WITHDRAW_max', 'HDYF_HDZYF_min', 'numsOf_GRYJCE_HDZYF_diff', 'GRZHSNJZYE_HDZYF_min', 'HDYF_WITHDRAW_mean', 'GRYJCE_GRJCJS_max', 'numsOf_GRZHYE_WITHDRAW_diff', 'GJSIGN_WITHSIGN2_count', 'WITHSIGN2_in_NOUSAGE_prop', 'DWSSHY_DKXE_std', 'WITHDRAW_TOP_USER1_min', 'DKLL_YEARPURINCM_sum', 'HDSYYF_WITHDRAW_mean', 'numsOf_GRJCJS_TOP_USER1_div', 'USAGE_RATE_DKLL_mean', 'numsOf_HDZYF_HDZYFZE_div', 'DWSSHY_GJJRATE_mean', 'numsOf_GRJCJS_TOP_USER1_multi', 'USAGE_AMOUNT_GRZHDNGJYE_min', 'DKYE_WITHDRAW_mean', 'WITHDRAW_GRJCJS_max', 'TOP_USER1_WITHDRAW_SUBMIT_max', 'WITHDRAW_GRYJCE_min', 'USAGE_RATE_DKXE_min', 'YEARPURINCM_HDSYYFZE_max', 'GRYJCE_USAGE_AMOUNT_sum', 'DWJJLX_HDSYYF_mean', 'numsOf_GRZHYE_WITHDRAW_SUBMIT_diff', 'GRZHDNGJYE_GRZHSNJZYE_sum', 'HDSYYF_DKXE_min', 'WITHSIGN_in_DWJJLX_prop', 'numsOf_GRJCJS_TOP_USER_multi', 'XINGBIE_in_GJJRATE_prop', 'GRJCJS_GRYJCE_sum', 'WITHDRAW_SUBMIT_YEARPURINCM_min', 'numsOf_GRJCJS_WITHDRAW_SUBMIT_multi', 'DWJJLX_HDSYYF_sum', 'TOP_USER1_YEARPURINCM_max', 'WITHDRAW_SUBMIT_HDYF_max', 'DWJJLX_HDSYYFZE_mean', 'WITHDRAW_YEARPURINCM_mean', 'DWSSHY_GRJCJS_mean', 'USAGE_AMOUNT_HDSYYFZE_mean', 'GRZHSNJZYE_HDSYYF_mean', 'HDZYFZE_HDYF_sum', 'GRZHYE_GRZHDNGJYE_max', 'GRZHSNJZYE_GRYJCE_min', 'XINGBIE_in_TOP_USER_prop', 'DKYE_GRZHDNGJYE_mean', 'HDYF_GRYJCE_min', 'DWSSHY_GRZHDNGJYE_std', 'TOP_USER1_YEARPURINCM_min', 'DWJJLX_GRZHYE_std', 'numsOf_YEARPURINCM_HDZYFZE_diff', 'HDSYYFZE_GRZHYE_mean', 'DKYE_HDZYF_mean', 'numsOf_GRZHSNJZYE_HDSYYFZE_add', 'HDZYFZE_GRZHDNGJYE_sum', 'DKYE_GRZHSNJZYE_mean', 'HDSYYFZE_GRYJCE_max', 'numsOf_GRYJCE_YEARPURINCM_multi', 'WITHDRAW_SUBMIT_GRZHDNGJYE_mean', 'TOP_USER_GRZHSNJZYE_std', 'GJSIGN_in_DWSSHY_prop', 'DWJJLX_GRZHDNGJYE_std', 'DKLL_GRJCJS_max', 'DKLL_HDYF_max', 'numsOf_GRZHDNGJYE_YEARPURINCM_diff', 'DKYE_HDZYF_min', 'GRZHDNGJYE_GRJCJS_mean', 'HDSYYF_WITHDRAW_SUBMIT_mean', 'USAGE_AMOUNT_GRJCJS_mean', 'numsOf_HDSYYFZE_TOP_USER1_multi', 'GRZHSNJZYE_DKXE_min', 'HDYF_HDZYF_mean', 'USAGE_AMOUNT_USAGE_RATE_min', 'DWSSHY_HDYF_sum', 'GRZHDNGJYE_USAGE_AMOUNT_sum', 'GJSIGN_ZHICHEN_count', 'WITHDRAW_GRJCJS_min', 'GJSIGN_in_GJJRATE_prop', 'HDSYYFZE_WITHDRAW_mean', 'GRZHDNGJYE_YEARPURINCM_sum', 'HDZYFZE_HDSYYF_max', 'HDSYYFZE_HDSYYF_max', 'WITHDRAW_SUBMIT_GRYJCE_min', 'DWJJLX_in_TOP_USER_prop', 'GRZHYE_HDYF_mean', 'HDZYF_HDSYYFZE_sum', 'DKXE_HDSYYF_sum', 'GRJCJS_WITHDRAW_SUBMIT_sum', 'GRZHDNGJYE_GRJCJS_min', 
                                                                                                                              'HDYF_HDSYYFZE_mean'] + ['TICHU_QUEREN','ZHICHEN_DKFFE_genFeat7_mean', 'ZHIYE_WITHDRAW_SUBMIT_mean', 'TOP_USER_DKXE_sum', 'TOP_USER_DKXE_mean', 'WITHSIGN_DKXE_mean', 'WITHSIGN_DKXE_max', 'XINGBIE_DKXE_min', 'ZHIYE_USAGE_RATE_std', 'ZHIYE_DKXE_mean', 'ZHIYE_TOP_USER1_std', 'YEARPURINCM_GRZHDNGJYE_std', 'ZHIYE_WITHDRAW_SUBMIT_std', 'WITHDRAW_SUBMIT_DKLL_min', 'DKLL_DKXE_min', 'TOP_USER_GJJRATE_std', 'ZHIYE_WITHDRAW_std', 'WITHDRAW_SUBMIT_GRYJCE_std', 'DWJJLX_DKXE_sum', 'ZHIYE_GJJRATE_mean', 'numsOf_YEARPURINCM_WITHDRAW_SUBMIT_diff', 'ZHIYE_TOP_USER1_mean', 'ZHIYE_TOP_USER_max', 'DWSSHY_GRZHDNGJYE_min', 'DWSSHY_GJJRATE_max', 'ZHIYE_TOP_USER_std', 'ZHIYE_TOP_USER_mean', 'TOP_USER_WITHDRAW_SUBMIT_std', 'TOP_USER_HDSYYFZE_std', 'ZHICHEN_in_ZHIYE_prop', 'GRZHSNJZYE_WITHDRAW_std', 'HDSYYF_DKXE_std', 'DWSSHY_9', 'DKFFE_DWJJLX_min', 'GJSIGN_in_USAGEOVER_prop', 'USAGEOVER_GJSIGN_count', 'ZHIYE_DKXE_std', 'DWSSHY_2', 'HDSYYF_GRJCJS_std', 'TOP_USER_6', 'DWSSHY_14', 'TOP_USER1_GJJRATE_min', 'ZHIYE_DKXE_min', 'ZHIYE_USAGE_RATE_mean', 'ZHIYE_WITHDRAW_mean', 'YEARPURINCM_DKLL_std', 'GJSIGN_DKXE_mean', 'TOP_USER_GRJCJS_std', 'ZHIYE_HDSYYFZE_max', 'ZHIYE_HDSYYFZE_std', 'TOP_USER_GRZHSNJZYE_max', 'ZHIYE_HDZYFZE_std', 'ZHIYE_HDSYYF_std', 'GJSIGN_DKXE_max', 'HDZYFZE_GJJRATE_min', 
                                                                                                                              'GJSIGN_DKXE_min'] + ['<100','numsOf_WITHDRAW_SUBMIT_DKXE_multi', 'HDZYF_WITHDRAW_SUBMIT_min', 'numsOf_HDSYYF_GJJRATE_multi', 'DWJJLX_WITHSIGN2_count', 'USAGE_RATE_DKXE_std', 'GRZHDNGJYE_GRZHSNJZYE_max', 'numsOf_GRZHYE_HDZYFZE_div', 'GRZHDNGJYE_TOP_USER1_std', 'GRZHDNGJYE_HDSYYFZE_mean', 'numsOf_YEARPURINCM_USAGE_AMOUNT_multi', 'numsOf_YEARPURINCM_HDZYFZE_div', 'GRZHDNGJYE_HDYF_std', 'USAGE_RATE_WITHDRAW_std', 'HDSYYFZE_GRZHYE_max', 'numsOf_USAGE_AMOUNT_DKXE_div', 'GRZHDNGJYE_YEARPURINCM_min', 'DWSSHY_TOP_USER1_std', 'numsOf_GRZHYE_TOP_USER1_div', 'HDZYFZE_GRZHSNJZYE_mean', 'USAGE_AMOUNT_GRZHSNJZYE_max', 'numsOf_USAGE_RATE_TOP_USER_diff', 'numsOf_HDSYYFZE_WITHDRAW_multi', 'numsOf_HDYF_GJJRATE_multi', 'DWSSHY_TOP_USER1_mean', 'USAGE_AMOUNT_GRZHDNGJYE_max', 'GRZHYE_HDSYYF_mean', 'numsOf_YEARPURINCM_TOP_USER1_div', 'numsOf_HDYF_TOP_USER1_multi', 'USAGE_RATE_DKXE_sum', 'numsOf_TOP_USER1_WITHDRAW_SUBMIT_diff', 'numsOf_HDSYYF_WITHDRAW_SUBMIT_multi', 'numsOf_HDSYYF_DKXE_div', 'numsOf_GRZHYE_GRZHSNJZYE_add', 'numsOf_GRZHYE_USAGE_RATE_multi', 'USAGE_RATE_HDSYYF_max', 'HDZYF_HDSYYF_min', 'USAGE_RATE_WITHDRAW_SUBMIT_std', 'TOP_USER1_DKLL_std', 'GRJCJS_HDZYFZE_min', 'numsOf_USAGE_RATE_TOP_USER1_div', 'numsOf_DKLL_HDZYF_div', 'GRZHYE_USAGE_AMOUNT_min', 'numsOf_WITHDRAW_SUBMIT_DKXE_div', 'DKXE_USAGE_RATE_min', 'DKYE_GRZHSNJZYE_std', 'HDSYYFZE_GRZHSNJZYE_max', 'numsOf_GRZHYE_GRZHSNJZYE_multi', 'DKXE_USAGE_AMOUNT_sum', 'DKFFE_USAGE_RATE_std', 'numsOf_GRJCJS_DKXE_div', 'DKXE_TOP_USER1_max', 'HDZYF_HDYF_max', 'USAGE_RATE_GRZHYE_max', 'numsOf_HDYF_WITHDRAW_SUBMIT_add', 'USAGE_AMOUNT_GJJRATE_mean', 'DKFFE_HDYF_std', 'numsOf_GRZHSNJZYE_WITHDRAW_SUBMIT_multi', 'TOP_USER1_HDYF_std', 'DWSSHY_HDYF_mean', 'HDZYF_USAGE_AMOUNT_max', 'numsOf_HDSYYF_TOP_USER_div', 'numsOf_GRZHDNGJYE_HDZYFZE_div', 'HDZYF_USAGE_RATE_max', 'WITHDRAW_SUBMIT_HDSYYF_mean', 'USAGE_AMOUNT_YEARPURINCM_min', 'numsOf_WITHDRAW_DKXE_multi', 'GRZHDNGJYE_USAGE_AMOUNT_mean', 'numsOf_HDSYYF_TOP_USER1_div', 'numsOf_HDYF_WITHDRAW_SUBMIT_diff', 'DWSSHY_in_GJSIGN_prop', 'HDZYF_HDSYYFZE_min', 'USAGE_RATE_HDZYF_max', 'numsOf_HDSYYF_HDSYYFZE_multi', 'numsOf_USAGE_RATE_WITHDRAW_multi', 'GRZHDNGJYE_HDSYYF_min', 'GRZHDNGJYE_GJJRATE_std', 'GRZHYE_HDSYYF_min', 'DWSSHY_in_WITHSIGN2_prop', 'numsOf_HDSYYF_USAGE_AMOUNT_div', 'numsOf_HDZYFZE_WITHDRAW_SUBMIT_diff', 'DWSSHY_DKXE_mean', 'GRZHDNGJYE_HDZYF_std', 'numsOf_GRZHSNJZYE_WITHDRAW_multi', 'HDZYFZE_HDSYYF_mean', 'numsOf_GRZHYE_DKXE_multi', 'HDZYFZE_USAGE_AMOUNT_min', 'numsOf_GRZHDNGJYE_USAGE_AMOUNT_multi', 'HDZYF_WITHDRAW_SUBMIT_max', 'USAGE_AMOUNT_WITHDRAW_SUBMIT_min', 'GRZHDNGJYE_USAGE_RATE_mean', 'DWSSHY_WITHDRAW_min', 'USAGE_AMOUNT_WITHDRAW_sum', 'DWSSHY_GRYJCE_std', 'DKYE_GRZHDNGJYE_std', 'GRYJCE_HDZYFZE_max', 'DKFFE_GRZHYE_std', 'HDZYF_HDSYYFZE_mean', 'HDZYFZE_TOP_USER1_sum', 'GRYJCE_TOP_USER1_max', 'HDZYF_TOP_USER1_std', 'GRJCJS_HDZYFZE_max', 'TOP_USER1_USAGE_RATE_mean', 'HDSYYFZE_WITHDRAW_min', 'numsOf_GRZHYE_GRZHDNGJYE_multi', 'HDZYFZE_HDYF_min', 'numsOf_GRZHSNJZYE_GRYJCE_multi', 'WITHDRAW_SUBMIT_USAGE_AMOUNT_mean', 'HDZYF_WITHDRAW_max', 'USAGE_RATE_YEARPURINCM_std', 'HDSYYFZE_WITHDRAW_SUBMIT_min', 'GRZHZT_DKXE_min', 'DKXE_HDZYFZE_std', 'DKYE_WITHDRAW_std', 'HDZYF_WITHDRAW_SUBMIT_mean', 'GRZHYE_GRYJCE_min', 'HDZYFZE_GRZHYE_mean', 'HDZYFZE_GRYJCE_std', 'HDZYFZE_DKXE_mean', 'WITHDRAW_USAGE_RATE_mean', 'numsOf_WITHDRAW_DKXE_div', 'DWJJLX_NOUSAGE_count', 'DWSSHY_HDYF_std', 'GRZHDNGJYE_HDZYFZE_mean', 'GRZHYE_USAGE_RATE_max', 'GRZHDNGJYE_GRZHYE_sum', 'numsOf_GRZHSNJZYE_GRZHDNGJYE_multi', 'numsOf_GRZHYE_USAGE_RATE_div', 'WITHDRAW_SUBMIT_HDSYYF_min', 'numsOf_GRZHDNGJYE_GJJRATE_div', 'GRZHDNGJYE_GRZHYE_min', 'numsOf_YEARPURINCM_GJJRATE_multi', 'DWJJLX_DKLL_mean', 'HDZYFZE_YEARPURINCM_sum', 'numsOf_DKLL_USAGE_RATE_multi', 'numsOf_GRZHYE_GJJRATE_div', 'WITHDRAW_HDSYYFZE_max', 'numsOf_WITHDRAW_WITHDRAW_SUBMIT_multi', 'numsOf_WITHDRAW_USAGE_AMOUNT_div', 'WITHDRAW_GRZHYE_mean', 'DWSSHY_TOP_USER1_max', 'numsOf_USAGE_RATE_GJJRATE_diff', 'TOP_USER_WITHSIGN_count', 'numsOf_USAGE_RATE_TOP_USER1_multi', 'numsOf_GRZHDNGJYE_GJJRATE_multi', 'GRZHDNGJYE_HDYF_max', 'numsOf_DKYE_HDZYF_div', 'DWSSHY_HDSYYFZE_max', 'numsOf_TOP_USER_USAGE_AMOUNT_multi', 'DKYE_YEARPURINCM_std', 'numsOf_HDSYYFZE_USAGE_AMOUNT_diff', 'USAGE_AMOUNT_DKXE_sum', 'HDZYFZE_GRZHSNJZYE_std', 'DKFFE_GRZHDNGJYE_mean', 'USAGE_RATE_GRZHDNGJYE_max', 'numsOf_GRZHSNJZYE_YEARPURINCM_multi', 'numsOf_GRJCJS_HDZYF_multi', 'USAGE_RATE_WITHDRAW_min', 'numsOf_GRYJCE_HDSYYF_diff', 'numsOf_HDYF_USAGE_AMOUNT_multi', 'USAGE_AMOUNT_HDSYYF_max', 'USAGE_AMOUNT_WITHDRAW_min', 'HDZYFZE_GRJCJS_min', 'HDSYYFZE_DKXE_std', 'WITHDRAW_TOP_USER_std', 'DKFFE_YEARPURINCM_mean', 'GRZHDNGJYE_YEARPURINCM_max', 'USAGE_AMOUNT_HDSYYFZE_max', 'GRZHDNGJYE_HDSYYF_max', 'numsOf_YEARPURINCM_DKXE_multi', 'HDZYF_USAGE_RATE_min', 'numsOf_TOP_USER1_USAGE_AMOUNT_add', 'GRZHYE_HDSYYF_max', 'DWJJLX_GRZHDNGJYE_mean', 'numsOf_USAGE_RATE_TOP_USER_div', 'numsOf_YEARPURINCM_HDZYF_div', 'GRZHYE_HDSYYFZE_max', 'USAGE_RATE_HDYF_std', 'TOP_USER_GJJRATE_count', 'XINGBIE_GJJRATE_count', 'DWSSHY_YEARPURINCM_max', 'DKYE_GRZHYE_std', 'USAGE_RATE_GRYJCE_std', 'GRZHYE_WITHDRAW_min', 'DKXE_WITHDRAW_SUBMIT_sum', 'DWSSHY_DKLL_mean', 'GRZHDNGJYE_GRYJCE_max', 'WITHDRAW_USAGE_RATE_std', 'DWSSHY_WITHDRAW_SUBMIT_max', 'GRZHYE_WITHDRAW_SUBMIT_min', 'GRZHSNJZYE_USAGE_AMOUNT_max', 'USAGE_RATE_HDSYYFZE_min', 'HDSYYFZE_WITHDRAW_max', 'numsOf_GRZHDNGJYE_HDZYF_div', 'GRZHSNJZYE_HDYF_max', 'DWSSHY_GRYJCE_min', 'numsOf_GRZHYE_GRYJCE_multi', 'USAGE_AMOUNT_WITHDRAW_SUBMIT_max', 'DKFFE_TOP_USER1_sum', 'GRZHYE_HDSYYFZE_min', 'WITHDRAW_SUBMIT_HDSYYFZE_mean', 'GRYJCE_DWJJLX_min', 'GRZHDNGJYE_GRZHSNJZYE_mean', 'DWSSHY_HDZYFZE_min', 'DKFFE_DKLL_mean', 'numsOf_HDZYFZE_WITHDRAW_multi', 'DKFFE_HDYF_mean', 'HDZYFZE_WITHDRAW_SUBMIT_min', 'USAGE_RATE_WITHDRAW_max', 'WITHDRAW_HDSYYF_std', 'WITHDRAW_USAGE_RATE_max', 'DKFFE_USAGE_RATE_mean', 'HDZYF_HDZYFZE_min', 'numsOf_TOP_USER1_WITHDRAW_SUBMIT_multi', 'HDZYF_HDZYFZE_mean', 'GRZHDNGJYE_DKXE_std', 'numsOf_GJJRATE_WITHDRAW_SUBMIT_multi', 'GRZHDNGJYE_HDSYYF_mean', 'GRZHDNGJYE_USAGE_RATE_std', 'HDSYYFZE_WITHDRAW_SUBMIT_max', 'DWSSHY_HDZYF_min', 'GRZHDNGJYE_HDSYYF_std', 'WITHDRAW_SUBMIT_HDSYYFZE_max', 'WITHDRAW_SUBMIT_GRZHYE_mean', 'DWJJLX_2', 'HDSYYF_WITHDRAW_max', 'GRZHDNGJYE_WITHDRAW_min', 'TOP_USER1_YEARPURINCM_mean', 'DKXE_HDSYYFZE_std', 'GRZHDNGJYE_USAGE_AMOUNT_std', 'HDSYYFZE_GRZHSNJZYE_std', 'DWSSHY_HDZYFZE_std', 'GRZHYE_HDSYYFZE_mean', 'HDZYFZE_DKXE_max', 'GRZHYE_USAGE_RATE_min', 'numsOf_TOP_USER1_USAGE_AMOUNT_diff', 'HDZYFZE_WITHDRAW_SUBMIT_max', 'TOP_USER1_GRZHDNGJYE_mean', 'DKFFE_HDZYFZE_std', 'HDSYYFZE_GRZHYE_std', 'USAGE_RATE_HDSYYF_std', 'DKYE_HDYF_std', 'USAGE_RATE_GRZHSNJZYE_max', 'DWSSHY_WITHDRAW_SUBMIT_mean', 'GRZHDNGJYE_DKYE_std', 'HDSYYF_DKXE_max', 'HDSYYFZE_HDYF_std', 'HDZYFZE_WITHDRAW_mean', 'TOP_USER1_USAGE_AMOUNT_mean', 'USAGE_AMOUNT_WITHDRAW_max', 'USAGE_RATE_WITHDRAW_sum', 'DKFFE_USAGE_AMOUNT_mean', 'HDZYFZE_HDSYYF_std', 'GRZHDNGJYE_USAGE_AMOUNT_max', 'GRZHYE_WITHDRAW_SUBMIT_mean', 'TOP_USER1_GRZHSNJZYE_min', 'DKYE_HDZYF_std', 'DKYE_DKXE_std', 'TOP_USER1_USAGE_RATE_std', 'DWSSHY_HDSYYF_max', 'HDZYF_DKFFE_std', 'numsOf_DKYE_HDSYYF_div', 'GRZHDNGJYE_GRZHYE_std', 'HDZYFZE_HDZYF_std', 'HDZYFZE_WITHDRAW_min', 'DWSSHY_TOP_USER_mean', 'HDZYFZE_GRYJCE_mean', 'GRZHSNJZYE_USAGE_RATE_min', 'DKYE_WITHDRAW_SUBMIT_std', 'GRYJCE_TOP_USER_sum', 'HDZYFZE_DKXE_min', 'HDSYYF_GRZHDNGJYE_min', 'GRZHDNGJYE_WITHDRAW_SUBMIT_min', 'DKFFE_DKXE_std', 'USAGE_RATE_HDSYYFZE_max', 'HDZYF_HDYF_std', 'GRZHDNGJYE_DKLL_std', 'TOP_USER1_GJJRATE_std', 'numsOf_GJJRATE_WITHDRAW_multi', 'GRZHDNGJYE_TOP_USER_std', 'DWSSHY_YEARPURINCM_mean', 'DKYE_HDSYYF_std', 'USAGE_AMOUNT_HDSYYF_std', 'DKFFE_GRZHYE_mean', 'numsOf_GRZHDNGJYE_TOP_USER1_multi', 'HDSYYFZE_GJJRATE_std', 'GRJCJS_DWJJLX_min', 'DWSSHY_WITHDRAW_max', 'DWSSHY_YEARPURINCM_std', 'HDZYF_TOP_USER1_sum', 'WITHDRAW_HDSYYFZE_min', 'GRYJCE_GJJRATE_max', 'numsOf_USAGE_RATE_GJJRATE_div', 'USAGE_RATE_WITHDRAW_SUBMIT_sum', 'HDSYYF_HDYF_min', 'HDZYFZE_YEARPURINCM_max', 'DWSSHY_HDSYYFZE_std', 'DWSSHY_WITHDRAW_std', 'numsOf_TOP_USER1_USAGE_AMOUNT_multi', 'DKFFE_GRJCJS_std', 'USAGE_AMOUNT_TOP_USER_std', 'HDZYF_TOP_USER_std', 'GRZHDNGJYE_USAGE_AMOUNT_min', 'numsOf_YEARPURINCM_USAGE_RATE_div', 'TOP_USER1_HDYF_mean', 'numsOf_HDSYYFZE_TOP_USER1_div', 'WITHDRAW_YEARPURINCM_std', 'DKYE_GRJCJS_std', 'GRZHDNGJYE_HDSYYFZE_min', 'GRZHSNJZYE_DKXE_max', 'TOP_USER1_GJJRATE_mean', 'DWJJLX_USAGEOVER_count', 'GRZHYE_HDZYFZE_min', 'HDSYYF_GRZHSNJZYE_max', 'GRZHDNGJYE_GRZHYE_max', 'USAGE_AMOUNT_WITHDRAW_SUBMIT_sum', 'numsOf_TOP_USER1_WITHDRAW_multi', 'HDZYF_GJJRATE_sum', 'HDZYF_GJJRATE_std', 'HDZYF_GRJCJS_std', 'DWJJLX_GRJCJS_std', 'GRZHSNJZYE_WITHDRAW_max', 'DWSSHY_USAGEOVER_count', 'YEARPURINCM_USAGE_RATE_sum', 'numsOf_GRJCJS_HDYF_multi', 'HDZYFZE_GRZHDNGJYE_max', 'WITHDRAW_SUBMIT_USAGE_RATE_max', 'GRZHDNGJYE_USAGE_RATE_min', 'WITHDRAW_WITHDRAW_SUBMIT_std', 'DKFFE_YEARPURINCM_std', 'HDSYYFZE_WITHDRAW_std', 'HDZYFZE_USAGE_AMOUNT_sum', 'GRZHYE_WITHDRAW_max', 'GRZHSNJZYE_YEARPURINCM_max', 'WITHDRAW_WITHDRAW_SUBMIT_sum', 'HDZYFZE_HDYF_max', 'YEARPURINCM_DKXE_sum', 'TOP_USER_in_GJJRATE_prop', 'GRZHSNJZYE_HDSYYFZE_max', 'TOP_USER1_WITHDRAW_SUBMIT_std', 'WITHDRAW_DKLL_std', 'HDSYYF_GRZHSNJZYE_min', 'HDZYF_HDSYYFZE_std', 'HDZYFZE_DKLL_std', 'DKXE_GJJRATE_min', 'DWSSHY_WITHDRAW_SUBMIT_min', 'WITHDRAW_SUBMIT_USAGE_RATE_min', 'DKFFE_WITHDRAW_SUBMIT_mean', 'WITHDRAW_GJJRATE_mean', 'HDZYFZE_USAGE_AMOUNT_mean', 'DKFFE_YEARPURINCM_sum', 'HDSYYFZE_GRZHDNGJYE_std', 'GRZHYE_YEARPURINCM_min', 'WITHDRAW_SUBMIT_USAGE_RATE_mean', 'YEARPURINCM_GRYJCE_sum', 'HDZYFZE_GRZHSNJZYE_min', 'WITHDRAW_SUBMIT_USAGE_AMOUNT_min', 'GRZHDNGJYE_HDZYFZE_std', 'DKFFE_DKXE_max', 'XINGBIE_GJSIGN_count', 'GRZHYE_HDZYF_std', 'GRZHDNGJYE_HDZYFZE_max', 'ZHIYE_in_DWJJLX_prop', 'numsOf_HDYF_USAGE_RATE_div', 'HDSYYF_GRYJCE_max', 'GRYJCE_GJJRATE_sum', 'GRZHYE_HDZYFZE_mean', 'HDSYYF_HDSYYFZE_sum', 'USAGE_RATE_WITHDRAW_SUBMIT_max', 'GRYJCE_GJJRATE_min', 'TOP_USER1_DKLL_mean', 'numsOf_DKLL_USAGE_RATE_div', 'HDZYFZE_HDZYF_min', 'GRZHDNGJYE_HDZYFZE_min', 'DKFFE_HDSYYF_mean', 'HDSYYFZE_GRJCJS_std', 'GRZHDNGJYE_DKLL_mean', 'USAGE_RATE_HDZYFZE_max', 'DWJJLX_YEARPURINCM_sum', 'DKFFE_GRZHDNGJYE_std', 'numsOf_TOP_USER1_USAGE_AMOUNT_div', 'HDYF_YEARPURINCM_min', 'WITHDRAW_HDZYFZE_max', 'DWSSHY_HDZYFZE_mean', 'HDZYF_GRYJCE_std', 'DWSSHY_USAGE_AMOUNT_mean', 'GRZHDNGJYE_TOP_USER1_mean', 'USAGE_AMOUNT_HDSYYFZE_min', 'HDSYYFZE_GRYJCE_std', 'HDSYYFZE_HDZYF_std', 'HDSYYFZE_YEARPURINCM_std', 'HDSYYF_YEARPURINCM_mean', 'GRZHDNGJYE_GRJCJS_std', 'numsOf_GRZHDNGJYE_USAGE_RATE_div', 'GRZHDNGJYE_GRYJCE_std', 'GRZHSNJZYE_HDSYYF_max', 'GRZHYE_WITHDRAW_SUBMIT_max', 'DKXE_TOP_USER_sum', 'WITHDRAW_SUBMIT_USAGE_AMOUNT_max', 'DKFFE_GRYJCE_std', 'GJSIGN_in_TOP_USER_prop', 'TOP_USER1_HDSYYF_mean', 'WITHDRAW_HDYF_std', 'HDZYFZE_GRZHSNJZYE_max', 'GRZHDNGJYE_GRZHSNJZYE_std', 'WITHDRAW_SUBMIT_HDZYFZE_mean', 'DKFFE_GRZHSNJZYE_std', 'WITHDRAW_DKFFE_std', 'USAGE_RATE_HDZYFZE_min', 'numsOf_HDSYYFZE_USAGE_RATE_div', 'DKFFE_WITHDRAW_min', 'WITHDRAW_GRJCJS_std', 'XINGBIE_WITHSIGN_count', 'DKFFE_DKXE_mean', 'numsOf_GJJRATE_TOP_USER1_multi', 'DKFFE_WITHDRAW_SUBMIT_max', 'HDZYFZE_HDSYYFZE_sum', 'GRZHDNGJYE_WITHDRAW_SUBMIT_std', 'TOP_USER1_WITHDRAW_SUBMIT_mean', 'HDSYYFZE_HDSYYF_std', 'WITHDRAW_TOP_USER1_std', 'GRZHSNJZYE_GRZHYE_max', 'DKFFE_GRZHSNJZYE_mean', 'numsOf_GRZHDNGJYE_GRYJCE_multi', 'HDZYF_GJJRATE_mean', 'USAGE_RATE_DKYE_std', 'TOP_USER_GJSIGN_count', 'WITHDRAW_HDZYFZE_min', 'USAGE_RATE_USAGE_AMOUNT_max', 'USAGE_RATE_DKLL_std', 'ZHICHEN_in_DWJJLX_prop', 'GRZHSNJZYE_HDYF_std', 'DKFFE_YEARPURINCM_min', 'DKXE_TOP_USER1_min', 'GRZHSNJZYE_DKXE_std', 'HDZYF_TOP_USER_sum', 'GRJCJS_GJJRATE_min', 'USAGE_RATE_HDZYFZE_mean', 'DKFFE_HDZYF_min', 'DWSSHY_DKXE_min', 'HDSYYFZE_WITHDRAW_SUBMIT_std', 'numsOf_HDSYYF_WITHDRAW_SUBMIT_diff', 'GRZHSNJZYE_GRZHDNGJYE_min', 'numsOf_WITHDRAW_SUBMIT_DKXE_diff', 'numsOf_HDSYYF_HDSYYFZE_div', 'GRZHDNGJYE_TOP_USER1_sum', 'DKFFE_WITHDRAW_std', 'WITHDRAW_HDZYF_std', 'GRJCJS_TOP_USER_max', 'HDSYYF_YEARPURINCM_min', 'DKXE_HDZYFZE_max', 'GRZHZT_in_GJSIGN_prop', 'DWSSHY_HDSYYF_sum', 'DWJJLX_YEARPURINCM_min', 'HDZYFZE_YEARPURINCM_min', 'GRZHSNJZYE_HDSYYFZE_min', 'DKFFE_GRYJCE_mean', 'GRZHSNJZYE_GRZHDNGJYE_max', 'GRZHDNGJYE_TOP_USER_mean', 'GRZHSNJZYE_YEARPURINCM_min', 'GRYJCE_TOP_USER_max', 'YEARPURINCM_HDSYYFZE_sum', 'numsOf_HDZYF_HDSYYF_multi', 'GRZHYE_HDZYFZE_max', 'DKFFE_DKYE_std', 'WITHDRAW_TOP_USER_mean', 'DKYE_DKLL_max', 'GJJRATE_GJSIGN_count', 'GRZHSNJZYE_DKFFE_std', 'HDSYYF_GRZHYE_max', 'HDYF_USAGE_AMOUNT_sum', 'GRZHDNGJYE_TOP_USER1_max', 'XINGBIE_WITHSIGN2_count', 'DWJJLX_DKLL_std', 'HDZYFZE_USAGE_AMOUNT_max', 'WITHDRAW_DKXE_std', 'HDSYYF_WITHDRAW_SUBMIT_min', 'GRZHYE_TOP_USER1_max', 'DWSSHY_USAGE_AMOUNT_std', 'WITHDRAW_DKYE_std', 'DWJJLX_HDYF_std', 'YEARPURINCM_HDSYYF_sum', 'DKFFE_WITHDRAW_mean', 'DWSSHY_WITHDRAW_SUBMIT_std', 'numsOf_HDSYYFZE_USAGE_AMOUNT_div', 'HDZYFZE_GRYJCE_max', 'WITHDRAW_DKLL_mean', 'WITHDRAW_SUBMIT_HDSYYFZE_min', 'DWJJLX_TOP_USER1_mean', 'WITHDRAW_SUBMIT_HDZYFZE_max', 'DKFFE_HDYF_max', 'DWSSHY_10', 'HDSYYF_YEARPURINCM_max', 'HDZYF_TOP_USER1_max', 'USAGE_AMOUNT_HDZYFZE_min', 'DKFFE_HDZYF_std', 'HDSYYF_GRZHDNGJYE_max', 'HDSYYF_WITHDRAW_SUBMIT_max', 'TOP_USER1_HDZYF_std', 'DWSSHY_GRZHSNJZYE_max', 'USAGE_RATE_TOP_USER1_max', 'HDZYF_HDZYFZE_max', 'GRZHSNJZYE_WITHDRAW_SUBMIT_mean', 'WITHDRAW_WITHDRAW_SUBMIT_min', 'DWJJLX_HDSYYFZE_max', 'GRZHYE_YEARPURINCM_std', 'WITHDRAW_WITHDRAW_SUBMIT_max', 'TOP_USER_WITHSIGN2_count', 'HDZYFZE_GRYJCE_min', 'numsOf_USAGE_RATE_USAGE_AMOUNT_div', 'USAGE_AMOUNT_USAGE_RATE_std', 'TOP_USER1_DKXE_mean', 'GRZHDNGJYE_WITHDRAW_SUBMIT_max', 'TOP_USER1_DKXE_min', 'TOP_USER1_WITHDRAW_max', 'HDZYFZE_TOP_USER_sum', 'WITHDRAW_GRZHSNJZYE_std', 'HDYF_DKXE_max', 'DKFFE_HDSYYF_std', 'GRZHSNJZYE_HDSYYF_std', 'DWSSHY_16', 'DWSSHY_GJJRATE_std', 'DWSSHY_GRZHYE_max', 'USAGE_AMOUNT_DKFFE_std', 'WITHDRAW_USAGE_AMOUNT_std', 'numsOf_GJJRATE_TOP_USER1_div', 'HDZYFZE_WITHDRAW_max', 'numsOf_DKLL_HDZYFZE_div', 'WITHDRAW_SUBMIT_YEARPURINCM_std', 'numsOf_HDYF_USAGE_AMOUNT_div', 'DWSSHY_DKXE_sum', 'TOP_USER1_GRJCJS_std', 'USAGE_RATE_DKLL_max', 'GRZHDNGJYE_WITHDRAW_std', 'HDYF_WITHDRAW_min', 'GRZHYE_TOP_USER1_min', 'TOP_USER1_HDSYYF_std', 'DWJJLX_GRYJCE_min', 'GRZHDNGJYE_TOP_USER_sum', 'GRZHSNJZYE_WITHDRAW_min', 'USAGE_AMOUNT_HDZYFZE_max', 'GRZHYE_DKXE_std', 'USAGEOVER_GJJRATE_count', 'GRZHSNJZYE_HDZYFZE_max', 'DKFFE_WITHDRAW_SUBMIT_min', 'USAGE_RATE_GJJRATE_max', 'WITHSIGN_in_GJJRATE_prop', 'DWJJLX_DKYE_std', 'numsOf_GRJCJS_GRYJCE_multi', 'XINGBIE_USAGEOVER_count', 'HDZYFZE_DKXE_sum', 'TOP_USER_in_GJSIGN_prop', 'WITHDRAW_SUBMIT_TOP_USER_mean', 'HDYF_GRZHDNGJYE_min', 'WITHDRAW_SUBMIT_TOP_USER_std', 'DWJJLX_HDZYFZE_mean', 'DWJJLX_WITHDRAW_std', 'GRZHDNGJYE_GJJRATE_sum', 'numsOf_GJJRATE_TOP_USER_div', 'GRJCJS_GJJRATE_max', 'HDYF_GRZHSNJZYE_max', 'DWJJLX_YEARPURINCM_mean', 'YEARPURINCM_HDYF_sum', 'HDYF_GRYJCE_max', 'WITHDRAW_TOP_USER1_max', 'WITHSIGN2_in_GJJRATE_prop', 'XINGBIE_NOUSAGE_count', 'WITHDRAW_SUBMIT_HDZYFZE_min', 'GRZHYE_DKYE_std', 'DKFFE_DKXE_sum', 'TOP_USER1_YEARPURINCM_std', 'DKFFE_GRJCJS_min', 'TOP_USER1_HDSYYFZE_mean', 'YEARPURINCM_GRZHYE_sum', 'GRZHYE_HDSYYF_std', 'GRZHDNGJYE_GJJRATE_mean', 'HDZYF_TOP_USER_max', 'YEARPURINCM_WITHDRAW_SUBMIT_sum', 'GRZHDNGJYE_HDSYYFZE_std', 'DWJJLX_GRJCJS_min', 'GRZHYE_DKLL_std', 'GRJCJS_TOP_USER_min', 'WITHDRAW_GJJRATE_std', 'DKXE_TOP_USER_max', 'DWJJLX_TOP_USER1_max', 'GRJCJS_TOP_USER1_min', 'numsOf_GJJRATE_TOP_USER_multi', 'WITHDRAW_SUBMIT_DKFFE_std', 'USAGE_AMOUNT_GJJRATE_max', 'GRZHYE_USAGE_RATE_std', 'USAGE_RATE_DKFFE_std', 'HDZYF_TOP_USER_mean', 'DWSSHY_USAGE_AMOUNT_sum', 'HDZYF_DKLL_std', 'GRZHYE_GJJRATE_std', 'HDYF_WITHDRAW_max', 'TOP_USER1_WITHDRAW_mean', 'DWSSHY_WITHDRAW_sum', 'TOP_USER1_USAGE_AMOUNT_min', 'WITHSIGN_GJJRATE_count', 'TOP_USER1_HDZYF_min', 'GRYJCE_TOP_USER1_min', 'DWJJLX_WITHDRAW_min', 'HDYF_GRZHYE_max', 'TOP_USER1_GRZHYE_mean', 'GRZHYE_TOP_USER_std', 'GRZHSNJZYE_HDZYFZE_std', 'TOP_USER1_GRYJCE_max', 'WITHDRAW_SUBMIT_WITHDRAW_sum', 'DKFFE_USAGE_AMOUNT_min', 'DWJJLX_GJJRATE_std', 'HDZYFZE_HDSYYFZE_std', 'WITHDRAW_GRZHDNGJYE_std', 'GRZHYE_USAGE_AMOUNT_std', 'YEARPURINCM_GRZHSNJZYE_sum', 'HDZYFZE_GRZHYE_min', 'TOP_USER1_GRZHYE_min', 'TOP_USER1_GRZHYE_std', 'GRZHYE_TOP_USER_mean', 'DKFFE_USAGE_AMOUNT_sum', 'WITHDRAW_GRZHYE_std', 'TOP_USER1_GRZHDNGJYE_std', 'HDYF_YEARPURINCM_max', 'DWJJLX_WITHDRAW_SUBMIT_min', 'USAGE_AMOUNT_TOP_USER1_max', 'TOP_USER1_USAGE_AMOUNT_std', 'DKFFE_DWSSHY_max', 'HDSYYFZE_HDZYFZE_max', 'GRZHSNJZYE_TOP_USER1_max', 'GRZHSNJZYE_HDZYFZE_min', 'WITHDRAW_SUBMIT_DKLL_mean', 'GJJRATE_WITHSIGN2_count', 'HDYF_WITHDRAW_SUBMIT_mean', 'GRZHYE_HDYF_std', 'TOP_USER1_HDZYFZE_std', 'GJJRATE_DKXE_min', 'GRZHYE_GRZHSNJZYE_std', 'USAGE_RATE_USAGE_AMOUNT_std', 'HDSYYF_WITHDRAW_min', 'GJJRATE_in_WITHSIGN2_prop', 'GJJRATE_GRZHSNJZYE_max', 'GRZHYE_DKFFE_std', 'numsOf_HDZYFZE_TOP_USER1_add', 'TOP_USER1_WITHDRAW_std', 'TOP_USER1_USAGE_AMOUNT_max', 'GRZHYE_GRJCJS_std', 'DWJJLX_HDYF_max', 'WITHDRAW_SUBMIT_WITHDRAW_std', 'USAGE_RATE_TOP_USER_std', 'TOP_USER1_DKYE_std', 'HDYF_GRZHDNGJYE_max', 'GRZHSNJZYE_USAGE_RATE_std', 'numsOf_DKLL_TOP_USER1_div', 'GRZHDNGJYE_TOP_USER1_min', 'HDZYF_GJJRATE_max', 'DKFFE_HDSYYFZE_sum', 'HDYF_WITHDRAW_SUBMIT_max', 'WITHDRAW_SUBMIT_GRZHDNGJYE_max', 'DKLL_GRJCJS_std', 'TOP_USER1_GRYJCE_std', 'HDZYFZE_HDSYYFZE_min', 'GRZHYE_GRYJCE_std', 'numsOf_GJJRATE_TOP_USER_add', 'HDZYFZE_GJJRATE_max', 'numsOf_USAGE_AMOUNT_WITHDRAW_SUBMIT_diff', 'WITHDRAW_HDZYFZE_std', 'GRYJCE_TOP_USER_min', 'GRZHZT_ZHICHEN_count', 'DWSSHY_NOUSAGE_count', 'TOP_USER1_USAGE_RATE_max', 'GRZHYE_TOP_USER1_std', 'HDYF_YEARPURINCM_std', 'GJJRATE_HDYF_max', 'TOP_USER1_GRZHSNJZYE_mean', 'HDSYYF_USAGE_RATE_sum', 'DWJJLX', 'DKXE_TOP_USER_min', 'numsOf_DKFFE_HDZYF_div', 'HDYF_HDSYYF_max', 'WITHDRAW_SUBMIT_TOP_USER1_max', 'DKFFE_USAGE_RATE_max', 'DKFFE_USAGE_AMOUNT_max', 'HDYF_USAGE_AMOUNT_std', 'GRZHSNJZYE_DKYE_std', 'WITHDRAW_TOP_USER_max', 'DWJJLX_DKXE_max', 'TOP_USER1_DKXE_std', 'TOP_USER1_HDYF_min', 'YEARPURINCM_GJJRATE_std', 'WITHDRAW_HDSYYFZE_std', 'GRZHYE_DKLL_min', 'GRZHYE_HDSYYFZE_std', 'GRZHYE_GJJRATE_mean', 'USAGE_RATE_TOP_USER1_std', 'YEARPURINCM_HDZYFZE_sum', 'GRZHSNJZYE_TOP_USER1_std', 'TOP_USER1_GRZHSNJZYE_std', 'GJJRATE_DKXE_mean', 'GRYJCE_DKLL_min', 'DKFFE_GRYJCE_min', 'GJJRATE_TOP_USER_std', 'GJJRATE_DKLL_mean', 'HDYF_GRZHSNJZYE_std', 'DKXE_GJJRATE_max', 'GRZHDNGJYE_GJJRATE_max', 'GJJRATE_HDZYFZE_min', 'WITHDRAW_SUBMIT_TOP_USER1_mean', 'GRZHYE_WITHDRAW_std', 'DKLL_GRZHSNJZYE_mean', 'DKYE_DKLL_std', 'HDZYF_HDZYFZE_std', 'GRZHYE_WITHDRAW_SUBMIT_std', 'WITHDRAW_SUBMIT_HDYF_std', 'HDZYFZE_HDSYYF_min', 'GRZHSNJZYE_GRZHDNGJYE_std', 'DWJJLX_YEARPURINCM_max', 'GRZHDNGJYE_GJJRATE_min', 'WITHDRAW_SUBMIT_GJJRATE_max', 'DWJJLX_HDZYFZE_std', 'USAGE_AMOUNT_TOP_USER1_std', 'USAGEOVER_TOP_USER_count', 'WITHDRAW_SUBMIT_GJJRATE_mean', 'GRZHYE_GJJRATE_max', 'numsOf_GJJRATE_TOP_USER_diff', 'DWJJLX_HDYF_mean', 'WITHDRAW_SUBMIT_USAGE_AMOUNT_std', 'DWJJLX_GRYJCE_std', 'WITHDRAW_SUBMIT_USAGE_RATE_std', 'WITHDRAW_TOP_USER_min', 'GRJCJS_DKLL_min', 'GRZHDNGJYE_TOP_USER_max', 'DKFFE_GJJRATE_max', 'USAGE_RATE_HDZYFZE_std', 'USAGE_RATE_TOP_USER_mean', 'numsOf_GRYJCE_USAGE_AMOUNT_div', 'HDZYF_GJJRATE_min', 'GRZHYE_TOP_USER_max', 'TOP_USER1_HDSYYFZE_min', 'GRZHYE_GRZHDNGJYE_std', 'WITHDRAW_SUBMIT_GRZHSNJZYE_std', 'TOP_USER_WITHDRAW_SUBMIT_max', 'numsOf_WITHDRAW_WITHDRAW_SUBMIT_diff', 'GJJRATE_3', 'DKFFE_HDSYYFZE_std', 'WITHDRAW_SUBMIT_TOP_USER1_std', 'numsOf_GRZHDNGJYE_WITHDRAW_SUBMIT_diff', 'GJJRATE_HDSYYFZE_max', 'DKXE_DKLL_min', 'TOP_USER1_HDSYYFZE_std', 'HDYF_HDZYFZE_min', 'HDZYFZE_USAGE_RATE_min', 'WITHDRAW_SUBMIT_HDSYYF_std', 'USAGE_AMOUNT_DKLL_std', 'USAGE_AMOUNT_DKYE_std', 'WITHDRAW_SUBMIT_HDZYFZE_std', 'WITHDRAW_SUBMIT_TOP_USER_max', 'USAGE_RATE_HDSYYFZE_std', 'GRYJCE_DKLL_max', 'GJJRATE_HDZYF_min', 'GRJCJS_DKLL_max', 'DWSSHY_GJJRATE_sum', 'GRZHDNGJYE_TOP_USER_min', 'GRZHSNJZYE_TOP_USER_std', 'GRZHSNJZYE_HDZYF_std', 'GJJRATE_HDZYFZE_std', 'GJJRATE_2', 'GRZHYE_DKLL_max', 'WITHDRAW_SUBMIT_DKYE_std', 'GRZHDNGJYE_DKLL_min', 'YEARPURINCM_TOP_USER1_sum', 'DKFFE_DWSSHY_min', 'YEARPURINCM_DKXE_std', 'DKYE_HDSYYFZE_std', 'WITHDRAW_SUBMIT_TOP_USER_min', 'GRZHSNJZYE_GRZHYE_std', 'TOP_USER1_USAGE_RATE_min', 'GRZHSNJZYE_DKLL_std', 'WITHSIGN_in_WITHSIGN2_prop', 'DWSSHY_0', 'YEARPURINCM_HDYF_std', 'HDSYYFZE_TOP_USER1_max', 'numsOf_DKFFE_HDZYFZE_div', 'DWSSHY_TOP_USER1_sum', 'HDYF_HDSYYFZE_min', 'WITHDRAW_SUBMIT_HDZYF_std', 'DWJJLX_DKXE_mean', 'USAGE_AMOUNT_TOP_USER_max', 'DKXE_DKLL_max', 'GJJRATE_DKLL_std', 'WITHDRAW_SUBMIT_DKLL_std', 'DWJJLX_3', 'TOP_USER_GRZHYE_mean', 'DWJJLX_TOP_USER1_std', 'TOP_USER_GRZHYE_std', 'YEARPURINCM_HDSYYF_std', 'YEARPURINCM_GJJRATE_sum', 'GJSIGN_in_WITHSIGN_prop', 'WITHDRAW_SUBMIT_GJJRATE_std', 'NOUSAGE_TOP_USER_count', 'numsOf_GRZHSNJZYE_WITHDRAW_SUBMIT_diff', 'numsOf_HDSYYFZE_WITHDRAW_SUBMIT_diff', 'TOP_USER1_GRYJCE_min', 'GJJRATE_GRYJCE_min', 'TOP_USER1_DKLL_min', 'GJJRATE_DKXE_max', 'DKFFE_DKLL_max', 'USAGE_AMOUNT_GJJRATE_min', 'numsOf_HDZYFZE_TOP_USER_multi', 'TOP_USER1_HDSYYF_min', 'GRZHSNJZYE_DKLL_min', 'HDYF_GRZHDNGJYE_std', 'HDYF_GJJRATE_max', 'GRZHSNJZYE_USAGE_AMOUNT_std', 'DKLL_GRZHYE_std', 'DKLL_DKXE_max', 'DKYE_HDZYFZE_std', 'YEARPURINCM_USAGE_RATE_std', 'USAGE_RATE_DKLL_min', 'GRZHDNGJYE_DKLL_max', 'GRZHSNJZYE_HDSYYFZE_std', 'GRZHSNJZYE_GJJRATE_max', 'HDYF_GRZHYE_std', 'DWJJLX_USAGE_RATE_std', 'age_genFeat5', 'DWJJLX_TOP_USER_mean', 'WITHDRAW_DKLL_max', 'DWJJLX_GJJRATE_max', 'age_genFeat6', 'DWSSHY_1', 'WITHDRAW_SUBMIT_TOP_USER1_min', 'GRZHYE_TOP_USER_min', 'GRZHYE_GJJRATE_min', 'WITHDRAW_SUBMIT_GRZHYE_std', 'GJJRATE_GRJCJS_min', 'GJJRATE_WITHDRAW_SUBMIT_min', 'DWJJLX_HDZYF_min', 'DKFFE_DWJJLX_max', 'DWSSHY_USAGE_AMOUNT_max', 'YEARPURINCM_TOP_USER_sum', 'XINGBIE_USAGE_AMOUNT_mean', 'YEARPURINCM_USAGE_AMOUNT_std', 'GJJRATE_5', 'WITHDRAW_SUBMIT_GRJCJS_std', 'HDYF_WITHDRAW_SUBMIT_std', 'GRZHSNJZYE_TOP_USER_max', 'GJJRATE_HDSYYFZE_mean', 'DWSSHY_7', 'WITHDRAW_SUBMIT_DKXE_std', 'WITHDRAW_DKLL_min', 'DKYE_DKLL_min', 'WITHSIGN_GJSIGN_count', 'DWSSHY_5', 'DWJJLX_DKXE_min', 'GRZHSNJZYE_GJJRATE_std', 'TOP_USER_DKXE_std', 'age_genFeat3', 'GRZHZT_DKXE_max', 'TOP_USER_GRZHYE_max', 'XINGBIE_WITHDRAW_min', 'TOP_USER_GRZHDNGJYE_std', 'TOP_USER_DKLL_mean', 'TOP_USER_DKLL_std', 'GRZHSNJZYE_DKLL_max', 'TOP_USER_HDZYF_std', 'GRZHSNJZYE_GRYJCE_std', 'DWJJLX_USAGE_AMOUNT_sum', 'DWJJLX_WITHDRAW_SUBMIT_max', 'DKLL_TOP_USER_mean', 'YEARPURINCM_GRZHYE_std', 'TOP_USER_HDZYF_max', 'GJJRATE_TOP_USER1_max', 'WITHDRAW_GRYJCE_std', 'HDYF_DKXE_std', 'USAGE_AMOUNT_TOP_USER_min', 'GRZHSNJZYE_WITHDRAW_SUBMIT_std', 'GRZHSNJZYE_GRJCJS_std', 'HDZYFZE_USAGE_AMOUNT_std', 'WITHDRAW_GJJRATE_max', 'GRZHYE_HDZYFZE_std', 'DWJJLX_HDZYFZE_min', 'DKFFE_USAGE_AMOUNT_std', 'numsOf_HDZYFZE_TOP_USER1_div', 'YEARPURINCM_DKFFE_std', 'HDYF_HDZYF_std', 'TOP_USER_GRZHDNGJYE_max', 'TOP_USER_YEARPURINCM_min', 'TOP_USER_GJJRATE_mean', 'DKYE_TOP_USER_max', 'GRZHSNJZYE_YEARPURINCM_std', 'DKLL_GRZHDNGJYE_mean', 'WITHDRAW_SUBMIT_DKLL_max', 'HDSYYF_GRZHSNJZYE_std', 'DWSSHY_11', 'HDZYF_TOP_USER_min', 'GJJRATE_4', 'WITHDRAW_SUBMIT_GJJRATE_min', 'WITHDRAW_SUBMIT_HDSYYFZE_std', 'DWSSHY_3', 'USAGE_AMOUNT_HDZYFZE_std', 'DWSSHY_4', 'TOP_USER1_DKLL_max', 'HDSYYF_YEARPURINCM_std', 'GJJRATE_GJJRATE_sum', 'TOP_USER_HDYF_std', 'TOP_USER_HDYF_mean', 'HDYF_GRJCJS_std', 'HDSYYFZE_DKLL_std', 'TOP_USER_USAGE_RATE_std', 'age_genFeat4', 'TOP_USER1_GJJRATE_max', 'HDYF_GJJRATE_std', 'HDYF_WITHDRAW_std', 'DWSSHY_8', 'TOP_USER_GRJCJS_max', 'TOP_USER_GRJCJS_min', 'HDSYYF_GRZHYE_std', 'DWJJLX_GRZHDNGJYE_max', 'DKYE_TOP_USER1_std', 'DWJJLX_GRZHYE_max', 'WITHDRAW_SUBMIT_GRZHDNGJYE_std', 'TOP_USER_GRJCJS_mean', 'DWSSHY_15', 'DKLL_GRZHDNGJYE_std', 'USAGE_RATE_TOP_USER_min', 'HDYF_DKLL_min', 'DKYE_USAGE_AMOUNT_std', 'USAGEOVER_WITHSIGN2_count', 'WITHDRAW_GJJRATE_min', 'DKLL_DKXE_mean', 'TOP_USER_in_ZHICHEN_prop', 'USAGE_RATE_TOP_USER_max', 'DWJJLX_WITHDRAW_SUBMIT_mean', 'HDYF_HDZYFZE_std', 'XINGBIE_DKXE_min', 'ZHIYE_WITHDRAW_SUBMIT_std', 'YEARPURINCM_GRZHDNGJYE_std', 'ZHIYE_TOP_USER1_std', 'ZHIYE_DKXE_mean', 'ZHIYE_USAGE_RATE_std', 'WITHSIGN_DKXE_mean', 'WITHSIGN_DKXE_max', 'TOP_USER_DKXE_mean', 'TOP_USER_DKXE_sum', 'ZHIYE_WITHDRAW_SUBMIT_mean', 'TOP_USER_WITHDRAW_SUBMIT_std', 'WITHDRAW_SUBMIT_DKLL_min', 'ZHIYE_TOP_USER_max', 'DKLL_DKXE_min', 'ZHIYE_WITHDRAW_std', 'WITHDRAW_SUBMIT_GRYJCE_std', 'DWJJLX_DKXE_sum', 'ZHIYE_GJJRATE_mean', 'numsOf_YEARPURINCM_WITHDRAW_SUBMIT_diff', 'ZHIYE_TOP_USER1_mean', 'DWSSHY_GRZHDNGJYE_min', 'DWSSHY_GJJRATE_max', 'ZHIYE_TOP_USER_std', 'ZHIYE_TOP_USER_mean', 'TOP_USER_HDSYYFZE_std', 'TOP_USER_GJJRATE_std', 'HDSYYF_DKXE_std', 'ZHIYE_USAGE_RATE_mean', 'TOP_USER1_GJJRATE_min', 'ZHICHEN_DKFFE_genFeat7_mean', 'ZHICHEN_in_ZHIYE_prop', 'DWSSHY_9', 'DKFFE_DWJJLX_min', 'GJSIGN_in_USAGEOVER_prop', 'USAGEOVER_GJSIGN_count', 'ZHIYE_DKXE_std', 'DWSSHY_2', 'HDSYYF_GRJCJS_std', 'TOP_USER_6', 'DWSSHY_14', 'GRZHSNJZYE_WITHDRAW_std', 'HDZYFZE_GJJRATE_min', 'ZHIYE_DKXE_min', 'ZHIYE_WITHDRAW_mean', 'YEARPURINCM_DKLL_std', 'GJSIGN_DKXE_mean', 'TOP_USER_GRJCJS_std', 'ZHIYE_HDSYYFZE_max', 'ZHIYE_HDSYYFZE_std', 'TOP_USER_GRZHSNJZYE_max', 'ZHIYE_HDZYFZE_std', 'ZHIYE_HDSYYF_std', 'GJSIGN_DKXE_max', 
                                                                                                                                                    'GJSIGN_DKXE_min'] 
    #+ ['<400','HDZYFZE_DKXE_std', 'HDZYF_DKXE_sum', 'USAGE_AMOUNT_WITHDRAW_mean', 'GRJCJS_GRZHSNJZYE_min', 'GRYJCE_YEARPURINCM_mean', 'GRJCJS_WITHDRAW_mean', 'GRYJCE_DKXE_max', 'HDZYFZE_YEARPURINCM_std', 'DWJJLX_in_GJJRATE_prop', 'USAGE_AMOUNT_DKXE_min', 'DWSSHY_in_XINGBIE_prop', 'HDZYFZE_GRZHYE_std', 'HDZYFZE_USAGE_RATE_mean', 'numsOf_GRZHSNJZYE_USAGE_AMOUNT_multi', 'USAGE_AMOUNT_GRYJCE_mean', 'USAGE_AMOUNT_GRZHSNJZYE_std', 'DWSSHY_XINGBIE_count', 'GRYJCE_HDZYFZE_mean', 'HDZYFZE_YEARPURINCM_mean', 'DKXE_GRYJCE_min', 'DWSSHY_DKXE_max', 'GRYJCE_HDZYF_std', 'DKXE_DKLL_std', 'GRYJCE_YEARPURINCM_min', 'GRJCJS_GRZHDNGJYE_mean', 'numsOf_HDZYFZE_DKXE_multi', 'GRJCJS_USAGE_AMOUNT_mean', 'DWSSHY_in_WITHSIGN_prop', 'GRJCJS_TOP_USER1_mean', 'GRJCJS_DKXE_sum', 'USAGE_AMOUNT_HDYF_max', 'numsOf_GRZHDNGJYE_USAGE_AMOUNT_div', 'WITHDRAW_SUBMIT_DKXE_max', 'DWJJLX_WITHSIGN_count', 'USAGE_AMOUNT_WITHDRAW_std', 'USAGE_RATE_GJJRATE_std', 'GRZHYE_DKXE_sum', 'GRJCJS_USAGE_RATE_mean', 'numsOf_GRZHSNJZYE_YEARPURINCM_diff', 'HDZYFZE_DKYE_std', 'GRYJCE_USAGE_RATE_mean', 'GRYJCE_USAGE_AMOUNT_mean', 'numsOf_GRZHSNJZYE_WITHDRAW_SUBMIT_div', 'GRJCJS_GRZHYE_max', 'GRJCJS_DWSSHY_max', 'GRYJCE_HDSYYFZE_max', 'GRJCJS_HDYF_min', 'GRZHYE_GRZHSNJZYE_max', 'GRZHSNJZYE_DKXE_sum', 'DKXE_HDZYFZE_mean', 'GRYJCE_GRZHSNJZYE_min', 'numsOf_TOP_USER_DKXE_multi', 'numsOf_HDSYYF_DKXE_multi', 'GRJCJS_DKXE_max', 'HDZYFZE_WITHDRAW_std', 'GRJCJS_GRZHSNJZYE_max', 'WITHDRAW_SUBMIT_GRZHDNGJYE_min', 'USAGE_AMOUNT_WITHDRAW_SUBMIT_mean', 'HDZYFZE_USAGE_RATE_max', 'GRJCJS_HDSYYFZE_max', 'DKXE_GRZHSNJZYE_mean', 'HDZYFZE_GRZHDNGJYE_std', 'numsOf_TOP_USER1_DKXE_multi', 'numsOf_GRYJCE_USAGE_AMOUNT_multi', 'numsOf_HDYF_WITHDRAW_SUBMIT_div', 'GRYJCE_DKXE_mean', 'numsOf_HDYF_DKXE_multi', 'numsOf_HDSYYFZE_TOP_USER1_add', 'GRJCJS_YEARPURINCM_min', 'DKXE_HDSYYFZE_mean', 'WITHDRAW_GRZHYE_min', 'HDZYFZE_WITHDRAW_SUBMIT_mean', 'numsOf_GRYJCE_HDZYFZE_multi', 'HDZYFZE_GJJRATE_mean', 'GRJCJS_USAGE_AMOUNT_sum', 'DKXE_GRYJCE_std', 'GRYJCE_WITHDRAW_SUBMIT_std', 'HDZYF_GRZHDNGJYE_sum', 'numsOf_YEARPURINCM_WITHDRAW_multi', 'numsOf_GRJCJS_WITHDRAW_add', 'HDZYF_HDYF_min', 'HDZYFZE_GRZHDNGJYE_mean', 'DKXE_DKFFE_std', 'GRJCJS_GRYJCE_min', 'numsOf_TOP_USER1_DKXE_div', 'DKXE_USAGE_AMOUNT_mean', 'USAGE_RATE_GRZHYE_mean', 'HDZYF_DKYE_std', 'USAGE_AMOUNT_DKXE_std', 'numsOf_TOP_USER1_WITHDRAW_SUBMIT_div', 'numsOf_HDYF_TOP_USER_div', 'DKXE_DKYE_std', 'USAGE_AMOUNT_GRZHDNGJYE_std', 'GRJCJS_DKXE_mean', 'numsOf_HDZYFZE_USAGE_RATE_div', 'USAGE_AMOUNT_YEARPURINCM_mean', 'numsOf_USAGE_AMOUNT_WITHDRAW_SUBMIT_div', 'DKXE_USAGE_RATE_std', 'DKXE_GRZHYE_std', 'HDZYFZE_GJJRATE_std', 'DWJJLX_GJSIGN_count', 'HDZYFZE_GRJCJS_std', 'USAGE_AMOUNT_GRZHSNJZYE_min', 'GRJCJS_WITHDRAW_SUBMIT_mean', 'GRYJCE_DWSSHY_max', 'USAGE_AMOUNT_YEARPURINCM_sum', 'numsOf_GRZHSNJZYE_HDSYYF_multi', 'numsOf_YEARPURINCM_TOP_USER_multi', 'GRYJCE_GRZHSNJZYE_max', 'GRYJCE_TOP_USER_mean', 'GRZHDNGJYE_HDSYYFZE_max', 'USAGE_RATE_YEARPURINCM_sum', 'WITHDRAW_SUBMIT_DKXE_mean', 'numsOf_GRZHDNGJYE_WITHDRAW_multi', 'HDZYF_HDSYYF_std', 'USAGE_RATE_GRZHSNJZYE_mean', 'WITHDRAW_GRZHSNJZYE_min', 'GRYJCE_DKXE_sum', 'HDZYF_USAGE_RATE_mean', 'DKXE_YEARPURINCM_mean', 'DKXE_HDYF_mean', 'GRJCJS_GRZHDNGJYE_max', 'GRZHYE_USAGE_AMOUNT_max', 'GRYJCE_GRZHDNGJYE_std', 'numsOf_USAGE_RATE_WITHDRAW_SUBMIT_div', 'GRJCJS_HDSYYF_mean', 'GRYJCE_GRZHDNGJYE_max', 'GRZHDNGJYE_DKXE_min', 'USAGE_AMOUNT_HDSYYF_min', 'DKFFE_GJJRATE_std', 'numsOf_GRJCJS_GRZHSNJZYE_div', 'numsOf_YEARPURINCM_WITHDRAW_SUBMIT_multi', 'GRZHDNGJYE_YEARPURINCM_std', 'numsOf_GRZHYE_USAGE_AMOUNT_multi', 'GRYJCE_WITHDRAW_min', 'DKXE_GRZHYE_mean', 'numsOf_GRZHYE_HDSYYF_multi', 'DKXE_GRZHSNJZYE_max', 'numsOf_DKLL_DKXE_div', 'DKXE_GRZHYE_max', 'numsOf_GRZHSNJZYE_USAGE_AMOUNT_diff', 'DWSSHY_WITHSIGN2_count', 'numsOf_HDZYFZE_WITHDRAW_SUBMIT_div', 'HDZYF_DKXE_max', 'numsOf_GRZHYE_WITHDRAW_SUBMIT_div', 'DKXE_HDSYYFZE_max', 'numsOf_GRYJCE_HDYF_add', 'numsOf_HDYF_TOP_USER1_div', 'DKXE_USAGE_RATE_mean', 'DKXE_USAGE_AMOUNT_max', 'DWSSHY_YEARPURINCM_sum', 'HDSYYFZE_HDZYF_max', 'GRYJCE_GRZHYE_min', 'HDZYF_YEARPURINCM_std', 'GRZHYE_HDYF_min', 'HDSYYFZE_GRZHDNGJYE_max', 'numsOf_YEARPURINCM_WITHDRAW_add', 'numsOf_HDSYYF_TOP_USER_multi', 'GRZHDNGJYE_DKXE_max', 'numsOf_USAGE_RATE_DKXE_div', 'numsOf_GRZHYE_WITHDRAW_multi', 'numsOf_GRYJCE_TOP_USER_div', 'numsOf_HDSYYFZE_WITHDRAW_SUBMIT_div', 'USAGE_AMOUNT_WITHDRAW_SUBMIT_std', 'GRYJCE_GRZHYE_mean', 'numsOf_GJJRATE_WITHDRAW_SUBMIT_div', 'numsOf_USAGE_AMOUNT_DKXE_multi', 'GRYJCE_GRZHYE_max', 'numsOf_HDSYYFZE_WITHDRAW_SUBMIT_multi', 'HDZYF_GRZHSNJZYE_max', 'HDZYF_HDSYYFZE_max', 'DWSSHY_GJSIGN_count', 'numsOf_GRZHSNJZYE_TOP_USER_div', 'DWSSHY_HDYF_max', 'USAGE_AMOUNT_GRZHYE_std', 'DKFFE_DKLL_std', 'DKFFE_WITHDRAW_SUBMIT_std', 'numsOf_GJJRATE_USAGE_AMOUNT_multi', 'numsOf_YEARPURINCM_HDSYYF_multi', 'numsOf_GRZHSNJZYE_GJJRATE_multi', 'USAGE_RATE_HDZYF_std', 'GRYJCE_GRZHDNGJYE_mean', 'HDZYF_GRZHSNJZYE_std', 'numsOf_GRYJCE_HDZYF_div', 'GRZHYE_GRYJCE_max', 'GRJCJS_GRZHYE_min', 'GRYJCE_DWSSHY_min', 'USAGE_RATE_USAGE_AMOUNT_min', 'HDZYF_GRZHSNJZYE_min', 'GRJCJS_WITHDRAW_SUBMIT_max', 'DKYE_GRYJCE_std', 'HDZYFZE_HDYF_mean', 'DWSSHY_USAGE_RATE_std', 'GRZHYE_DKXE_max', 'DKXE_GRZHDNGJYE_sum', 'DKXE_DKLL_mean', 'USAGE_RATE_GRZHYE_std', 'DKXE_YEARPURINCM_std', 'numsOf_TOP_USER_WITHDRAW_SUBMIT_diff', 'USAGE_AMOUNT_YEARPURINCM_std', 'USAGE_RATE_GRZHDNGJYE_std', 'USAGE_AMOUNT_GRZHYE_max', 'GRJCJS_WITHDRAW_min', 'GRJCJS_WITHDRAW_max', 'numsOf_GRYJCE_WITHDRAW_SUBMIT_div', 'numsOf_GRZHYE_TOP_USER_div', 'numsOf_GRZHDNGJYE_TOP_USER_div', 'HDZYF_WITHDRAW_mean', 'DKXE_GRZHSNJZYE_std', 'HDZYF_WITHDRAW_min', 'WITHDRAW_GRYJCE_max', 'numsOf_GRZHYE_USAGE_AMOUNT_div', 'DKFFE_GJJRATE_mean', 'numsOf_GRYJCE_HDSYYFZE_multi', 'USAGE_AMOUNT_GRZHDNGJYE_mean', 'USAGE_AMOUNT_GRZHSNJZYE_mean', 'WITHDRAW_SUBMIT_GRYJCE_mean', 'DKXE_GRZHDNGJYE_max', 'numsOf_GRZHYE_HDZYF_div', 'numsOf_YEARPURINCM_HDYF_multi', 'HDZYF_HDSYYF_sum', 'WITHDRAW_HDSYYF_min', 'numsOf_GRZHYE_HDZYFZE_multi', 'numsOf_GRJCJS_USAGE_RATE_multi', 'GRJCJS_USAGE_AMOUNT_min', 'DWSSHY_HDSYYF_std', 'USAGE_RATE_HDYF_min', 'USAGE_AMOUNT_HDYF_std', 'USAGE_AMOUNT_HDSYYF_mean', 'numsOf_GRYJCE_TOP_USER1_multi', 'DKXE_TOP_USER_mean', 'numsOf_YEARPURINCM_WITHDRAW_diff', 'WITHDRAW_USAGE_AMOUNT_min', 'numsOf_GRZHSNJZYE_USAGE_AMOUNT_div', 'DWSSHY_DKLL_std', 'HDZYF_DKXE_std', 'numsOf_GRZHSNJZYE_DKXE_multi', 'HDSYYF_HDZYF_max', 'numsOf_GRZHDNGJYE_WITHDRAW_SUBMIT_div', 'numsOf_USAGE_RATE_DKXE_multi', 'numsOf_HDZYFZE_USAGE_AMOUNT_multi', 'GRZHDNGJYE_YEARPURINCM_mean', 'numsOf_HDZYF_HDYF_multi', 'DKXE_HDSYYF_min', 'DKXE_WITHDRAW_SUBMIT_mean', 'numsOf_GRYJCE_TOP_USER1_div', 'USAGE_RATE_GRJCJS_std', 'DKXE_GRZHDNGJYE_std', 'HDZYF_HDYF_sum', 'HDZYF_GRZHDNGJYE_std', 'numsOf_USAGE_RATE_GJJRATE_multi', 'HDZYF_GRZHSNJZYE_mean', 'numsOf_HDYF_DKXE_div', 'GRYJCE_WITHDRAW_SUBMIT_mean', 'XINGBIE_TOP_USER_count', 'DKXE_USAGE_AMOUNT_std', 'GRJCJS_USAGE_RATE_max', 'GRZHDNGJYE_DKXE_mean', 'GRYJCE_WITHDRAW_max', 'numsOf_TOP_USER_WITHDRAW_SUBMIT_div', 'GRZHYE_YEARPURINCM_max', 'USAGE_AMOUNT_GRYJCE_min', 'numsOf_HDSYYFZE_GJJRATE_multi', 'USAGE_RATE_GRZHSNJZYE_min', 'numsOf_YEARPURINCM_DKXE_div', 'USAGE_AMOUNT_GJJRATE_std', 'GRJCJS_USAGE_AMOUNT_max', 'WITHDRAW_SUBMIT_GRZHYE_min', 'GRJCJS_DKLL_mean', 'numsOf_GRZHSNJZYE_HDZYFZE_div', 'GRYJCE_HDSYYF_min', 'GRJCJS_WITHDRAW_SUBMIT_min', 'HDZYF_GRZHYE_std', 'DKXE_GJJRATE_std', 'DWSSHY_GRZHDNGJYE_mean', 'GRYJCE_WITHDRAW_mean', 'GRZHDNGJYE_HDYF_mean', 'HDZYF_HDYF_mean', 'GRZHDNGJYE_HDYF_min', 'numsOf_GRZHSNJZYE_USAGE_RATE_div', 'HDZYF_USAGE_AMOUNT_std', 'HDZYF_USAGE_AMOUNT_mean', 'GRJCJS_GJJRATE_mean', 'DKXE_GRZHSNJZYE_min', 'GRYJCE_GJJRATE_std', 'USAGE_AMOUNT_GRYJCE_max', 'DKXE_GRZHDNGJYE_mean', 'DKXE_HDYF_min', 'numsOf_GRJCJS_GRZHYE_div', 'numsOf_GRZHSNJZYE_GRYJCE_div', 'numsOf_HDYF_HDZYFZE_multi', 'GRJCJS_DWSSHY_min', 'numsOf_GRZHDNGJYE_TOP_USER_multi', 'GRYJCE_USAGE_AMOUNT_max', 'numsOf_TOP_USER_DKXE_div', 'DKXE_TOP_USER1_mean', 'DKXE_WITHDRAW_SUBMIT_std', 'numsOf_HDSYYFZE_GJJRATE_div', 'numsOf_GRZHSNJZYE_DKXE_div', 'WITHDRAW_SUBMIT_GRYJCE_max', 'HDSYYFZE_YEARPURINCM_max', 'USAGE_RATE_GRZHYE_min', 'HDSYYFZE_YEARPURINCM_sum', 'DKYE_GJJRATE_std', 'HDZYF_WITHDRAW_SUBMIT_sum', 'HDZYF_GRZHDNGJYE_max', 'GRYJCE_USAGE_RATE_max', 'GRZHDNGJYE_USAGE_RATE_max', 'DKXE_GJJRATE_mean', 'numsOf_HDSYYF_GJJRATE_div', 'USAGE_AMOUNT_GRZHYE_min', 'numsOf_GRZHSNJZYE_TOP_USER1_div', 'USAGE_AMOUNT_HDZYF_std', 'GRZHDNGJYE_GRZHSNJZYE_min', 'HDZYF_USAGE_AMOUNT_min', 'WITHDRAW_HDSYYF_mean', 'USAGE_RATE_DKXE_max', 'numsOf_HDSYYFZE_DKXE_div', 'numsOf_GRZHDNGJYE_DKXE_multi', 'HDZYF_USAGE_RATE_std', 'USAGE_RATE_YEARPURINCM_min', 'DKXE_HDSYYF_mean', 'numsOf_HDSYYF_HDYF_multi', 'HDZYF_USAGE_AMOUNT_sum', 'GRZHDNGJYE_GRJCJS_max', 'numsOf_HDSYYFZE_DKXE_multi', 'HDSYYFZE_YEARPURINCM_min', 'WITHDRAW_HDYF_min', 'numsOf_GRYJCE_GJJRATE_div', 'numsOf_GRZHSNJZYE_TOP_USER1_multi', 'numsOf_TOP_USER_USAGE_AMOUNT_div', 'HDZYF_WITHDRAW_SUBMIT_std', 'DKXE_GRYJCE_max', 'HDZYF_GRZHDNGJYE_mean', 'numsOf_GRJCJS_HDZYFZE_div', 'WITHDRAW_HDYF_mean', 'GRYJCE_HDZYFZE_min', 'GRYJCE_HDYF_min', 'USAGE_AMOUNT_YEARPURINCM_max', 'numsOf_YEARPURINCM_TOP_USER_div', 'numsOf_HDSYYF_TOP_USER1_multi', 'GRYJCE_USAGE_AMOUNT_min', 'DKXE_WITHDRAW_mean', 'numsOf_GRZHDNGJYE_HDSYYFZE_multi', 'HDZYF_YEARPURINCM_min', 'GRYJCE_GRJCJS_std', 'WITHDRAW_SUBMIT_YEARPURINCM_mean', 'HDZYF_YEARPURINCM_mean', 'HDZYF_GRZHYE_mean', 'numsOf_GJJRATE_USAGE_AMOUNT_div', 'numsOf_WITHDRAW_USAGE_AMOUNT_multi', 'numsOf_GRZHYE_DKXE_div', 'HDZYF_WITHDRAW_std', 'numsOf_GRZHYE_GJJRATE_multi', 'numsOf_GRZHSNJZYE_HDSYYFZE_multi', 'numsOf_GRZHSNJZYE_HDZYFZE_multi', 'GRYJCE_HDZYFZE_std', 'numsOf_USAGE_RATE_TOP_USER_multi', 'DKXE_WITHDRAW_sum', 'WITHDRAW_SUBMIT_GRZHSNJZYE_min', 'DKXE_HDSYYF_max', 'USAGE_RATE_GRYJCE_max', 'numsOf_GRZHDNGJYE_HDYF_multi', 'HDSYYFZE_USAGE_RATE_sum', 'GRYJCE_WITHDRAW_std', 'DKXE_WITHDRAW_min', 'USAGE_RATE_GRZHSNJZYE_std', 'USAGE_RATE_HDYF_mean', 'HDZYF_GRZHYE_max', 'numsOf_GRZHYE_HDSYYFZE_multi', 'DKXE_HDZYFZE_min', 'DKXE_YEARPURINCM_min', 'numsOf_GRZHYE_TOP_USER_multi', 'numsOf_GRZHYE_GRYJCE_div', 'numsOf_YEARPURINCM_USAGE_AMOUNT_div', 'GRJCJS_HDSYYFZE_min', 'GRJCJS_HDSYYF_min', 'numsOf_YEARPURINCM_HDSYYFZE_multi', 'numsOf_HDYF_HDSYYFZE_multi', 'DKXE_WITHDRAW_SUBMIT_max', 'numsOf_HDYF_HDZYFZE_div', 'numsOf_GRYJCE_USAGE_RATE_div', 'numsOf_HDYF_WITHDRAW_SUBMIT_multi', 'WITHDRAW_USAGE_AMOUNT_max', 'numsOf_DKLL_WITHDRAW_SUBMIT_div', 'GRYJCE_WITHDRAW_SUBMIT_min', 'numsOf_GRZHYE_YEARPURINCM_add', 'HDZYF_YEARPURINCM_max', 'DKXE_HDYF_std', 'numsOf_GRZHSNJZYE_TOP_USER_multi', 'GRYJCE_HDSYYFZE_min', 'numsOf_GRZHYE_WITHDRAW_SUBMIT_multi', 'GRZHSNJZYE_GRYJCE_max', 'USAGE_AMOUNT_DKXE_max', 'DWSSHY_GRJCJS_min', 'numsOf_HDYF_TOP_USER_multi', 'numsOf_HDSYYFZE_TOP_USER_multi', 'numsOf_USAGE_AMOUNT_WITHDRAW_SUBMIT_multi', 'numsOf_HDYF_USAGE_RATE_multi', 'numsOf_GRZHDNGJYE_TOP_USER1_div', 'DKXE_HDSYYFZE_min', 'GRYJCE_TOP_USER1_mean', 'numsOf_GRZHSNJZYE_USAGE_RATE_multi', 'WITHDRAW_SUBMIT_DKXE_min', 'USAGE_AMOUNT_GRZHYE_mean', 'numsOf_GRYJCE_TOP_USER_multi', 'numsOf_GRZHYE_TOP_USER1_multi', 'GRZHZT_USAGEOVER_count', 'GRYJCE_WITHDRAW_SUBMIT_sum', 'DKXE_GRZHYE_min', 'USAGE_AMOUNT_HDYF_min', 'GRYJCE_DKLL_mean', 'DKXE_WITHDRAW_SUBMIT_min', 'DKXE_HDSYYF_std', 'numsOf_GRZHSNJZYE_HDZYF_div', 'numsOf_GRZHDNGJYE_HDZYFZE_multi', 'GRYJCE_GJJRATE_mean', 'DKXE_HDYF_max', 'HDZYF_GRZHYE_min', 'numsOf_TOP_USER_WITHDRAW_SUBMIT_multi', 'GRJCJS_HDZYFZE_std', 'numsOf_GRJCJS_GRZHDNGJYE_add', 'GRJCJS_TOP_USER1_max', 'numsOf_GRZHSNJZYE_GJJRATE_div', 'DKXE_WITHDRAW_std', 'numsOf_YEARPURINCM_TOP_USER1_multi', 'DKXE_HDZYF_std', 'DKXE_USAGE_AMOUNT_min', 'DKXE_WITHDRAW_max', 'GRYJCE_DWJJLX_max', 'WITHDRAW_SUBMIT_GRZHSNJZYE_mean', 'USAGE_AMOUNT_GRYJCE_std', 'USAGE_AMOUNT_HDYF_mean', 'numsOf_HDYF_GJJRATE_div', 'GRYJCE_HDSYYFZE_std', 'DKXE_HDYF_sum', 'GRJCJS_HDSYYFZE_std', 'numsOf_WITHDRAW_WITHDRAW_SUBMIT_div', 'numsOf_YEARPURINCM_HDZYFZE_multi', 'numsOf_GRZHDNGJYE_USAGE_RATE_multi', 'DKXE_GRZHDNGJYE_min', 'numsOf_GRZHYE_YEARPURINCM_multi', 'WITHDRAW_SUBMIT_HDYF_mean', 'HDZYF_USAGE_RATE_sum', 'WITHDRAW_GRZHSNJZYE_mean', 'HDZYF_GRZHDNGJYE_min', 'GRJCJS_USAGE_RATE_min', 'WITHDRAW_SUBMIT_HDYF_min', 'DKXE_GRJCJS_std', 'numsOf_YEARPURINCM_GJJRATE_div', 'DKXE_USAGE_RATE_max', 'GRYJCE_USAGE_RATE_min', 'numsOf_TOP_USER_WITHDRAW_multi', 'GRJCJS_DWJJLX_max', 'USAGE_AMOUNT_GRJCJS_std', 'numsOf_YEARPURINCM_USAGE_RATE_multi', 
    #                                                                                                                                                'GRYJCE_WITHDRAW_SUBMIT_max']
#
#
#

In [127]:
# One-hot 编码
for f in tqdm(cate_cols):
    df[f] = df[f].map(dict(zip(df[f].unique(), range(df[f].nunique()))))
    df[f + '_count'] = df[f].map(df[f].value_counts())
    df = pd.concat([df,pd.get_dummies(df[f],prefix=f"{f}")],axis=1)

#离散型特征关联
cate_cols_combine = [[cate_cols[i], cate_cols[j]] for i in range(len(cate_cols)) \
                     for j in range(i + 1, len(cate_cols))]

#离散型联合计数占比
for f1, f2 in tqdm(cate_cols_combine):
    df['{}_{}_count'.format(f1, f2)] = df.groupby([f1, f2])['id'].transform('count')
    df['{}_in_{}_prop'.format(f1, f2)] = df['{}_{}_count'.format(f1, f2)] / df[f2 + '_count']
    df['{}_in_{}_prop'.format(f2, f1)] = df['{}_{}_count'.format(f1, f2)] / df[f1 + '_count']


#离散型变量下各个数值型指标统计
for f1 in tqdm(cate_cols + genFeats2 + genFeats3):
    g = df.groupby(f1)
    for f2 in num_cols + gen_feats:
        for stat in ['sum', 'mean', 'std', 'max', 'min', 'std']:
            if '{}_{}_{}'.format(f1, f2, stat) not in drop_feature:
                df['{}_{}_{}'.format(f1, f2, stat)] = g[f2].transform(stat)
for f1 in tqdm(cate_cols):
    for f3 in genFeats2 + genFeats3:
        for stat in ['sum', 'mean']:
            if '{}_{}_{}'.format(f1, f3, stat) not in drop_feature:
                df['{}_{}_{}'.format(f1, f3, stat)] = g[f3].transform(stat)

    
#连续型变量下各个数值特征指标
num_cols_gen_feats = num_cols + gen_feats
for f1 in tqdm(num_cols_gen_feats):
    g = df.groupby(f1)
    for f2 in num_cols_gen_feats:
        if f1 != f2:
            for stat in ['sum', 'mean', 'std', 'max', 'min', 'std']:
                if '{}_{}_{}'.format(f1, f2, stat) not in drop_feature:
                    df['{}_{}_{}'.format(f1, f2, stat)] = g[f2].transform(stat)

#特殊连续变量下各个数值特征指标
num_cols_gen_feats = num_cols + gen_feats
for f1 in tqdm(['GRJCJS','GRYJCE','DKFFE']):
    g = df.groupby(f1)
    for f2 in cate_cols0:
        if f1 != f2:
            for stat in ['max','min']:
                if '{}_{}_{}'.format(f1, f2, stat) not in drop_feature:
                    df['{}_{}_{}'.format(f1, f2, stat)] = g[f2].transform(stat)
                    
#连续性数值间的特征工程
for i in tqdm(range(len(num_cols_gen_feats))):
    for j in range(i + 1, len(num_cols_gen_feats)):
        if f'numsOf_{num_cols_gen_feats[i]}_{num_cols_gen_feats[j]}_add' not in drop_feature:
            df[f'numsOf_{num_cols_gen_feats[i]}_{num_cols_gen_feats[j]}_add'] = df[num_cols_gen_feats[i]] + df[num_cols_gen_feats[j]]
        if f'numsOf_{num_cols_gen_feats[i]}_{num_cols_gen_feats[j]}_diff'  not in drop_feature:
            df[f'numsOf_{num_cols_gen_feats[i]}_{num_cols_gen_feats[j]}_diff'] = df[num_cols_gen_feats[i]] - df[num_cols_gen_feats[j]]
        if f'numsOf_{num_cols_gen_feats[i]}_{num_cols_gen_feats[j]}_multi' not in drop_feature:
            df[f'numsOf_{num_cols_gen_feats[i]}_{num_cols_gen_feats[j]}_multi'] = df[num_cols_gen_feats[i]] * df[num_cols_gen_feats[j]]
        if f'numsOf_{num_cols_gen_feats[i]}_{num_cols_gen_feats[j]}_div' not in drop_feature:
            df[f'numsOf_{num_cols_gen_feats[i]}_{num_cols_gen_feats[j]}_div'] = df[num_cols_gen_feats[i]] / (df[num_cols_gen_feats[j]] + 0.0000000001)

100%|██████████| 22/22 [00:04<00:00,  4.56it/s]


训练集、测试集

In [128]:
train_df = df[df['label'].isna() == False].reset_index(drop=True)
test_df = df[df['label'].isna() == True].reset_index(drop=True)

display(train_df.shape, test_df.shape)

(39980, 925)

(15000, 925)

In [129]:
#2.利用相关系数，剔除具有强线性关系的指标
cols = [col for col in train_df.columns if col not in ['id', 'label'] + drop_feature]
to_delete = set()
print(len(cols))
for i in range(len(cols)):
    if i%100 == 0:
        print (f"{i} ")
    if i not in to_delete:
        for j in range(i+1,len(cols)):
            if j not in to_delete:
                if(abs(np.corrcoef(train_df[cols[i]].values,train_df[cols[j]].values))[0][1]>0.85):
                    to_delete.add(cols[sorted([i,j])[0]])
to_delete = list(to_delete)
print(f"{len(to_delete)} indics will be deleted !")

575
0 
100 
200 
300 
400 
500 
141 indics will be deleted !


In [130]:
print(to_delete)

['GRZHYE_GRZHSNJZYE_max', 'numsOf_GRZHDNGJYE_HDSYYFZE_multi', 'GRJCJS_GRZHDNGJYE_max', 'USAGE_RATE_GRZHSNJZYE_min', 'DKXE_GRZHYE_mean', 'DKXE_HDZYF_min', 'HDZYF_GRZHDNGJYE_mean', 'USAGE_RATE_GRZHYE_mean', 'DKXE_HDYF_sum', 'numsOf_DKLL_DKXE_div', 'HDZYF_WITHDRAW_mean', 'DKXE_USAGE_AMOUNT_mean', 'HDZYF_HDSYYF_sum', 'numsOf_HDYF_HDSYYFZE_multi', 'DKXE_HDSYYF_mean', 'HDZYF_GRZHYE_max', 'DKXE_GRZHSNJZYE_mean', 'GRJCJS_DWJJLX_max', 'numsOf_YEARPURINCM_TOP_USER_div', 'numsOf_GRYJCE_HDSYYFZE_multi', 'numsOf_YEARPURINCM_HDZYFZE_multi', 'GRZHDNGJYE_DKXE_min', 'HDZYF_USAGE_RATE_mean', 'numsOf_GRZHYE_HDZYFZE_multi', 'numsOf_YEARPURINCM_HDSYYFZE_multi', 'numsOf_GRYJCE_TOP_USER_div', 'DKXE_GRYJCE_min', 'numsOf_GRYJCE_GJJRATE_multi', 'GRYJCE_YEARPURINCM_sum', 'HDZYF_YEARPURINCM_mean', 'GRJCJS_USAGE_AMOUNT_mean', 'GRJCJS_GRZHYE_min', 'GRZHDNGJYE_DKXE_mean', 'numsOf_GRYJCE_HDZYFZE_multi', 'HDZYF_GRZHYE_mean', 'numsOf_DKYE_YEARPURINCM_div', 'USAGE_RATE_YEARPURINCM_sum', 'numsOf_HDZYF_HDYF_multi', 'GRJCJ

In [131]:
# 3.刷新有效指标
cols = [col for col in cols if col not in ['id', 'label'] + drop_feature + to_delete]
print(len(cols))

434


In [132]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, roc_auc_score

import time
from lightgbm.callback import reset_parameter,early_stopping
import math

In [133]:
def tpr_weight_funtion(y_true,y_predict):
    d = pd.DataFrame()
    d['prob'] = list(y_predict)
    d['y'] = list(y_true)
    d = d.sort_values(['prob'], ascending=[0])
    y = d.y
    PosAll = pd.Series(y).value_counts()[1]
    NegAll = pd.Series(y).value_counts()[0]
    pCumsum = d['y'].cumsum()
    nCumsum = np.arange(len(y)) - pCumsum + 1
    pCumsumPer = pCumsum / PosAll
    nCumsumPer = nCumsum / NegAll
    TR1 = pCumsumPer[abs(nCumsumPer-0.001).idxmin()]
    TR2 = pCumsumPer[abs(nCumsumPer-0.005).idxmin()]
    TR3 = pCumsumPer[abs(nCumsumPer-0.01).idxmin()]
    
    return 'TPR',0.4 * TR1 + 0.3 * TR2 + 0.3 * TR3,True


In [134]:
import random
from lightgbm.sklearn import LGBMClassifier
oof = np.zeros(train_df.shape[0])
feat_imp_df = pd.DataFrame({'feat': cols, 'imp': 0})
test_df['prob'] = 0
LOOPROUND = 4000
clf = LGBMClassifier(
    boosting_type = 'gbdt',
   #g learning_rate=0.03,  # 0.05 0.5786
    learning_rate=0.03,
    n_estimators=LOOPROUND,
    max_depth = 2,
    random_state=1023,
    subsample_for_bin=20000,
    tree = 'voting',
    is_unbalance=True,
   # reg_alpha=0.005,
    #reg_lambda=0.005,
    reg_alpha=0.005,
    reg_lambda=0.005,
    #bagging_freq = 20,
    #bagging_fraction = 0.8,
    feature_fraction = 0.8,
    min_data_in_leaf=10,
    #min_child_weight= 10,
    #min_split_gain = 0.1,
)

# 利用 call-back 函数，动态设置树深度，叶子节点数量以及学习率
reset_parameters = {
    'max_depth' : [ 2+(i//3) if 2+(i//3) <101 else 101  for i in range(LOOPROUND)  ],
        'num_leaves' : [ 2+(i//3) if 2+(i//3) <201 else 201 for i in range(LOOPROUND) ],
    'learning_rate' : [ 0.03 - (i//100)*0.005 if 0.03 - (i//100)*0.005>0.005 else 0.005 for i in range(LOOPROUND) ],
}


modellist = []
val_aucs = []
seeds = [ 1023 ]#,2048,23198,33224,132,34,657,876,7646,75457,456457,456,887,87]
for seed in seeds: 
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    for i, (trn_idx, val_idx) in enumerate(skf.split(train_df, train_df['label'])):
        print('--------------------- Fold: {} ---------------------'.format(i))
        t = time.time()
        trn_x, trn_y = train_df[cols].iloc[trn_idx].reset_index(drop=True), train_df['label'].values[trn_idx]
        val_x, val_y = train_df[cols].iloc[val_idx].reset_index(drop=True), train_df['label'].values[val_idx]
        print("验证集分布情况：{} positive of {}".format(int(sum(val_y)),len(val_y)))
        clf.fit(
            trn_x, trn_y,
            eval_set=[(val_x, val_y)],
            categorical_feature=list(set(cate_cols).intersection(cols)),
            eval_metric='binary_error',
            early_stopping_rounds = 200,
            verbose=200,
            callbacks = [reset_parameter(**reset_parameters)]
        )
        modellist.append(clf)
        feat_imp_df['imp'] += clf.feature_importances_ / skf.n_splits
        oof[val_idx] +=  clf.predict_proba(val_x)[:, 1] 
        test_df['prob'] += clf.predict_proba(test_df[cols])[:, 1] / skf.n_splits / len(seeds)

    cv_auc = roc_auc_score(train_df['label'], oof)
    val_aucs.append(cv_auc)
    print('\ncv_auc: ', cv_auc)
print(val_aucs, np.mean(val_aucs))

--------------------- Fold: 0 ---------------------
验证集分布情况：552 positive of 7997
Training until validation scores don't improve for 200 rounds.
[200]	valid_0's binary_error: 0.0802801
[400]	valid_0's binary_error: 0.0436414


KeyboardInterrupt: 

In [101]:
print(val_aucs, np.mean(val_aucs))
tpr = round(tpr_weight_funtion(train_df['label'], oof)[1], 6)
tpr, round(np.mean(val_aucs), 5)

[0.9385456850749775] 0.9385456850749775


(0.568299, 0.93855)

In [79]:
########################################
#一定要用全量数据再走一边，不然亏死，走一遍，分数提升1% ！！！
########################################
LOOPROUND = 2000
clf = LGBMClassifier(
    boosting_type = 'goss',
    learning_rate=0.03,
    n_estimators=LOOPROUND,
    max_depth = 2,
    random_state=1023,
    subsample_for_bin=20000,
    #metric='binary_error',
    is_unbalance=True,
    reg_alpha=0.005,
    reg_lambda=0.005,
    #bagging_freq = 20,
    #bagging_fraction = 0.8,
    feature_fraction = 0.8,
    min_data_in_leaf=10,
    #min_child_weight= 10,
    #min_split_gain = 0.1,
)

# 利用 call-back 函数，动态设置树深度，叶子节点数量以及学习率
reset_parameters = {
    'max_depth' : [ 2+(i//3) if 2+(i//3) <101 else 101  for i in range(LOOPROUND)  ],
        'num_leaves' : [ 2+(i//3) if 2+(i//3) <201 else 201 for i in range(LOOPROUND) ],
    'learning_rate' : [ 0.03 - (i//100)*0.005 if 0.03 - (i//100)*0.005>0.005 else 0.01 for i in range(LOOPROUND) ],
}

clf.fit(
    train_df[cols],train_df['label'],
            categorical_feature=list(set(cate_cols).intersection(cols)),
            eval_metric=tpr_weight_funtion,
            verbose=200,
            callbacks = [reset_parameter(**reset_parameters)]
        )
test_df['prob'] += clf.predict_proba(test_df[cols])[:, 1]

In [80]:
# 输出结果到提交
submit['id'] = test_df['id']
submit['label'] = test_df['prob']
submit = pd.merge(test,submit,on='id',how='left')[['id','label']]
submit['label'] = submit['label'].apply(lambda x: 0 if pd.isnull(x) else x)

submit.to_csv('../sub/submission{}_{}.csv'.format(tpr, round(np.mean(val_aucs), 6)), index = False)
submit.shape


(15000, 2)

In [52]:
#指标重要性分析
dImp = pd.DataFrame({
        'column': cols,
        'importance': [sum(i)/len(i) for i in np.array([ i.feature_importances_ for i in modellist ]).transpose()] ,
    }).sort_values(by='importance',ascending=False).reset_index()

In [57]:
print(dImp[dImp['importance']<400].sort_values('importance',ascending=False).column.values.tolist())

['HDZYFZE_DKXE_std', 'HDZYF_DKXE_sum', 'USAGE_AMOUNT_WITHDRAW_mean', 'GRJCJS_GRZHSNJZYE_min', 'GRYJCE_YEARPURINCM_mean', 'GRJCJS_WITHDRAW_mean', 'GRYJCE_DKXE_max', 'HDZYFZE_YEARPURINCM_std', 'DWJJLX_in_GJJRATE_prop', 'USAGE_AMOUNT_DKXE_min', 'DWSSHY_in_XINGBIE_prop', 'HDZYFZE_GRZHYE_std', 'HDZYFZE_USAGE_RATE_mean', 'numsOf_GRZHSNJZYE_USAGE_AMOUNT_multi', 'USAGE_AMOUNT_GRYJCE_mean', 'USAGE_AMOUNT_GRZHSNJZYE_std', 'DWSSHY_XINGBIE_count', 'GRYJCE_HDZYFZE_mean', 'HDZYFZE_YEARPURINCM_mean', 'DKXE_GRYJCE_min', 'DWSSHY_DKXE_max', 'GRYJCE_HDZYF_std', 'DKXE_DKLL_std', 'GRYJCE_YEARPURINCM_min', 'GRJCJS_GRZHDNGJYE_mean', 'numsOf_HDZYFZE_DKXE_multi', 'GRJCJS_USAGE_AMOUNT_mean', 'DWSSHY_in_WITHSIGN_prop', 'GRJCJS_TOP_USER1_mean', 'GRJCJS_DKXE_sum', 'USAGE_AMOUNT_HDYF_max', 'numsOf_GRZHDNGJYE_USAGE_AMOUNT_div', 'WITHDRAW_SUBMIT_DKXE_max', 'DWJJLX_WITHSIGN_count', 'USAGE_AMOUNT_WITHDRAW_std', 'USAGE_RATE_GJJRATE_std', 'GRZHYE_DKXE_sum', 'GRJCJS_USAGE_RATE_mean', 'numsOf_GRZHSNJZYE_YEARPURINCM_diff', 

In [28]:
dImp.to_csv('dImp.csv')